# Implementation of a low-temperature District Heating Network in different cities across Europe and the United States of America: a case study

In [ ]:
import pandas as pd
import numpy as np

import os

from bokeh.io import curdoc
from bokeh.layouts import row, layout
from bokeh.models import BasicTicker, BooleanFilter, CDSView, ColorBar, ColumnDataSource, FactorRange, GroupFilter, Legend, LinearColorMapper, LabelSet, PrintfTickFormatter, Span, LinearAxis, Range1d 
from bokeh.models import HoverTool
from bokeh.palettes import Spectral6, Spectral11, Spectral5, Viridis256, Colorblind, Magma256, Turbo256
from bokeh.plotting import figure, output_notebook, show
from bokeh.transform import factor_cmap, factor_mark, transform
from bokeh.themes import Theme
from IPython.display import display, Math
from scipy import optimize, stats
from math import exp, cos, sqrt, pi, sin, sqrt
from sklearn.metrics import mean_squared_error
from typing import List

from src.NOAA2010Dataset import NOAA2010Dataset
from src.InsPireDataset import InsPireDataset

output_notebook()

## USA Data

We acquired the data from the National Oceanic and Atmospheric Administration (NOAA). The data included information of different cities across the United States with respect to their latitude, longitude, stations, air temperature, humidity, and many more. We solely used the air temperature and selected four different cities, namely: 

- Miami: located in the state of Florida, South-East of the country.
- Fresno: located in the state of Florida, South-West of the country.
- Olympia: located in the state of Washington, North-West of the country.
- Rochester: located in the state of New York, North-East of the country.

Located on the map, they look like this:
    
<img src="./images/USA cities.png">  

## Europe Data

We acquired the data from the InsPire project. We solely used the air temperature and selected four different cities, namely: 

- Rome: located in Italy.
- Madrid: located in Spain.
- Stuttgart: located in Germany.
- London: located in the United Kingdom.

    


Located on the map, they look like this:

<img src="./images/Europe_cities.png">  

## Data Loading and Figure Properties

In [ ]:
noaa2010Dataset = NOAA2010Dataset()
insPireDataset = InsPireDataset()

AVAILABLE_DATASETS = dict()
AVAILABLE_DATASETS.update(noaa2010Dataset.load_processed_data())
AVAILABLE_DATASETS.update(insPireDataset.load_processed_data())
AVAILABLE_DATASETS.keys()


In [ ]:
miami = AVAILABLE_DATASETS[NOAA2010Dataset.MIAMI_FL].reset_index(drop=False)
fresno = AVAILABLE_DATASETS[NOAA2010Dataset.FRESNO_CA].reset_index(drop=False)
olympia = AVAILABLE_DATASETS[NOAA2010Dataset.OLYMPIA_WA].reset_index(drop=False)
rochester = AVAILABLE_DATASETS[NOAA2010Dataset.ROCHESTER_NY].reset_index(drop=False)

london = AVAILABLE_DATASETS[InsPireDataset.LONDON_UK].reset_index(drop=False)
madrid = AVAILABLE_DATASETS[InsPireDataset.MADRID_SPA].reset_index(drop=False)
rome = AVAILABLE_DATASETS[InsPireDataset.ROME_IT].reset_index(drop=False)
stuttgart = AVAILABLE_DATASETS[InsPireDataset.STUTTGART_GER].reset_index(drop=False)

In [ ]:
us_data = miami.append(other=fresno).append(other=olympia).append(other=rochester)
us_source = ColumnDataSource(us_data)

europe_data = london.append(other=madrid).append(other=rome).append(other=stuttgart)
europe_source = ColumnDataSource(europe_data)

In [ ]:
miami_view = CDSView(source=us_source, filters=[GroupFilter(column_name='city_key', group=NOAA2010Dataset.MIAMI_FL)])
fresno_view = CDSView(source=us_source, filters=[GroupFilter(column_name='city_key', group=NOAA2010Dataset.FRESNO_CA)])
olympia_view = CDSView(source=us_source, filters=[GroupFilter(column_name='city_key', group=NOAA2010Dataset.OLYMPIA_WA)])
rochester_view = CDSView(source=us_source, filters=[GroupFilter(column_name='city_key', group=NOAA2010Dataset.ROCHESTER_NY)])

miami_winter_view = CDSView(source=us_source, filters=[BooleanFilter((us_data.city_key == NOAA2010Dataset.MIAMI_FL) & (us_data.season == "winter"))])
fresno_winter_view = CDSView(source=us_source, filters=[BooleanFilter((us_data.city_key == NOAA2010Dataset.FRESNO_CA) & (us_data.season == "winter"))])
olympia_winter_view = CDSView(source=us_source, filters=[BooleanFilter((us_data.city_key == NOAA2010Dataset.OLYMPIA_WA) & (us_data.season == "winter"))])
rochester_winter_view = CDSView(source=us_source, filters=[BooleanFilter((us_data.city_key == NOAA2010Dataset.ROCHESTER_NY) & (us_data.season == "winter"))])

miami_spring_view = CDSView(source=us_source, filters=[BooleanFilter((us_data.city_key == NOAA2010Dataset.MIAMI_FL) & (us_data.season == "spring"))])
fresno_spring_view = CDSView(source=us_source, filters=[BooleanFilter((us_data.city_key == NOAA2010Dataset.FRESNO_CA) & (us_data.season == "spring"))])
olympia_spring_view = CDSView(source=us_source, filters=[BooleanFilter((us_data.city_key == NOAA2010Dataset.OLYMPIA_WA) & (us_data.season == "spring"))])
rochester_spring_view = CDSView(source=us_source, filters=[BooleanFilter((us_data.city_key == NOAA2010Dataset.ROCHESTER_NY) & (us_data.season == "spring"))])

miami_summer_view = CDSView(source=us_source, filters=[BooleanFilter((us_data.city_key == NOAA2010Dataset.MIAMI_FL) & (us_data.season == "summer"))])
fresno_summer_view = CDSView(source=us_source, filters=[BooleanFilter((us_data.city_key == NOAA2010Dataset.FRESNO_CA) & (us_data.season == "summer"))])
olympia_summer_view = CDSView(source=us_source, filters=[BooleanFilter((us_data.city_key == NOAA2010Dataset.OLYMPIA_WA) & (us_data.season == "summer"))])
rochester_summer_view = CDSView(source=us_source, filters=[BooleanFilter((us_data.city_key == NOAA2010Dataset.ROCHESTER_NY) & (us_data.season == "summer"))])

miami_fall_view = CDSView(source=us_source, filters=[BooleanFilter((us_data.city_key == NOAA2010Dataset.MIAMI_FL) & (us_data.season == "fall"))])
fresno_fall_view = CDSView(source=us_source, filters=[BooleanFilter((us_data.city_key == NOAA2010Dataset.FRESNO_CA) & (us_data.season == "fall"))])
olympia_fall_view = CDSView(source=us_source, filters=[BooleanFilter((us_data.city_key == NOAA2010Dataset.OLYMPIA_WA) & (us_data.season == "fall"))])
rochester_fall_view = CDSView(source=us_source, filters=[BooleanFilter((us_data.city_key == NOAA2010Dataset.ROCHESTER_NY) & (us_data.season == "fall"))])

london_view = CDSView(source=europe_source, filters=[GroupFilter(column_name='city_key', group=InsPireDataset.LONDON_UK)])
madrid_view = CDSView(source=europe_source, filters=[GroupFilter(column_name='city_key', group=InsPireDataset.MADRID_SPA)])
rome_view = CDSView(source=europe_source, filters=[GroupFilter(column_name='city_key', group=InsPireDataset.ROME_IT)])
stuttgart_view = CDSView(source=europe_source, filters=[GroupFilter(column_name='city_key', group=InsPireDataset.STUTTGART_GER)])

london_winter_view = CDSView(source=europe_source, filters=[BooleanFilter((europe_data.city_key == InsPireDataset.LONDON_UK) & (europe_data.season == "winter"))])
madrid_winter_view = CDSView(source=europe_source, filters=[BooleanFilter((europe_data.city_key == InsPireDataset.MADRID_SPA) & (europe_data.season == "winter"))])
rome_winter_view = CDSView(source=europe_source, filters=[BooleanFilter((europe_data.city_key == InsPireDataset.ROME_IT) & (europe_data.season == "winter"))])
stuttgart_winter_view = CDSView(source=europe_source, filters=[BooleanFilter((europe_data.city_key == InsPireDataset.STUTTGART_GER) & (europe_data.season == "winter"))])

london_spring_view = CDSView(source=europe_source, filters=[BooleanFilter((europe_data.city_key == InsPireDataset.LONDON_UK) & (europe_data.season == "spring"))])
madrid_spring_view = CDSView(source=europe_source, filters=[BooleanFilter((europe_data.city_key == InsPireDataset.MADRID_SPA) & (europe_data.season == "spring"))])
rome_spring_view = CDSView(source=europe_source, filters=[BooleanFilter((europe_data.city_key == InsPireDataset.ROME_IT) & (europe_data.season == "spring"))])
stuttgart_spring_view = CDSView(source=europe_source, filters=[BooleanFilter((europe_data.city_key == InsPireDataset.STUTTGART_GER) & (europe_data.season == "spring"))])

london_summer_view = CDSView(source=europe_source, filters=[BooleanFilter((europe_data.city_key == InsPireDataset.LONDON_UK) & (europe_data.season == "summer"))])
madrid_summer_view = CDSView(source=europe_source, filters=[BooleanFilter((europe_data.city_key == InsPireDataset.MADRID_SPA) & (europe_data.season == "summer"))])
rome_summer_view = CDSView(source=europe_source, filters=[BooleanFilter((europe_data.city_key == InsPireDataset.ROME_IT) & (europe_data.season == "summer"))])
stuttgart_summer_view = CDSView(source=europe_source, filters=[BooleanFilter((europe_data.city_key == InsPireDataset.STUTTGART_GER) & (europe_data.season == "summer"))])

london_fall_view = CDSView(source=europe_source, filters=[BooleanFilter((europe_data.city_key == InsPireDataset.LONDON_UK) & (europe_data.season == "fall"))])
madrid_fall_view = CDSView(source=europe_source, filters=[BooleanFilter((europe_data.city_key == InsPireDataset.MADRID_SPA) & (europe_data.season == "fall"))])
rome_fall_view = CDSView(source=europe_source, filters=[BooleanFilter((europe_data.city_key == InsPireDataset.ROME_IT) & (europe_data.season == "fall"))])
stuttgart_fall_view = CDSView(source=europe_source, filters=[BooleanFilter((europe_data.city_key == InsPireDataset.STUTTGART_GER) & (europe_data.season == "fall"))])


miami_2_weeks_winter_view = CDSView(source=us_source, filters=[BooleanFilter((us_data.city_key == NOAA2010Dataset.MIAMI_FL) & (us_data.timestamp >= "2010-01-10") & (us_data.timestamp <= "2010-01-24"))])
fresno_2_weeks_winter_view = CDSView(source=us_source, filters=[BooleanFilter((us_data.city_key == NOAA2010Dataset.FRESNO_CA) & (us_data.timestamp >= "2010-01-10") & (us_data.timestamp <= "2010-01-24"))])
olympia_2_weeks_winter_view = CDSView(source=us_source, filters=[BooleanFilter((us_data.city_key == NOAA2010Dataset.OLYMPIA_WA) & (us_data.timestamp >= "2010-01-10") & (us_data.timestamp <= "2010-01-24"))])
rochester_2_weeks_winter_view = CDSView(source=us_source, filters=[BooleanFilter((us_data.city_key == NOAA2010Dataset.ROCHESTER_NY) & (us_data.timestamp >= "2010-01-10") & (us_data.timestamp <= "2010-01-24"))])

miami_2_weeks_summer_view = CDSView(source=us_source, filters=[BooleanFilter((us_data.city_key == NOAA2010Dataset.MIAMI_FL) & (us_data.timestamp >= "2010-07-15") & (us_data.timestamp <= "2010-07-29"))])
fresno_2_weeks_summer_view = CDSView(source=us_source, filters=[BooleanFilter((us_data.city_key == NOAA2010Dataset.FRESNO_CA) & (us_data.timestamp >= "2010-07-15") & (us_data.timestamp <= "2010-07-29"))])
olympia_2_weeks_summer_view = CDSView(source=us_source, filters=[BooleanFilter((us_data.city_key == NOAA2010Dataset.OLYMPIA_WA) & (us_data.timestamp >= "2010-07-15") & (us_data.timestamp <= "2010-07-29"))])
rochester_2_weeks_summer_view = CDSView(source=us_source, filters=[BooleanFilter((us_data.city_key == NOAA2010Dataset.ROCHESTER_NY) & (us_data.timestamp >= "2010-07-15") & (us_data.timestamp <= "2010-07-29"))])


london_2_weeks_winter_view = CDSView(source=europe_source, filters=[BooleanFilter((europe_data.city_key == InsPireDataset.LONDON_UK) & (europe_data.timestamp >= "2017-01-10") & (europe_data.timestamp <= "2017-01-24"))])
madrid_2_weeks_winter_view = CDSView(source=europe_source, filters=[BooleanFilter((europe_data.city_key == InsPireDataset.MADRID_SPA) & (europe_data.timestamp >= "2017-01-10") & (europe_data.timestamp <= "2017-01-24"))])
rome_2_weeks_winter_view = CDSView(source=europe_source, filters=[BooleanFilter((europe_data.city_key == InsPireDataset.ROME_IT) & (europe_data.timestamp >= "2017-01-10") & (europe_data.timestamp <= "2017-01-24"))])
stuttgart_2_weeks_winter_view = CDSView(source=europe_source, filters=[BooleanFilter((europe_data.city_key == InsPireDataset.STUTTGART_GER) & (europe_data.timestamp >= "2017-01-10") & (europe_data.timestamp <= "2017-01-24"))])

london_2_weeks_summer_view = CDSView(source=europe_source, filters=[BooleanFilter((europe_data.city_key == InsPireDataset.LONDON_UK) & (europe_data.timestamp >= "2017-07-15") & (europe_data.timestamp <= "2017-07-29"))])
madrid_2_weeks_summer_view = CDSView(source=europe_source, filters=[BooleanFilter((europe_data.city_key == InsPireDataset.MADRID_SPA) & (europe_data.timestamp >= "2017-07-15") & (europe_data.timestamp <= "2017-07-29"))])
rome_2_weeks_summer_view = CDSView(source=europe_source, filters=[BooleanFilter((europe_data.city_key == InsPireDataset.ROME_IT) & (europe_data.timestamp >= "2017-07-15") & (europe_data.timestamp <= "2017-07-29"))])
stuttgart_2_weeks_summer_view = CDSView(source=europe_source, filters=[BooleanFilter((europe_data.city_key == InsPireDataset.STUTTGART_GER) & (europe_data.timestamp >= "2017-07-15") & (europe_data.timestamp <= "2017-07-29"))])



## Bokeh Properties

In [ ]:
color_map = {
    "miami": "#2c53d2", #navy
    "fresno": "#b30000", #red
    "olympia": "#00b300", #green
    "rochester": "#8296a5", #gray
    
    "london": "#2c53d2",  #navy
    "madrid": "#b30000", #red
    "rome": "#00b300", #green
    "stuttgart": "#8296a5", #gray
    
    "winter": '#2c53d2', #blue
    "spring": '#00b300', #green
    "summer": '#b30000', #red
    "fall":  '#ef9331', #orange
    
    "electricity": 'gold',#'#fffec6', #yellow
    "source_water": '#4a87b8', #blue
    "source1": '#e4744f', #orange
    "source2":  '#b5dba9', #green

    "RMSE": "gold" , #yellow
}

In [ ]:
# The width on these plots is equivalent to ms in datetime, i.e., width = 1 === width = 1ms
ONE_DAY_WIDTH = 1 * 24 * 60 * 60 * 1000

In [ ]:
FIGURE_TOOLS = {
    "toolbar_location": "above",
    "tools": "box_zoom,pan,undo,redo,reset,save,zoom_in,zoom_out",
    "plot_width": 700, 
    "plot_height": 400, 
}

FULL_WIDTH_FIGURE_TOOLS = {
    "plot_width": 1400, 
    "plot_height": 400,
}

FULL_HEIGHT_FIGURE_TOOLS = {
    "plot_width": 700, 
    "plot_height": 800,
}

FULL_FIGURE_TOOLS = {
    "plot_width": 1400, 
    "plot_height": 800,
}

print(FIGURE_TOOLS, FULL_WIDTH_FIGURE_TOOLS, FULL_HEIGHT_FIGURE_TOOLS,  FULL_FIGURE_TOOLS)

In [ ]:
curdoc().theme = Theme("./bokeh_theme.json")

In [ ]:
def heatmap(colorbar, *args, **kwargs):
    fig = figure(*args, **kwargs)
    fig.grid.visible = False
    fig.axis.axis_line_color = None
    fig.axis.major_tick_line_color = None
    fig.axis.major_label_standoff = 0
    fig.xaxis.major_label_orientation = 1.0
    fig.add_layout(color_bar, 'right')
    
    return fig

## Air Temperature

In [ ]:
date_hour_air_temp_hover_tool = HoverTool(tooltips=[('Date', '@date{%Y/%m/%d}'),
                                                    ("Hour", "@hour"),
                                                    ('Air Temperature', '@air_temp{%0.2f ºC}'),],
                                          formatters={'@date': 'datetime',
                                                      "@hour": "numeral",
                                                      '@air_temp': 'printf',},)

mapper = LinearColorMapper(palette=Spectral11, low=us_data["air_temp"].min(), high=us_data["air_temp"].max())
color_bar = ColorBar(color_mapper=mapper)

miami_air_temp_heatmap = heatmap(color_bar, title="Miami, FL - Air Temperature", x_axis_label="Day in year", y_axis_label="Hour of day", x_axis_type="datetime", **FIGURE_TOOLS)
miami_air_temp_heatmap.add_tools(date_hour_air_temp_hover_tool)
miami_air_temp_heatmap.rect(x="date", y="hour", width=ONE_DAY_WIDTH, height=1, source=us_source, view=miami_view, line_color=None, fill_color=transform('air_temp', mapper))

fresno_air_temp_heatmap = heatmap(color_bar, title="Fresno, CA - Air Temperature ", x_axis_label="Day in year", y_axis_label="Hour of day", x_axis_type="datetime", **FIGURE_TOOLS)
fresno_air_temp_heatmap.add_tools(date_hour_air_temp_hover_tool)
fresno_air_temp_heatmap.rect(x="date", y="hour", width=ONE_DAY_WIDTH, height=1, source=us_source, view=fresno_view, line_color=None, fill_color=transform('air_temp', mapper))

olympia_air_temp_heatmap = heatmap(color_bar, title="Olympia, WA - Air Temperature", x_axis_label="Day in year", y_axis_label="Hour of day", x_axis_type="datetime", **FIGURE_TOOLS)
olympia_air_temp_heatmap.add_tools(date_hour_air_temp_hover_tool)
olympia_air_temp_heatmap.rect(x="date", y="hour", width=ONE_DAY_WIDTH, height=1, source=us_source, view=olympia_view, line_color=None, fill_color=transform('air_temp', mapper))

rochester_air_temp_heatmap = heatmap(color_bar, title="Rochester, NY - Air Temperature ", x_axis_label="Day in year", y_axis_label="Hour of day", x_axis_type="datetime", **FIGURE_TOOLS)
rochester_air_temp_heatmap.add_tools(date_hour_air_temp_hover_tool)
rochester_air_temp_heatmap.rect(x="date", y="hour", width=ONE_DAY_WIDTH, height=1, source=us_source, view=rochester_view, line_color=None, fill_color=transform('air_temp', mapper))




In [ ]:
mapper = LinearColorMapper(palette=Spectral11, low=europe_data["air_temp"].min(), high=europe_data["air_temp"].max())
color_bar = ColorBar(color_mapper=mapper, location=(0, 0))

london_air_temp_heatmap = heatmap(color_bar, title="London, UK - Air Temperature", x_axis_label="Day in year", y_axis_label="Hour of day", x_axis_type="datetime", **FIGURE_TOOLS)
london_air_temp_heatmap.add_tools(date_hour_air_temp_hover_tool)
london_air_temp_heatmap.rect(x="date", y="hour", width=ONE_DAY_WIDTH, height=1, source=europe_source, view=london_view, line_color=None, fill_color=transform('air_temp', mapper))

sttutgart_air_temp_heatmap = heatmap(color_bar, title="Stuttgart, Germany - Air Temperature", x_axis_label="Day in year", y_axis_label="Hour of day", x_axis_type="datetime", **FIGURE_TOOLS)
sttutgart_air_temp_heatmap.add_tools(date_hour_air_temp_hover_tool)
sttutgart_air_temp_heatmap.rect(x="date", y="hour", width=ONE_DAY_WIDTH, height=1, source=europe_source, view=stuttgart_view, line_color=None, fill_color=transform('air_temp', mapper))

rome_air_temp_heatmap = heatmap(color_bar, title="Rome, Italy - Air Temperature", x_axis_label="Day in year", y_axis_label="Hour of day", x_axis_type="datetime", **FIGURE_TOOLS)
rome_air_temp_heatmap.add_tools(date_hour_air_temp_hover_tool)
rome_air_temp_heatmap.rect(x="date", y="hour", width=ONE_DAY_WIDTH, height=1, source=europe_source, view=rome_view, line_color=None, fill_color=transform('air_temp', mapper))

madrid_air_temp_heatmap = heatmap(color_bar, title="Madrid, Spain - Air Temperature", x_axis_label="Day in year", y_axis_label="Hour of day", x_axis_type="datetime", **FIGURE_TOOLS)
madrid_air_temp_heatmap.add_tools(date_hour_air_temp_hover_tool)
madrid_air_temp_heatmap.rect(x="date", y="hour", width=ONE_DAY_WIDTH, height=1, source=europe_source, view=madrid_view, line_color=None, fill_color=transform('air_temp', mapper))


### USA

In [ ]:
show(layout([[rochester_air_temp_heatmap, olympia_air_temp_heatmap,],[fresno_air_temp_heatmap, miami_air_temp_heatmap],])) 

### Europe

In [ ]:
show(layout([[london_air_temp_heatmap, sttutgart_air_temp_heatmap],[rome_air_temp_heatmap, madrid_air_temp_heatmap],]))

## Heat Demand


In [ ]:
date_hour_total_consumption_hover_tool = HoverTool(tooltips=[('Date', '@date{%Y/%m/%d}'),
                                                             ("Hour", "@hour"),
                                                             ('Total Heat', '@Total_consumption{%0.2f kW}'),],
                                                   formatters={'@date': 'datetime',
                                                               "@hour": "numeral",
                                                               '@Total_consumption': 'printf',},)

In [ ]:
mapper = LinearColorMapper(palette=Spectral11, low=us_data["Total_consumption"].min(), high=us_data["Total_consumption"].max())
color_bar = ColorBar(color_mapper=mapper, location=(0, 0))

rochester_total_consumption_heatmap = heatmap(color_bar, 
                                              x_axis_label="Days", 
                                              y_axis_label="Hour", 
                                              x_axis_type="datetime", 
                                              title="Rochester, NY - Heat Consumption over the year", 
                                              **FIGURE_TOOLS)
rochester_total_consumption_heatmap.rect(x="date", y="hour", width=ONE_DAY_WIDTH, height=1, source=us_source,view=rochester_view,line_color=None, fill_color=transform('Total_consumption', mapper))
rochester_total_consumption_heatmap.add_tools(date_hour_total_consumption_hover_tool)

olympia_total_consumption_heatmap = heatmap(color_bar, 
                                            x_axis_label="Days", 
                                            y_axis_label="Hour", 
                                            x_axis_type="datetime", 
                                            title="Olympia, WA - Heat Consumption over the year", 
                                            y_range=rochester_total_consumption_heatmap.y_range,
                                            x_range=rochester_total_consumption_heatmap.x_range,
                                            **FIGURE_TOOLS)
olympia_total_consumption_heatmap.rect(x="date", y="hour", width=ONE_DAY_WIDTH, height=1, source=us_source,view=olympia_view,line_color=None, fill_color=transform('Total_consumption', mapper))
olympia_total_consumption_heatmap.add_tools(date_hour_total_consumption_hover_tool)

fresno_total_consumption_heatmap = heatmap(color_bar, 
                                           x_axis_label="Days", 
                                           y_axis_label="Hour", 
                                           x_axis_type="datetime", 
                                           title="Fresno, CA - Heat Consumption over the year",
                                           y_range=rochester_total_consumption_heatmap.y_range,
                                           x_range=rochester_total_consumption_heatmap.x_range, 
                                           **FIGURE_TOOLS)
fresno_total_consumption_heatmap.rect(x="date", y="hour", width=ONE_DAY_WIDTH, height=1, source=us_source,view=fresno_view,line_color=None, fill_color=transform('Total_consumption', mapper))
fresno_total_consumption_heatmap.add_tools(date_hour_total_consumption_hover_tool)

miami_total_consumption_heatmap = heatmap(color_bar, 
                                          x_axis_label="Days", 
                                          y_axis_label="Hour", 
                                          x_axis_type="datetime", 
                                          title="Miami, FL - Heat Consumption over the year", 
                                          y_range=rochester_total_consumption_heatmap.y_range,
                                          x_range=rochester_total_consumption_heatmap.x_range,
                                          **FIGURE_TOOLS)
miami_total_consumption_heatmap.rect(x="date", y="hour", width=ONE_DAY_WIDTH, height=1, source=us_source,view=miami_view,line_color=None, fill_color=transform('Total_consumption', mapper))
miami_total_consumption_heatmap.add_tools(date_hour_total_consumption_hover_tool)



In [ ]:
mapper = LinearColorMapper(palette=Spectral11, low=europe_data["Total_consumption"].min(), high=europe_data["Total_consumption"].max())
color_bar = ColorBar(color_mapper=mapper, location=(0, 0))

london_total_consumption_heatmap = figure(x_axis_label="Days", 
                                          y_axis_label="Hour", 
                                          x_axis_type="datetime", 
                                          title="London, United Kingdom - Heat Consumption over the year", 
                                          **FIGURE_TOOLS)
london_total_consumption_heatmap.rect(x="date", y="hour", width=ONE_DAY_WIDTH, height=1, source=europe_source,view=london_view,line_color=None, fill_color=transform('Total_consumption', mapper))
london_total_consumption_heatmap.add_tools(date_hour_total_consumption_hover_tool)

madrid_total_consumption_heatmap = figure(x_axis_label="Days", 
                                          y_axis_label="Hour", 
                                          x_axis_type="datetime", 
                                          title="Madrid, Spain - Heat Consumption over the year", 
                                          y_range=london_total_consumption_heatmap.y_range,
                                          x_range=london_total_consumption_heatmap.x_range,
                                          **FIGURE_TOOLS)
madrid_total_consumption_heatmap.rect(x="date", y="hour", width=ONE_DAY_WIDTH, height=1, source=europe_source,view=madrid_view,line_color=None, fill_color=transform('Total_consumption', mapper))
madrid_total_consumption_heatmap.add_tools(date_hour_total_consumption_hover_tool)

rome_total_consumption_heatmap = figure(x_axis_label="Days", 
                                        y_axis_label="Hour", 
                                        x_axis_type="datetime", 
                                        title="Rome, Italy - Heat Consumption over the year",
                                        y_range=london_total_consumption_heatmap.y_range,
                                        x_range=london_total_consumption_heatmap.x_range, 
                                        **FIGURE_TOOLS)
rome_total_consumption_heatmap.rect(x="date", y="hour", width=ONE_DAY_WIDTH, height=1, source=europe_source,view=rome_view,line_color=None, fill_color=transform('Total_consumption', mapper))
rome_total_consumption_heatmap.add_tools(date_hour_total_consumption_hover_tool)

stuttgart_total_consumption_heatmap = figure(x_axis_label="Days", 
                                             y_axis_label="Hour", 
                                             x_axis_type="datetime", 
                                             title="Sttutgart, Germany - Heat Consumption over the year", 
                                             y_range=london_total_consumption_heatmap.y_range,
                                             x_range=london_total_consumption_heatmap.x_range,
                                             **FIGURE_TOOLS)
stuttgart_total_consumption_heatmap.rect(x="date", y="hour", width=ONE_DAY_WIDTH, height=1, source=europe_source,view=stuttgart_view,line_color=None, fill_color=transform('Total_consumption', mapper))
stuttgart_total_consumption_heatmap.add_tools(date_hour_total_consumption_hover_tool)



### USA

In [ ]:
show(layout([[rochester_total_consumption_heatmap, olympia_total_consumption_heatmap],[fresno_total_consumption_heatmap, miami_total_consumption_heatmap],]))

### Europe

In [ ]:
show(layout([[london_total_consumption_heatmap, stuttgart_total_consumption_heatmap],[rome_total_consumption_heatmap, madrid_total_consumption_heatmap],]))

## Load Duration Curve 



In a heat load duration (HLD) diagram the hourly or daily heat loads are sorted in descending order, with the highest thermal load of the year (winter peaks) located to the left and the lowest to the right (summer nights). The HLD diagram will be different annualy depending on the weather conditions, therefore for planning purposes it is recommended to use long-term duration diagrams based on historical data.

Source: S. Werner, “District Heating and Cooling,” Elsevier, 2013.

### USA

In [ ]:
rochester_data = us_data[(us_data["city_key"] == NOAA2010Dataset.ROCHESTER_NY)]["Total_consumption"]# / 1000
olympia_data = us_data[(us_data["city_key"] == NOAA2010Dataset.OLYMPIA_WA)]["Total_consumption"] #/ 1000
fresno_data = us_data[(us_data["city_key"] == NOAA2010Dataset.FRESNO_CA)]["Total_consumption"] #/ 1000
miami_data = us_data[(us_data["city_key"] == NOAA2010Dataset.MIAMI_FL)]["Total_consumption"] #/ 1000

x_usa = np.array(range(1, rochester_data.shape[0] + 1))
x_usa_max = max(x_usa)
x_usa_cumsum = x_usa * 100 / x_usa_max 

y_rochester = rochester_data.sort_values(ascending=False)
y_rochester_max = rochester_data.max()
y_rochester_norm = y_rochester * 100 / y_rochester_max


y_olympia = olympia_data.sort_values(ascending=False)
y_olympia_max = olympia_data.max()
y_olympia_norm = y_olympia * 100 / y_olympia_max


y_fresno = fresno_data.sort_values(ascending=False)
y_fresno_max = fresno_data.max()
y_fresno_norm = y_fresno * 100 / y_fresno_max


y_miami = miami_data.sort_values(ascending=False)
y_miami_max = miami_data.max()
y_miami_norm = y_miami * 100 / y_miami_max

max_across_usa = max(y_rochester_max, y_olympia_max, y_fresno_max, y_miami_max,)
y_rochester_norm_all =  y_rochester * 100 / max_across_usa
y_olympia_norm_all =  y_olympia * 100 / max_across_usa
y_fresno_norm_all =  y_fresno * 100 / max_across_usa
y_miami_norm_all =  y_miami * 100 / max_across_usa

In [ ]:
load_duration_curve_usa_y_equal_x_equal = figure(title="LDC for USA",
                                                 x_range=(0,9000),
                                                 y_axis_label="Thermal power [MW]",
                                                 x_axis_label="Hours in the year",
                                                 **{**FIGURE_TOOLS, **FULL_WIDTH_FIGURE_TOOLS})
l1 = load_duration_curve_usa_y_equal_x_equal.line(x=x_usa, y=y_rochester, color=color_map["rochester"], alpha=1, line_width=2.5)
l2 = load_duration_curve_usa_y_equal_x_equal.line(x=x_usa, y=y_olympia, color=color_map["olympia"], alpha=1, line_width=2.5)
l3 = load_duration_curve_usa_y_equal_x_equal.line(x=x_usa, y=y_fresno, color=color_map["fresno"], alpha=1, line_width=2.5)
l4 = load_duration_curve_usa_y_equal_x_equal.line(x=x_usa, y=y_miami, color=color_map["miami"], alpha=1, line_width=2.5)
load_duration_curve_usa_y_equal_x_equal.add_layout(Legend(items=[("Rochester", [l1]), ("Olympia", [l2]), ("Fresno", [l3]), ("Miami", [l4]),], ), "below")


load_duration_curve_usa_y_equal_x_norm = figure(title="LDC for USA",
                                                x_range=(0,105),
                                                y_axis_label="Thermal power [MW]",
                                                x_axis_label="Hours in the year [%]",
                                                **{**FIGURE_TOOLS, **FULL_WIDTH_FIGURE_TOOLS})
l1 = load_duration_curve_usa_y_equal_x_norm.line(x=x_usa_cumsum, y=y_rochester, color=color_map["rochester"], alpha=1, line_width=2.5)
l2 = load_duration_curve_usa_y_equal_x_norm.line(x=x_usa_cumsum, y=y_olympia, color=color_map["olympia"], alpha=1, line_width=2.5)
l3 = load_duration_curve_usa_y_equal_x_norm.line(x=x_usa_cumsum, y=y_fresno, color=color_map["fresno"], alpha=1, line_width=2.5)
l4 = load_duration_curve_usa_y_equal_x_norm.line(x=x_usa_cumsum, y=y_miami, color=color_map["miami"], alpha=1, line_width=2.5)
load_duration_curve_usa_y_equal_x_norm.add_layout(Legend(items=[("Rochester", [l1]), ("Olympia", [l2]), ("Fresno", [l3]), ("Miami", [l4]),], ), "below")

load_duration_curve_usa_y_norm_x_equal = figure(title="LDC for USA - Normalized across cities",
                                                x_range=(0,9000),
                                                y_axis_label="Thermal power [%]",
                                                x_axis_label="Hours in the year",
                                                **{**FIGURE_TOOLS, **FULL_WIDTH_FIGURE_TOOLS})
l1 = load_duration_curve_usa_y_norm_x_equal.line(x=x_usa, y=y_rochester_norm_all, color=color_map["rochester"], alpha=1, line_width=2.5)
l2 = load_duration_curve_usa_y_norm_x_equal.line(x=x_usa, y=y_olympia_norm_all, color=color_map["olympia"], alpha=1, line_width=2.5)
l3 = load_duration_curve_usa_y_norm_x_equal.line(x=x_usa, y=y_fresno_norm_all, color=color_map["fresno"], alpha=1, line_width=2.5)
l4 = load_duration_curve_usa_y_norm_x_equal.line(x=x_usa, y=y_miami_norm_all, color=color_map["miami"], alpha=1, line_width=2.5)
load_duration_curve_usa_y_norm_x_equal.add_layout(Legend(items=[("Rochester Norm", [l1]), ("Olympia Norm", [l2]), ("Fresno Norm", [l3]), ("Miami Norm", [l4]),], ), "below")

load_duration_curve_usa_y_norm_x_norm = figure(title="LDC for USA - Normalized across cities",
                                               x_range=(0,105),
                                               y_axis_label="Thermal power [%]",
                                               x_axis_label="Hours in the year [%]",
                                               **{**FIGURE_TOOLS, **FULL_WIDTH_FIGURE_TOOLS})
l1 = load_duration_curve_usa_y_norm_x_norm.line(x=x_usa_cumsum, y=y_rochester_norm_all, color=color_map["rochester"], alpha=1, line_width=2.5)
l2 = load_duration_curve_usa_y_norm_x_norm.line(x=x_usa_cumsum, y=y_olympia_norm_all, color=color_map["olympia"], alpha=1, line_width=2.5)
l3 = load_duration_curve_usa_y_norm_x_norm.line(x=x_usa_cumsum, y=y_fresno_norm_all, color=color_map["fresno"], alpha=1, line_width=2.5)
l4 = load_duration_curve_usa_y_norm_x_norm.line(x=x_usa_cumsum, y=y_miami_norm_all, color=color_map["miami"], alpha=1, line_width=2.5)
load_duration_curve_usa_y_norm_x_norm.add_layout(Legend(items=[("Rochester Norm", [l1]), ("Olympia Norm", [l2]), ("Fresno Norm", [l3]), ("Miami Norm", [l4]),], ), "below")


load_duration_curve_usa_norm_y_norm_x_equal = figure(title="LDC for USA - Normalization per city",
                                                     x_range=(0,9000),
                                                     y_axis_label="Thermal power [%]",
                                                     x_axis_label="Hours in the year",
                                                     **{**FIGURE_TOOLS, **FULL_WIDTH_FIGURE_TOOLS})
l1 = load_duration_curve_usa_norm_y_norm_x_equal.line(x=x_usa, y=y_rochester_norm, color=color_map["rochester"], alpha=1, line_width=2.5)
l2 = load_duration_curve_usa_norm_y_norm_x_equal.line(x=x_usa, y=y_olympia_norm, color=color_map["olympia"], alpha=1, line_width=2.5)
l3 = load_duration_curve_usa_norm_y_norm_x_equal.line(x=x_usa, y=y_fresno_norm, color=color_map["fresno"], alpha=1, line_width=2.5)
l4 = load_duration_curve_usa_norm_y_norm_x_equal.line(x=x_usa, y=y_miami_norm, color=color_map["miami"], alpha=1, line_width=2.5)
load_duration_curve_usa_norm_y_norm_x_equal.add_layout(Legend(items=[("Rochester Norm", [l1]), ("Olympia Norm", [l2]), ("Fresno Norm", [l3]), ("Miami Norm", [l4]),], ), "below")


load_duration_curve_usa_norm_y_norm_x_norm = figure(title="LDC for USA - Normalization per city",
                                                    x_range=(0,105),
                                                    y_axis_label="Thermal power [%]",
                                                    x_axis_label="Hours in the year [%]",
                                                    **{**FIGURE_TOOLS, **FULL_WIDTH_FIGURE_TOOLS})
l1 = load_duration_curve_usa_norm_y_norm_x_norm.line(x=x_usa_cumsum, y=y_rochester_norm, color=color_map["rochester"], alpha=1, line_width=2.5)
l2 = load_duration_curve_usa_norm_y_norm_x_norm.line(x=x_usa_cumsum, y=y_olympia_norm, color=color_map["olympia"], alpha=1, line_width=2.5)
l3 = load_duration_curve_usa_norm_y_norm_x_norm.line(x=x_usa_cumsum, y=y_fresno_norm, color=color_map["fresno"], alpha=1, line_width=2.5)
l4 = load_duration_curve_usa_norm_y_norm_x_norm.line(x=x_usa_cumsum, y=y_miami_norm, color=color_map["miami"], alpha=1, line_width=2.5)
load_duration_curve_usa_norm_y_norm_x_norm.add_layout(Legend(items=[("Rochester Norm", [l1]), ("Olympia Norm", [l2]), ("Fresno Norm", [l3]), ("Miami Norm", [l4]),], ), "below")



In [ ]:
show(layout([[load_duration_curve_usa_y_equal_x_equal],[load_duration_curve_usa_norm_y_norm_x_equal], ]))

### Europe

In [ ]:
london_data = europe_data[(europe_data["city_key"] == InsPireDataset.LONDON_UK)]["Total_consumption"]# /1000
stuttgart_data = europe_data[(europe_data["city_key"] == InsPireDataset.STUTTGART_GER)]["Total_consumption"]#/1000
rome_data = europe_data[(europe_data["city_key"] == InsPireDataset.ROME_IT)]["Total_consumption"]#/1000
madrid_data = europe_data[(europe_data["city_key"] == InsPireDataset.MADRID_SPA)]["Total_consumption"]#/1000

x_europe = np.array(range(1,london_data.shape[0] + 1))
x_europe_max = max(x_europe)
x_europe_cumsum = x_europe * 100 / x_europe_max 

y_london = london_data.sort_values(ascending=False)
y_london_max = london_data.max()
y_london_norm = y_london * 100 / y_london_max


y_stuttgart = stuttgart_data.sort_values(ascending=False)
y_stuttgart_max = stuttgart_data.max()
y_stuttgart_norm = y_stuttgart * 100 / y_stuttgart_max


y_rome = rome_data.sort_values(ascending=False)
y_rome_max = rome_data.max()
y_rome_norm = y_rome * 100 / y_rome_max


y_madrid = madrid_data.sort_values(ascending=False)
y_madrid_max = madrid_data.max()
y_madrid_norm = y_madrid * 100 / y_madrid_max

max_across_europe = max(y_london_max, y_stuttgart_max, y_rome_max, y_madrid_max,)
y_london_norm_all =  y_london * 100 / max_across_europe
y_stuttgart_norm_all =  y_stuttgart * 100 / max_across_europe
y_rome_norm_all =  y_rome * 100 / max_across_europe
y_madrid_norm_all =  y_madrid * 100 / max_across_europe

In [ ]:
load_duration_curve_europe_y_equal_x_equal = figure(title="LDC for Europe",
                                                    x_range=(0,9000),
                                                    y_axis_label="Thermal power [MW]",
                                                    x_axis_label="Hours in the year",
                                                    **{**FIGURE_TOOLS, **FULL_WIDTH_FIGURE_TOOLS})
l1 = load_duration_curve_europe_y_equal_x_equal.line(x=x_europe, y=y_london, color=color_map["london"], alpha=1, line_width=2.5)
l2 = load_duration_curve_europe_y_equal_x_equal.line(x=x_europe, y=y_stuttgart, color=color_map["stuttgart"], alpha=1, line_width=2.5)
l3 = load_duration_curve_europe_y_equal_x_equal.line(x=x_europe, y=y_rome, color=color_map["rome"], alpha=1, line_width=2.5)
l4 = load_duration_curve_europe_y_equal_x_equal.line(x=x_europe, y=y_madrid, color=color_map["madrid"], alpha=1, line_width=2.5)
load_duration_curve_europe_y_equal_x_equal.add_layout(Legend(items=[("London", [l1]), ("Stuttgart", [l2]), ("Rome", [l3]), ("Madrid", [l4]),], ), "below")

load_duration_curve_europe_y_equal_x_norm = figure(title="LDC for Europe",
                                                   x_range=(0,105),
                                                   y_axis_label="Thermal power [MW]",
                                                   x_axis_label="Hours in the year [%]",
                                                   **{**FIGURE_TOOLS, **FULL_WIDTH_FIGURE_TOOLS})
l1 = load_duration_curve_europe_y_equal_x_norm.line(x=x_europe_cumsum, y=y_london, color=color_map["london"], alpha=1, line_width=2.5)
l2 = load_duration_curve_europe_y_equal_x_norm.line(x=x_europe_cumsum, y=y_stuttgart, color=color_map["stuttgart"], alpha=1, line_width=2.5)
l3 = load_duration_curve_europe_y_equal_x_norm.line(x=x_europe_cumsum, y=y_rome, color=color_map["rome"], alpha=1, line_width=2.5)
l4 = load_duration_curve_europe_y_equal_x_norm.line(x=x_europe_cumsum, y=y_madrid, color=color_map["madrid"], alpha=1, line_width=2.5)
load_duration_curve_europe_y_equal_x_norm.add_layout(Legend(items=[("London", [l1]), ("Stuttgart", [l2]), ("Rome", [l3]), ("Madrid", [l4]),], ), "below")

load_duration_curve_europe_y_norm_x_equal = figure(title="LDC for Europe - Normalized between cities",
                                                   x_range=(0,9000),
                                                   y_axis_label="Thermal power [%]",
                                                   x_axis_label="Hours in the year",
                                                   **{**FIGURE_TOOLS, **FULL_WIDTH_FIGURE_TOOLS})
l1 = load_duration_curve_europe_y_norm_x_equal.line(x=x_europe, y=y_london_norm_all, color=color_map["london"], alpha=1, line_width=2.5)
l2 = load_duration_curve_europe_y_norm_x_equal.line(x=x_europe, y=y_stuttgart_norm_all, color=color_map["stuttgart"], alpha=1, line_width=2.5)
l3 = load_duration_curve_europe_y_norm_x_equal.line(x=x_europe, y=y_rome_norm_all, color=color_map["rome"], alpha=1, line_width=2.5)
l4 = load_duration_curve_europe_y_norm_x_equal.line(x=x_europe, y=y_madrid_norm_all, color=color_map["madrid"], alpha=1, line_width=2.5)
load_duration_curve_europe_y_norm_x_equal.add_layout(Legend(items=[("London Norm", [l1]), ("Stuttgart Norm", [l2]), ("Rome Norm", [l3]), ("Madrid Norm", [l4]),], ), "below")

load_duration_curve_europe_y_norm_x_norm = figure(title="LDC for Europe - Normalized between cities",
                                                  x_range=(0,105),
                                                  y_axis_label="Thermal power [%]",
                                                  x_axis_label="Hours in the year [%]",
                                                  **{**FIGURE_TOOLS, **FULL_WIDTH_FIGURE_TOOLS})
l1 = load_duration_curve_europe_y_norm_x_norm.line(x=x_europe_cumsum, y=y_london_norm_all, color=color_map["london"], alpha=1, line_width=2.5)
l2 = load_duration_curve_europe_y_norm_x_norm.line(x=x_europe_cumsum, y=y_stuttgart_norm_all, color=color_map["stuttgart"], alpha=1, line_width=2.5)
l3 = load_duration_curve_europe_y_norm_x_norm.line(x=x_europe_cumsum, y=y_rome_norm_all, color=color_map["rome"], alpha=1, line_width=2.5)
l4 = load_duration_curve_europe_y_norm_x_norm.line(x=x_europe_cumsum, y=y_madrid_norm_all, color=color_map["madrid"], alpha=1, line_width=2.5)
load_duration_curve_europe_y_norm_x_norm.add_layout(Legend(items=[("London Norm", [l1]), ("Stuttgart Norm", [l2]), ("Rome Norm", [l3]), ("Madrid Norm", [l4]),], ), "below")

load_duration_curve_europe_norm_y_norm_x_equal = figure(title="LDC for Europe - Each city is normalized",
                                                        x_range=(0,9000),
                                                        y_axis_label="Thermal power [%]",
                                                        x_axis_label="Hours in the year",
                                                        **{**FIGURE_TOOLS, **FULL_WIDTH_FIGURE_TOOLS})
l1 = load_duration_curve_europe_norm_y_norm_x_equal.line(x=x_europe, y=y_london_norm, color=color_map["london"], alpha=1, line_width=2.5)
l2 = load_duration_curve_europe_norm_y_norm_x_equal.line(x=x_europe, y=y_stuttgart_norm, color=color_map["stuttgart"], alpha=1, line_width=2.5)
l3 = load_duration_curve_europe_norm_y_norm_x_equal.line(x=x_europe, y=y_rome_norm, color=color_map["rome"], alpha=1, line_width=2.5)
l4 = load_duration_curve_europe_norm_y_norm_x_equal.line(x=x_europe, y=y_madrid_norm, color=color_map["madrid"], alpha=1, line_width=2.5)
load_duration_curve_europe_norm_y_norm_x_equal.add_layout(Legend(items=[("London Norm", [l1]), ("Stuttgart Norm", [l2]), ("Rome Norm", [l3]), ("Madrid Norm", [l4]),], ), "below")

load_duration_curve_europe_norm_y_norm_x_norm = figure(title="LDC for Europe - Each city is normalized",
                                                       x_range=(0,105),
                                                       y_axis_label="Thermal power [%]",
                                                       x_axis_label="Hours in the year",
                                                       **{**FIGURE_TOOLS, **FULL_WIDTH_FIGURE_TOOLS})
l1 = load_duration_curve_europe_norm_y_norm_x_norm.line(x=x_europe_cumsum, y=y_london_norm, color=color_map["london"], alpha=1, line_width=2.5)
l2 = load_duration_curve_europe_norm_y_norm_x_norm.line(x=x_europe_cumsum, y=y_stuttgart_norm, color=color_map["stuttgart"], alpha=1, line_width=2.5)
l3 = load_duration_curve_europe_norm_y_norm_x_norm.line(x=x_europe_cumsum, y=y_rome_norm, color=color_map["rome"], alpha=1, line_width=2.5)
l4 = load_duration_curve_europe_norm_y_norm_x_norm.line(x=x_europe_cumsum, y=y_madrid_norm, color=color_map["madrid"], alpha=1, line_width=2.5)
load_duration_curve_europe_norm_y_norm_x_norm.add_layout(Legend(items=[("London Norm", [l1]), ("Stuttgart Norm", [l2]), ("Rome Norm", [l3]), ("Madrid Norm", [l4]),], ), "below")



In [ ]:
show(layout([[load_duration_curve_europe_y_equal_x_equal],[load_duration_curve_europe_norm_y_norm_x_equal],]))

## Heat power signature


The weather dependence of the heat load is shown using a system heat power signature (HPS). A HPS is generated combining the values of thermal demand with the corresponding outdoor temperatures. In this plot we present hourly data, showing a clear linear interdependence between the thermal load and outdoor temperatures for space heating. 

It is expected that solar gains through windows are present between 8 - 15°C, resulting in a smaller heat load than the line representing the HPS. This is a real-world explanation for using the heating degree day threshold temperature when calculating degree-days.

Source: S. Werner, “District Heating and Cooling,” Elsevier, 2013.

In [ ]:
heat_power_signature_tool = HoverTool(tooltips=[('Air Temperature', '@air_temp{%0.2f ºC}'),
                                                ('Total Heat', '@Total_consumption{%0.2f kW}'),
                                                ("Adjusted Heat", '@Total_consumption_fit{%0.2f kW}')],
                                      formatters={'@air_temp': 'printf',
                                                  '@Total_consumption': 'printf',
                                                  '@Total_consumption_fit': 'printf',},)

all_heat_power_signature_tool = HoverTool(tooltips=[('Air Temperature', '@air_temp{%0.2f ºC}'),
                                                ('Total Heat', '@Total_consumption{%0.2f kW}'),],
                                      formatters={'@air_temp': 'printf',
                                                  '@Total_consumption': 'printf',},)

### USA

In [ ]:
miami_heat_power_signature = figure(title="Miami, Florida",
                                    y_axis_label="Thermal Power [kW]",
                                    x_axis_label="Outdoor Temperature [ºC]",
                                    **FIGURE_TOOLS
                                    )
miami_heat_power_signature.add_tools(heat_power_signature_tool)
l1 = miami_heat_power_signature.circle(x="air_temp", y="Total_consumption", color=color_map["miami"], alpha=0.2, source=us_source, view=miami_view)
l2 = miami_heat_power_signature.line(x="air_temp", y="Total_consumption_fit", color=color_map["RMSE"], alpha=1, line_width=2.5, source=us_source, view=miami_view)
miami_heat_power_signature.add_layout(Legend(items=[("Total Heat [kW]", [l1]), ("RMSE [kW]", [l2]),], ), "below")


fresno_heat_power_signature = figure(title="Fresno, California",
                                     y_range=miami_heat_power_signature.y_range,
                                     x_range=miami_heat_power_signature.x_range,
                                     y_axis_label="Thermal Power [kW]",
                                     x_axis_label="Outdoor Temperature [ºC]",
                                     **FIGURE_TOOLS
                                    )
fresno_heat_power_signature.add_tools(heat_power_signature_tool)
l1 = fresno_heat_power_signature.circle(x="air_temp", y="Total_consumption", color=color_map["fresno"], alpha=0.2, source=us_source, view=fresno_view)
l2 = fresno_heat_power_signature.line(x="air_temp", y="Total_consumption_fit", color=color_map["RMSE"], alpha=1, line_width=2.5, source=us_source, view=fresno_view)
fresno_heat_power_signature.add_layout(Legend(items=[("Total Heat [kW]", [l1]), ("RMSE [kW]", [l2]),], ), "below")


olympia_heat_power_signature = figure(title="Olympia, Washington",
                                      y_range=miami_heat_power_signature.y_range,
                                      x_range=miami_heat_power_signature.x_range,
                                      y_axis_label="Thermal Power [kW]",
                                      x_axis_label="Outdoor Temperature [ºC]",
                                      **FIGURE_TOOLS
                                     )
olympia_heat_power_signature.add_tools(heat_power_signature_tool)
l1 = olympia_heat_power_signature.circle(x="air_temp", y="Total_consumption", color=color_map["olympia"], alpha=0.2, source=us_source, view=olympia_view)
l2 = olympia_heat_power_signature.line(x="air_temp", y="Total_consumption_fit", color=color_map["RMSE"], alpha=1, line_width=2.5, source=us_source, view=olympia_view)
olympia_heat_power_signature.add_layout(Legend(items=[("Total Heat [kW]", [l1]), ("RMSE [kW]", [l2]),], ),  "below")


rochester_heat_power_signature = figure(title="Rochester, New York",
                                        y_range=miami_heat_power_signature.y_range,
                                        x_range=miami_heat_power_signature.x_range,
                                        y_axis_label="Thermal Power [kW]",
                                        x_axis_label="Outdoor Temperature [ºC]",
                                        **FIGURE_TOOLS
                                       )
rochester_heat_power_signature.add_tools(heat_power_signature_tool)
l1 = rochester_heat_power_signature.circle(x="air_temp", y="Total_consumption", color=color_map["rochester"], alpha=0.2, source=us_source, view=rochester_view)
l2 = rochester_heat_power_signature.line(x="air_temp", y="Total_consumption_fit", color=color_map["RMSE"], alpha=1, line_width=2.5, source=us_source, view=rochester_view)
rochester_heat_power_signature.add_layout(Legend(items=[("Total Heat [kW]", [l1]), ("RMSE [kW]", [l2]),], ),  "below")


all_usa_heat_power_signature = figure(title="Total Heat based on outdoor temperature",
                                      y_axis_label="Thermal Power [kW]",
                                      x_axis_label="Outdoor Temperature [ºC]",
                                      **{**FIGURE_TOOLS, **FULL_WIDTH_FIGURE_TOOLS})
all_usa_heat_power_signature.add_tools(all_heat_power_signature_tool)
l1 = all_usa_heat_power_signature.circle(x="air_temp", y="Total_consumption", color=color_map["fresno"], alpha=0.2, source=us_source, view=fresno_view)
l2 = all_usa_heat_power_signature.circle(x="air_temp", y="Total_consumption", color=color_map["olympia"], alpha=0.2, source=us_source, view=olympia_view)
l3 = all_usa_heat_power_signature.circle(x="air_temp", y="Total_consumption", color=color_map["rochester"], alpha=0.2, source=us_source, view=rochester_view)
l4 = all_usa_heat_power_signature.circle(x="air_temp", y="Total_consumption", color=color_map["miami"], alpha=0.2, source=us_source, view=miami_view)
all_usa_heat_power_signature.add_layout(Legend(items=[("Fresno, CA", [l1]), ("Olympia, WA", [l2]), ("Rochester, NY", [l3]), ("Miami, FL", [l4]),], ), "below")



In [ ]:
show(layout([[all_usa_heat_power_signature],[rochester_heat_power_signature, olympia_heat_power_signature, ],[fresno_heat_power_signature, miami_heat_power_signature],]))

### Europe

In [ ]:
london_heat_power_signature = figure(title="London, United Kingdom",
                                     y_axis_label="Thermal Power [kW]",
                                     x_axis_label="Outdoor Temperature [ºC]",
                                     **FIGURE_TOOLS
                                    )
london_heat_power_signature.add_tools(heat_power_signature_tool)
l1 = london_heat_power_signature.circle(x="air_temp", y="Total_consumption", color=color_map["london"], alpha=0.2, source=europe_source, view=london_view)
l2 = london_heat_power_signature.line(x="air_temp", y="Total_consumption_fit", color=color_map["RMSE"], alpha=1, line_width=2.5, source=europe_source, view=london_view)
london_heat_power_signature.add_layout(Legend(items=[("Total Heat [kW]", [l1]), ("RMSE [kW]", [l2]),], ), "below")


madrid_heat_power_signature = figure(title="Madrid, Spain",
                                     y_range=london_heat_power_signature.y_range,
                                     x_range=london_heat_power_signature.x_range,
                                     y_axis_label="Thermal Power [kW]",
                                     x_axis_label="Outdoor Temperature [ºC]",
                                     **FIGURE_TOOLS
                                    )
madrid_heat_power_signature.add_tools(heat_power_signature_tool)
l1 = madrid_heat_power_signature.circle(x="air_temp", y="Total_consumption", color=color_map["madrid"], alpha=0.2, source=europe_source, view=madrid_view)
l2 = madrid_heat_power_signature.line(x="air_temp", y="Total_consumption_fit", color=color_map["RMSE"], alpha=1, line_width=2.5, source=europe_source, view=madrid_view)
madrid_heat_power_signature.add_layout(Legend(items=[("Total Heat [kW]", [l1]), ("RMSE [kW]", [l2]),], ), "below")


rome_heat_power_signature = figure(title="Rome, Italy",
                                   y_range=london_heat_power_signature.y_range,
                                   x_range=london_heat_power_signature.x_range,
                                   y_axis_label="Thermal Power [kW]",
                                   x_axis_label="Outdoor Temperature [ºC]",
                                   **FIGURE_TOOLS
                                  )
rome_heat_power_signature.add_tools(heat_power_signature_tool)
l1 = rome_heat_power_signature.circle(x="air_temp", y="Total_consumption", color=color_map["rome"], alpha=0.2, source=europe_source, view=rome_view)
l2 = rome_heat_power_signature.line(x="air_temp", y="Total_consumption_fit", color=color_map["RMSE"], alpha=1, line_width=2.5, source=europe_source, view=rome_view)
rome_heat_power_signature.add_layout(Legend(items=[("Total Heat [kW]", [l1]), ("RMSE [kW]", [l2]),], ), "below")


stuttgart_heat_power_signature = figure(title="Stuttgart, Germany",
                                        y_range=london_heat_power_signature.y_range,
                                        x_range=london_heat_power_signature.x_range,
                                        y_axis_label="Thermal Power [kW]",
                                        x_axis_label="Outdoor Temperature [ºC]",
                                        **FIGURE_TOOLS
                                       )
stuttgart_heat_power_signature.add_tools(heat_power_signature_tool)
l1 = stuttgart_heat_power_signature.circle(x="air_temp", y="Total_consumption", color=color_map["stuttgart"], alpha=0.2, source=europe_source, view=stuttgart_view)
l2 = stuttgart_heat_power_signature.line(x="air_temp", y="Total_consumption_fit", color=color_map["RMSE"], alpha=1, line_width=2.5, source=europe_source, view=stuttgart_view)
stuttgart_heat_power_signature.add_layout(Legend(items=[("Total Heat [kW]", [l1]), ("RMSE [kW]", [l2]),], ), "below")


all_europe_heat_power_signature = figure(title="Power vs Air Temp",
                                         y_axis_label="Thermal Power [kW]",
                                         x_axis_label="Outdoor Temperature [ºC]",
                                         **{**FIGURE_TOOLS, **FULL_WIDTH_FIGURE_TOOLS})
all_europe_heat_power_signature.add_tools(all_heat_power_signature_tool)
l1 = all_europe_heat_power_signature.circle(x="air_temp", y="Total_consumption", color=color_map["london"], alpha=0.2, source=europe_source, view=london_view)
l2 = all_europe_heat_power_signature.circle(x="air_temp", y="Total_consumption", color=color_map["stuttgart"], alpha=0.2, source=europe_source, view=stuttgart_view)
l3 = all_europe_heat_power_signature.circle(x="air_temp", y="Total_consumption", color=color_map["rome"], alpha=0.2, source=europe_source, view=rome_view)
l4 = all_europe_heat_power_signature.circle(x="air_temp", y="Total_consumption", color=color_map["madrid"], alpha=0.2, source=europe_source, view=madrid_view)
all_europe_heat_power_signature.add_layout(Legend(items=[("London, UK", [l1]), ("Stuttgart, Germany", [l2]), ("Rome, Italy", [l3]), ("Madrid, Spain", [l4]),], ), "below")


In [ ]:
show(layout([[all_europe_heat_power_signature],[london_heat_power_signature, stuttgart_heat_power_signature],[rome_heat_power_signature, madrid_heat_power_signature],]))

## Sources and network temperatures 

In [ ]:
sources_and_network_temp_hover_tool = HoverTool(tooltips=[('Date', '@timestamp{%Y/%m/%d %H:%M:%S}'),
                                                          ('Source 1 Temperature', '@source1_temp{%0.2f ºC}'),
                                                          ('Source 2 Temperature', '@source2_temp{%0.2f ºC}'),
                                                          ('Aquifer Temperature', '@aquifer_temp{%0.2f ºC}'),
                                                          ('Network Temperature', '@net_temp{%0.2f ºC}'),],
                                                formatters={'@timestamp': 'datetime',
                                                            '@source1_temp': 'printf',
                                                            '@source2_temp': 'printf',
                                                            '@aquifer_temp': 'printf',
                                                            '@net_temp': 'printf',},)

### USA

In [ ]:
winter_x_values = pd.date_range(start="2010-01-10", freq="D", periods=15)
summer_x_values = pd.date_range(start="2010-07-15", freq="D", periods=15)

rochester_winter_sources_temp = figure(x_axis_type="datetime",
                                       x_range=(winter_x_values[0], winter_x_values[-1]),
                                       title="Rochester, NY - Winter - Temperatures of sources, aquifer and network",
                                       y_axis_label="Temperature (ºC)",
                                       x_axis_label="Date",
                                       **{**FIGURE_TOOLS, **FULL_WIDTH_FIGURE_TOOLS}
                                      )
rochester_winter_sources_temp.add_tools(sources_and_network_temp_hover_tool)
l1 = rochester_winter_sources_temp.line(x="timestamp", y="source1_temp", color=color_map["source1"], alpha=1, source=us_source, view=rochester_2_weeks_winter_view, line_width=4)
l2 = rochester_winter_sources_temp.line(x="timestamp", y="source2_temp", color=color_map["source2"], alpha=1, source=us_source, view=rochester_2_weeks_winter_view, line_width=4)
l3 = rochester_winter_sources_temp.line(x="timestamp", y="aquifer_temp", color=color_map["source_water"], alpha=1, source=us_source, view=rochester_2_weeks_winter_view, line_width=4)
l4 = rochester_winter_sources_temp.triangle(x="timestamp", y="net_temp", color=color_map["electricity"], alpha=1, source=us_source, view=rochester_2_weeks_winter_view, size=14)
rochester_winter_sources_temp.add_layout(Legend(items=[("Source 1", [l1]), ("Source 2", [l2]), ("Aquifer", [l3]), ("Network", [l4])],), "below")


rochester_summer_sources_temp = figure(x_axis_type="datetime",
                                       x_range=(summer_x_values[0], summer_x_values[-1]),
                                       title="Rochester, NY - Summer - Temperatures of sources, aquifer and network",
                                       y_axis_label="Temperature (ºC)",
                                       x_axis_label="Date",
                                       **{**FIGURE_TOOLS, **FULL_WIDTH_FIGURE_TOOLS}
                                      )
rochester_summer_sources_temp.add_tools(sources_and_network_temp_hover_tool)
l1 = rochester_summer_sources_temp.line(x="timestamp", y="source1_temp", color=color_map["source1"], alpha=1, source=us_source, view=rochester_2_weeks_summer_view, line_width=4)
l2 = rochester_summer_sources_temp.line(x="timestamp", y="source2_temp", color=color_map["source2"], alpha=1, source=us_source, view=rochester_2_weeks_summer_view, line_width=4)
l3 = rochester_summer_sources_temp.line(x="timestamp", y="aquifer_temp", color=color_map["source_water"], alpha=1, source=us_source, view=rochester_2_weeks_summer_view, line_width=4)
l4 = rochester_summer_sources_temp.triangle(x="timestamp", y="net_temp", color=color_map["electricity"], alpha=1, source=us_source, view=rochester_2_weeks_summer_view, size=14)
rochester_summer_sources_temp.add_layout(Legend(items=[("Source 1", [l1]), ("Source 2", [l2]), ("Aquifer", [l3]), ("Network", [l4])],), "below")


fresno_winter_sources_temp = figure(x_axis_type="datetime",
                                    x_range=(winter_x_values[0], winter_x_values[-1]),
                                    title="Fresno, CA - Winter - Temperatures of sources, aquifer and network",
                                    y_axis_label="Temperature (ºC)",
                                    x_axis_label="Date",
                                    **{**FIGURE_TOOLS, **FULL_WIDTH_FIGURE_TOOLS}
                                   )
fresno_winter_sources_temp.add_tools(sources_and_network_temp_hover_tool)
l1 = fresno_winter_sources_temp.line(x="timestamp", y="source1_temp", color=color_map["source1"], alpha=1, source=us_source, view=fresno_2_weeks_winter_view, line_width=4)
l2 = fresno_winter_sources_temp.line(x="timestamp", y="source2_temp", color=color_map["source2"], alpha=1, source=us_source, view=fresno_2_weeks_winter_view, line_width=4)
l3 = fresno_winter_sources_temp.line(x="timestamp", y="aquifer_temp", color=color_map["source_water"], alpha=1, source=us_source, view=fresno_2_weeks_winter_view, line_width=4)
l4 = fresno_winter_sources_temp.triangle(x="timestamp", y="net_temp", color=color_map["electricity"], alpha=1, source=us_source, view=fresno_2_weeks_winter_view, size=14)
fresno_winter_sources_temp.add_layout(Legend(items=[("Source 1", [l1]), ("Source 2", [l2]), ("Aquifer", [l3]), ("Network", [l4])],), "below")
     
                                    
fresno_summer_sources_temp = figure(x_axis_type="datetime",
                                    x_range=(summer_x_values[0], summer_x_values[-1]),
                                    title="Fresno, CA - Summer - Temperatures of sources, aquifer and network",
                                    y_axis_label="Temperature (ºC)",
                                    x_axis_label="Date",
                                    **{**FIGURE_TOOLS, **FULL_WIDTH_FIGURE_TOOLS}
                                   )
fresno_summer_sources_temp.add_tools(sources_and_network_temp_hover_tool)
l1 = fresno_summer_sources_temp.line(x="timestamp", y="source1_temp", color=color_map["source1"], alpha=1, source=us_source, view=fresno_2_weeks_summer_view, line_width=4)
l2 = fresno_summer_sources_temp.line(x="timestamp", y="source2_temp", color=color_map["source2"], alpha=1, source=us_source, view=fresno_2_weeks_summer_view, line_width=4)
l3 = fresno_summer_sources_temp.line(x="timestamp", y="aquifer_temp", color=color_map["source_water"], alpha=1, source=us_source, view=fresno_2_weeks_summer_view, line_width=4)
l4 = fresno_summer_sources_temp.triangle(x="timestamp", y="net_temp", color=color_map["electricity"], alpha=1, source=us_source, view=fresno_2_weeks_summer_view, size=14)
fresno_summer_sources_temp.add_layout(Legend(items=[("Source 1", [l1]), ("Source 2", [l2]), ("Aquifer", [l3]), ("Network", [l4])],), "below")


In [ ]:
show(layout([[rochester_winter_sources_temp],[rochester_summer_sources_temp]]))    

### Europe

In [ ]:
winter_x_values = pd.date_range(start="2017-01-10", freq="D", periods=15)
summer_x_values = pd.date_range(start="2017-07-15", freq="D", periods=15)

london_winter_sources_temp = figure(x_axis_type="datetime",
                                    x_range=(winter_x_values[0], winter_x_values[-1]),
                                    title="London, UK - Winter - Temperatures of sources, aquifer and network",
                                    y_axis_label="Temperature (ºC)",
                                    x_axis_label="Date",
                                    **{**FIGURE_TOOLS, **FULL_WIDTH_FIGURE_TOOLS}
                                   )
london_winter_sources_temp.add_tools(sources_and_network_temp_hover_tool)
l1 = london_winter_sources_temp.line(x="timestamp", y="source1_temp", color=color_map["source1"], alpha=1, source=europe_source, view=london_2_weeks_winter_view, line_width=4)
l2 = london_winter_sources_temp.line(x="timestamp", y="source2_temp", color=color_map["source2"], alpha=1, source=europe_source, view=london_2_weeks_winter_view, line_width=4)
l3 = london_winter_sources_temp.line(x="timestamp", y="aquifer_temp", color=color_map["source_water"], alpha=1, source=europe_source, view=london_2_weeks_winter_view, line_width=4)
l4 = london_winter_sources_temp.triangle(x="timestamp", y="net_temp", color=color_map["electricity"], alpha=1, source=europe_source, view=london_2_weeks_winter_view, size=14)
london_winter_sources_temp.add_layout(Legend(items=[("Source 1", [l1]), ("Source 2", [l2]), ("Aquifer", [l3]), ("Network", [l4])],), "below")


london_summer_sources_temp = figure(x_axis_type="datetime",
                                    x_range=(summer_x_values[0], summer_x_values[-1]),
                                    title="London, UK - Summer - Temperatures of sources, aquifer and network",
                                    y_axis_label="Temperature (ºC)",
                                    x_axis_label="Date",
                                    **{**FIGURE_TOOLS, **FULL_WIDTH_FIGURE_TOOLS}
                                   )
london_summer_sources_temp.add_tools(sources_and_network_temp_hover_tool)
l1 = london_summer_sources_temp.line(x="timestamp", y="source1_temp", color=color_map["source1"], alpha=1, source=europe_source, view=london_2_weeks_summer_view, line_width=4)
l2 = london_summer_sources_temp.line(x="timestamp", y="source2_temp", color=color_map["source2"], alpha=1, source=europe_source, view=london_2_weeks_summer_view, line_width=4)
l3 = london_summer_sources_temp.line(x="timestamp", y="aquifer_temp", color=color_map["source_water"], alpha=1, source=europe_source, view=london_2_weeks_summer_view, line_width=4)
l4 = london_summer_sources_temp.triangle(x="timestamp", y="net_temp", color=color_map["electricity"], alpha=1, source=europe_source, view=london_2_weeks_summer_view, size=14)
london_summer_sources_temp.add_layout(Legend(items=[("Source 1", [l1]), ("Source 2", [l2]), ("Aquifer", [l3]), ("Network", [l4])],), "below")


rome_winter_sources_temp = figure(x_axis_type="datetime",
                                    x_range=(winter_x_values[0], winter_x_values[-1]),
                                    title="Rome, Italy - Winter - Temperatures of sources, aquifer and network",
                                    y_axis_label="Temperature (ºC)",
                                    x_axis_label="Date",
                                    **{**FIGURE_TOOLS, **FULL_WIDTH_FIGURE_TOOLS}
                                   )
rome_winter_sources_temp.add_tools(sources_and_network_temp_hover_tool)
l1 = rome_winter_sources_temp.line(x="timestamp", y="source1_temp", color=color_map["source1"], alpha=1, source=europe_source, view=rome_2_weeks_winter_view, line_width=4)
l2 = rome_winter_sources_temp.line(x="timestamp", y="source2_temp", color=color_map["source2"], alpha=1, source=europe_source, view=rome_2_weeks_winter_view, line_width=4)
l3 = rome_winter_sources_temp.line(x="timestamp", y="aquifer_temp", color=color_map["source_water"], alpha=1, source=europe_source, view=rome_2_weeks_winter_view, line_width=4)
l4 = rome_winter_sources_temp.triangle(x="timestamp", y="net_temp", color=color_map["electricity"], alpha=1, source=europe_source, view=rome_2_weeks_winter_view, size=14)
rome_winter_sources_temp.add_layout(Legend(items=[("Source 1", [l1]), ("Source 2", [l2]), ("Aquifer", [l3]), ("Network", [l4])],), "below")


rome_summer_sources_temp = figure(x_axis_type="datetime",
                                    x_range=(summer_x_values[0], summer_x_values[-1]),
                                    title="Rome, Italy - Summer - Temperatures of sources, aquifer and network",
                                    y_axis_label="Temperature (ºC)",
                                    x_axis_label="Date",
                                    **{**FIGURE_TOOLS, **FULL_WIDTH_FIGURE_TOOLS}
                                   )
rome_summer_sources_temp.add_tools(sources_and_network_temp_hover_tool)
l1 = rome_summer_sources_temp.line(x="timestamp", y="source1_temp", color=color_map["source1"], alpha=1, source=europe_source, view=rome_2_weeks_summer_view, line_width=4)
l2 = rome_summer_sources_temp.line(x="timestamp", y="source2_temp", color=color_map["source2"], alpha=1, source=europe_source, view=rome_2_weeks_summer_view, line_width=4)
l3 = rome_summer_sources_temp.line(x="timestamp", y="aquifer_temp", color=color_map["source_water"], alpha=1, source=europe_source, view=rome_2_weeks_summer_view, line_width=4)
l4 = rome_summer_sources_temp.triangle(x="timestamp", y="net_temp", color=color_map["electricity"], alpha=1, source=europe_source, view=rome_2_weeks_summer_view, size=14)
rome_summer_sources_temp.add_layout(Legend(items=[("Source 1", [l1]), ("Source 2", [l2]), ("Aquifer", [l3]), ("Network", [l4])],), "below")


In [ ]:
show(layout([[london_winter_sources_temp],[rome_winter_sources_temp],]))

## Weekly Electric Power By Season

### Europe

In [ ]:
dayofweek = range(0,7)
seasons = ["winter", "spring", "summer", "fall"]
cities = [InsPireDataset.LONDON_UK, InsPireDataset.MADRID_SPA, InsPireDataset.ROME_IT, InsPireDataset.STUTTGART_GER]

total_consumption_average_per_day = dict()
electrical_power_use_season_tool_by_city = dict()
for city in cities:
    for season in seasons:
        days_for_city_and_season = np.empty(shape=(7,), dtype=np.float32)
        
        for day in dayofweek:
            days_for_city_and_season[day] = europe_data[(europe_data.city_key == city) & (europe_data.season == season) & (europe_data.dayofweek == day)]["E_el"].mean()#/1000
    
        total_consumption_average_per_day[f"{city}_{season}"] = days_for_city_and_season
        electrical_power_use_season_tool_by_city[city] = HoverTool(tooltips=[("Day", "@dayofweek{%d}"),
                                                                             ('Electricity Winter', f'@{city}_winter' + '{%0.2f MW}'),
                                                                             ('Electricity Spring', f'@{city}_spring' + '{%0.2f MW}'),
                                                                             ('Electricity Summer', f'@{city}_summer' + '{%0.2f MW}'),
                                                                             ('Electricity Fall', f'@{city}_fall' + '{%0.2f MW}'),],
                                                                   formatters={'@hour': 'printf',
                                                                               f'@{city}_winter': 'printf',
                                                                               f'@{city}_spring': 'printf',
                                                                               f'@{city}_summer': 'printf',
                                                                               f'@{city}_fall': "printf"},)
        
x=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

europe_electricity_season_source = ColumnDataSource({"dayofweek": x, **total_consumption_average_per_day})

In [ ]:
london_seasonality_total_consumption = figure(title="London, United Kingdom",
                                              x_range=FactorRange(*x),
                                              x_axis_label="Day of the week",
                                              y_axis_label="Seasonal electric power [MW]",
                                              **FIGURE_TOOLS,
                                             )
london_seasonality_total_consumption.add_tools(electrical_power_use_season_tool_by_city[InsPireDataset.LONDON_UK])
l11 = london_seasonality_total_consumption.line(x="dayofweek", y=f"{InsPireDataset.LONDON_UK}_winter", source=europe_electricity_season_source, color=color_map["winter"], alpha=0.7, line_width=3)
l12 = london_seasonality_total_consumption.square(x="dayofweek", y=f"{InsPireDataset.LONDON_UK}_winter", source=europe_electricity_season_source, color=color_map["winter"], alpha=0.7, size=10)
l21 = london_seasonality_total_consumption.line(x="dayofweek", y=f"{InsPireDataset.LONDON_UK}_spring", source=europe_electricity_season_source, color=color_map["spring"], alpha=0.7,  line_width=3)
l22 = london_seasonality_total_consumption.circle(x="dayofweek", y=f"{InsPireDataset.LONDON_UK}_spring", source=europe_electricity_season_source, color=color_map["spring"], alpha=0.7,  size=10)
l31 = london_seasonality_total_consumption.line(x="dayofweek", y=f"{InsPireDataset.LONDON_UK}_summer", source=europe_electricity_season_source, color=color_map["summer"], alpha=0.7,  line_width=3)
l32 = london_seasonality_total_consumption.diamond(x="dayofweek", y=f"{InsPireDataset.LONDON_UK}_summer", source=europe_electricity_season_source, color=color_map["summer"], alpha=0.7,  size=10)
l41 = london_seasonality_total_consumption.line(x="dayofweek", y=f"{InsPireDataset.LONDON_UK}_fall", source=europe_electricity_season_source, color=color_map["fall"], alpha=0.7,  line_width=3)
l42 = london_seasonality_total_consumption.cross(x="dayofweek", y=f"{InsPireDataset.LONDON_UK}_fall", source=europe_electricity_season_source, color=color_map["fall"], alpha=0.7, size=10)
london_seasonality_total_consumption.add_layout(Legend(items=[("Winter 2017", [l11, l12]), ("Spring 2017", [l21, l22]), ("Summer 2017", [l31, l32]), ("Fall 2017", [l41, l42]),],), "below")


stuttgart_seasonality_total_consumption = figure(title="Stuttgart, Germany",
                                                 y_range=london_seasonality_total_consumption.y_range,
                                                 x_range=london_seasonality_total_consumption.x_range,
                                                 x_axis_label="Day of the week",
                                                 y_axis_label="Seasonal electric power [MW]",
                                                 **FIGURE_TOOLS,
                                                )
stuttgart_seasonality_total_consumption.add_tools(electrical_power_use_season_tool_by_city[InsPireDataset.STUTTGART_GER])
l11 = stuttgart_seasonality_total_consumption.line(x="dayofweek", y=f"{InsPireDataset.STUTTGART_GER}_winter", source=europe_electricity_season_source, color=color_map["winter"], alpha=0.7, line_width=3)
l12 = stuttgart_seasonality_total_consumption.square(x="dayofweek", y=f"{InsPireDataset.STUTTGART_GER}_winter", source=europe_electricity_season_source, color=color_map["winter"], alpha=0.7, size=10)
l21 = stuttgart_seasonality_total_consumption.line(x="dayofweek", y=f"{InsPireDataset.STUTTGART_GER}_spring", source=europe_electricity_season_source, color=color_map["spring"], alpha=0.7,  line_width=3)
l22 = stuttgart_seasonality_total_consumption.circle(x="dayofweek", y=f"{InsPireDataset.STUTTGART_GER}_spring", source=europe_electricity_season_source, color=color_map["spring"], alpha=0.7,  size=10)
l31 = stuttgart_seasonality_total_consumption.line(x="dayofweek", y=f"{InsPireDataset.STUTTGART_GER}_summer", source=europe_electricity_season_source, color=color_map["summer"], alpha=0.7,  line_width=3)
l32 = stuttgart_seasonality_total_consumption.diamond(x="dayofweek", y=f"{InsPireDataset.STUTTGART_GER}_summer", source=europe_electricity_season_source, color=color_map["summer"], alpha=0.7,  size=10)
l41 = stuttgart_seasonality_total_consumption.line(x="dayofweek", y=f"{InsPireDataset.STUTTGART_GER}_fall", source=europe_electricity_season_source, color=color_map["fall"], alpha=0.7,  line_width=3)
l42 = stuttgart_seasonality_total_consumption.cross(x="dayofweek", y=f"{InsPireDataset.STUTTGART_GER}_fall", source=europe_electricity_season_source, color=color_map["fall"], alpha=0.7, size=10)
stuttgart_seasonality_total_consumption.add_layout(Legend(items=[("Winter 2017", [l11, l12]), ("Spring 2017", [l21, l22]), ("Summer 2017", [l31, l32]), ("Fall 2017", [l41, l42]),],), "below")



rome_seasonality_total_consumption = figure(title="Rome, Italy",
                                            y_range=london_seasonality_total_consumption.y_range,
                                            x_range=london_seasonality_total_consumption.x_range,
                                            x_axis_label="Day of the week",
                                            y_axis_label="Seasonal electric power [MW]",
                                            **FIGURE_TOOLS,
                                           )
rome_seasonality_total_consumption.add_tools(electrical_power_use_season_tool_by_city[InsPireDataset.ROME_IT])
l11 = rome_seasonality_total_consumption.line(x="dayofweek", y=f"{InsPireDataset.ROME_IT}_winter", source=europe_electricity_season_source, color=color_map["winter"], alpha=0.7, line_width=3)
l12 = rome_seasonality_total_consumption.square(x="dayofweek", y=f"{InsPireDataset.ROME_IT}_winter", source=europe_electricity_season_source, color=color_map["winter"], alpha=0.7, size=10)
l21 = rome_seasonality_total_consumption.line(x="dayofweek", y=f"{InsPireDataset.ROME_IT}_spring", source=europe_electricity_season_source, color=color_map["spring"], alpha=0.7,  line_width=3)
l22 = rome_seasonality_total_consumption.circle(x="dayofweek", y=f"{InsPireDataset.ROME_IT}_spring", source=europe_electricity_season_source, color=color_map["spring"], alpha=0.7,  size=10)
l31 = rome_seasonality_total_consumption.line(x="dayofweek", y=f"{InsPireDataset.ROME_IT}_summer", source=europe_electricity_season_source, color=color_map["summer"], alpha=0.7,  line_width=3)
l32 = rome_seasonality_total_consumption.diamond(x="dayofweek", y=f"{InsPireDataset.ROME_IT}_summer", source=europe_electricity_season_source, color=color_map["summer"], alpha=0.7,  size=10)
l41 = rome_seasonality_total_consumption.line(x="dayofweek", y=f"{InsPireDataset.ROME_IT}_fall", source=europe_electricity_season_source, color=color_map["fall"], alpha=0.7,  line_width=3)
l42 = rome_seasonality_total_consumption.cross(x="dayofweek", y=f"{InsPireDataset.ROME_IT}_fall", source=europe_electricity_season_source, color=color_map["fall"], alpha=0.7, size=10)
rome_seasonality_total_consumption.add_layout(Legend(items=[("Winter 2017", [l11, l12]), ("Spring 2017", [l21, l22]), ("Summer 2017", [l31, l32]), ("Fall 2017", [l41, l42]),],), "below")


madrid_seasonality_total_consumption = figure(title=" Madrid, Spain",
                                              y_range=london_seasonality_total_consumption.y_range,
                                              x_range=london_seasonality_total_consumption.x_range,
                                              x_axis_label="Day of the week",
                                              y_axis_label="Seasonal electric power [MW]",
                                              **FIGURE_TOOLS,
                                             )
madrid_seasonality_total_consumption.add_tools(electrical_power_use_season_tool_by_city[InsPireDataset.MADRID_SPA])
l11 = madrid_seasonality_total_consumption.line(x="dayofweek", y=f"{InsPireDataset.MADRID_SPA}_winter", source=europe_electricity_season_source, color=color_map["winter"], alpha=0.7, line_width=3)
l12 = madrid_seasonality_total_consumption.square(x="dayofweek", y=f"{InsPireDataset.MADRID_SPA}_winter", source=europe_electricity_season_source, color=color_map["winter"], alpha=0.7, size=10)
l21 = madrid_seasonality_total_consumption.line(x="dayofweek", y=f"{InsPireDataset.MADRID_SPA}_spring", source=europe_electricity_season_source, color=color_map["spring"], alpha=0.7,  line_width=3)
l22 = madrid_seasonality_total_consumption.circle(x="dayofweek", y=f"{InsPireDataset.MADRID_SPA}_spring", source=europe_electricity_season_source, color=color_map["spring"], alpha=0.7,  size=10)
l31 = madrid_seasonality_total_consumption.line(x="dayofweek", y=f"{InsPireDataset.MADRID_SPA}_summer", source=europe_electricity_season_source, color=color_map["summer"], alpha=0.7,  line_width=3)
l32 = madrid_seasonality_total_consumption.diamond(x="dayofweek", y=f"{InsPireDataset.MADRID_SPA}_summer", source=europe_electricity_season_source, color=color_map["summer"], alpha=0.7,  size=10)
l41 = madrid_seasonality_total_consumption.line(x="dayofweek", y=f"{InsPireDataset.MADRID_SPA}_fall", source=europe_electricity_season_source, color=color_map["fall"], alpha=0.7,  line_width=3)
l42 = madrid_seasonality_total_consumption.cross(x="dayofweek", y=f"{InsPireDataset.MADRID_SPA}_fall", source=europe_electricity_season_source, color=color_map["fall"], alpha=0.7, size=10)
madrid_seasonality_total_consumption.add_layout(Legend(items=[("Winter 2017", [l11, l12]), ("Spring 2017", [l21, l22]), ("Summer 2017", [l31, l32]), ("Fall 2017", [l41, l42]),],), "below")


In [ ]:
show(layout([[london_seasonality_total_consumption, stuttgart_seasonality_total_consumption],[rome_seasonality_total_consumption, madrid_seasonality_total_consumption]])) 

### USA

In [ ]:
dayofweek = range(0,7)
seasons = ["winter", "spring", "summer", "fall"]
cities = [NOAA2010Dataset.MIAMI_FL, NOAA2010Dataset.FRESNO_CA, NOAA2010Dataset.OLYMPIA_WA, NOAA2010Dataset.ROCHESTER_NY]

total_consumption_average_per_day = dict()
electrical_power_use_season_tool_by_city = dict()
for city in cities:
    for season in seasons:
        days_for_city_and_season = np.empty(shape=(7,), dtype=np.float32)
        
        for day in dayofweek:
            days_for_city_and_season[day] = us_data[(us_data.city_key == city) & (us_data.season == season) & (us_data.dayofweek == day)]["E_el"].mean()#/1000
    
        total_consumption_average_per_day[f"{city}_{season}"] = days_for_city_and_season
        electrical_power_use_season_tool_by_city[city] = HoverTool(tooltips=[("Day", "@dayofweek{%d}"),
                                                                             ('Electricity Winter', f'@{city}_winter' + '{%0.2f MW}'),
                                                                             ('Electricity Spring', f'@{city}_spring' + '{%0.2f MW}'),
                                                                             ('Electricity Summer', f'@{city}_summer' + '{%0.2f MW}'),
                                                                             ('Electricity Fall', f'@{city}_fall' + '{%0.2f MW}'),],
                                                                   formatters={'@hour': 'printf',
                                                                               f'@{city}_winter': 'printf',
                                                                               f'@{city}_spring': 'printf',
                                                                               f'@{city}_summer': 'printf',
                                                                               f'@{city}_fall': "printf"},)
        
x=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

usa_electricity_season_source = ColumnDataSource({"dayofweek": x, **total_consumption_average_per_day})

In [ ]:
rochester_seasonality_total_consumption = figure(title="Rochester, NY",
                                                 x_range=FactorRange(*x),
                                                 x_axis_label="Day of the week",
                                                 y_axis_label="Seasonal electric power [MW]",
                                                 **FIGURE_TOOLS,
                                                )
rochester_seasonality_total_consumption.add_tools(electrical_power_use_season_tool_by_city[NOAA2010Dataset.ROCHESTER_NY])
l11 = rochester_seasonality_total_consumption.line(x="dayofweek", y=f"{NOAA2010Dataset.ROCHESTER_NY}_winter", source=usa_electricity_season_source, color=color_map["winter"], alpha=0.7, line_width=3)
l12 = rochester_seasonality_total_consumption.square(x="dayofweek", y=f"{NOAA2010Dataset.ROCHESTER_NY}_winter", source=usa_electricity_season_source, color=color_map["winter"], alpha=0.7, size=10)
l21 = rochester_seasonality_total_consumption.line(x="dayofweek", y=f"{NOAA2010Dataset.ROCHESTER_NY}_spring", source=usa_electricity_season_source, color=color_map["spring"], alpha=0.7,  line_width=3)
l22 = rochester_seasonality_total_consumption.circle(x="dayofweek", y=f"{NOAA2010Dataset.ROCHESTER_NY}_spring", source=usa_electricity_season_source, color=color_map["spring"], alpha=0.7,  size=10)
l31 = rochester_seasonality_total_consumption.line(x="dayofweek", y=f"{NOAA2010Dataset.ROCHESTER_NY}_summer", source=usa_electricity_season_source, color=color_map["summer"], alpha=0.7,  line_width=3)
l32 = rochester_seasonality_total_consumption.diamond(x="dayofweek", y=f"{NOAA2010Dataset.ROCHESTER_NY}_summer", source=usa_electricity_season_source, color=color_map["summer"], alpha=0.7,  size=10)
l41 = rochester_seasonality_total_consumption.line(x="dayofweek", y=f"{NOAA2010Dataset.ROCHESTER_NY}_fall", source=usa_electricity_season_source, color=color_map["fall"], alpha=0.7,  line_width=3)
l42 = rochester_seasonality_total_consumption.cross(x="dayofweek", y=f"{NOAA2010Dataset.ROCHESTER_NY}_fall", source=usa_electricity_season_source, color=color_map["fall"], alpha=0.7, size=10)
rochester_seasonality_total_consumption.add_layout(Legend(items=[("Winter 2010", [l11, l12]), ("Spring 2010", [l21, l22]), ("Summer 2010", [l31, l32]), ("Fall 2010", [l41, l42]),],), "below")


olympia_seasonality_total_consumption = figure(title="Olympia, WA",
                                               x_axis_label="Day of the week",
                                               y_axis_label="Seasonal electric power [MW]",
                                               y_range=rochester_seasonality_total_consumption.y_range,
                                               x_range=rochester_seasonality_total_consumption.x_range,
                                               **FIGURE_TOOLS,
                                              )
olympia_seasonality_total_consumption.add_tools(electrical_power_use_season_tool_by_city[NOAA2010Dataset.OLYMPIA_WA])
l11 = olympia_seasonality_total_consumption.line(x="dayofweek", y=f"{NOAA2010Dataset.OLYMPIA_WA}_winter", source=usa_electricity_season_source, color=color_map["winter"], alpha=0.7, line_width=3)
l12 = olympia_seasonality_total_consumption.square(x="dayofweek", y=f"{NOAA2010Dataset.OLYMPIA_WA}_winter", source=usa_electricity_season_source, color=color_map["winter"], alpha=0.7, size=10)
l21 = olympia_seasonality_total_consumption.line(x="dayofweek", y=f"{NOAA2010Dataset.OLYMPIA_WA}_spring", source=usa_electricity_season_source, color=color_map["spring"], alpha=0.7,  line_width=3)
l22 = olympia_seasonality_total_consumption.circle(x="dayofweek", y=f"{NOAA2010Dataset.OLYMPIA_WA}_spring", source=usa_electricity_season_source, color=color_map["spring"], alpha=0.7,  size=10)
l31 = olympia_seasonality_total_consumption.line(x="dayofweek", y=f"{NOAA2010Dataset.OLYMPIA_WA}_summer", source=usa_electricity_season_source, color=color_map["summer"], alpha=0.7,  line_width=3)
l32 = olympia_seasonality_total_consumption.diamond(x="dayofweek", y=f"{NOAA2010Dataset.OLYMPIA_WA}_summer", source=usa_electricity_season_source, color=color_map["summer"], alpha=0.7,  size=10)
l41 = olympia_seasonality_total_consumption.line(x="dayofweek", y=f"{NOAA2010Dataset.OLYMPIA_WA}_fall", source=usa_electricity_season_source, color=color_map["fall"], alpha=0.7,  line_width=3)
l42 = olympia_seasonality_total_consumption.cross(x="dayofweek", y=f"{NOAA2010Dataset.OLYMPIA_WA}_fall", source=usa_electricity_season_source, color=color_map["fall"], alpha=0.7, size=10)
olympia_seasonality_total_consumption.add_layout(Legend(items=[("Winter 2010", [l11, l12]), ("Spring 2010", [l21, l22]), ("Summer 2010", [l31, l32]), ("Fall 2010", [l41, l42]),],), "below")


fresno_seasonality_total_consumption = figure(title="Fresno, CA",
                                               x_axis_label="Day of the week",
                                               y_axis_label="Seasonal electric power [MW]",
                                               y_range=rochester_seasonality_total_consumption.y_range,
                                               x_range=rochester_seasonality_total_consumption.x_range,
                                               **FIGURE_TOOLS,
                                              )
fresno_seasonality_total_consumption.add_tools(electrical_power_use_season_tool_by_city[NOAA2010Dataset.FRESNO_CA])
l11 = fresno_seasonality_total_consumption.line(x="dayofweek", y=f"{NOAA2010Dataset.FRESNO_CA}_winter", source=usa_electricity_season_source, color=color_map["winter"], alpha=0.7, line_width=3)
l12 = fresno_seasonality_total_consumption.square(x="dayofweek", y=f"{NOAA2010Dataset.FRESNO_CA}_winter", source=usa_electricity_season_source, color=color_map["winter"], alpha=0.7, size=10)
l21 = fresno_seasonality_total_consumption.line(x="dayofweek", y=f"{NOAA2010Dataset.FRESNO_CA}_spring", source=usa_electricity_season_source, color=color_map["spring"], alpha=0.7,  line_width=3)
l22 = fresno_seasonality_total_consumption.circle(x="dayofweek", y=f"{NOAA2010Dataset.FRESNO_CA}_spring", source=usa_electricity_season_source, color=color_map["spring"], alpha=0.7,  size=10)
l31 = fresno_seasonality_total_consumption.line(x="dayofweek", y=f"{NOAA2010Dataset.FRESNO_CA}_summer", source=usa_electricity_season_source, color=color_map["summer"], alpha=0.7,  line_width=3)
l32 = fresno_seasonality_total_consumption.diamond(x="dayofweek", y=f"{NOAA2010Dataset.FRESNO_CA}_summer", source=usa_electricity_season_source, color=color_map["summer"], alpha=0.7,  size=10)
l41 = fresno_seasonality_total_consumption.line(x="dayofweek", y=f"{NOAA2010Dataset.FRESNO_CA}_fall", source=usa_electricity_season_source, color=color_map["fall"], alpha=0.7,  line_width=3)
l42 = fresno_seasonality_total_consumption.cross(x="dayofweek", y=f"{NOAA2010Dataset.FRESNO_CA}_fall", source=usa_electricity_season_source, color=color_map["fall"], alpha=0.7, size=10)
fresno_seasonality_total_consumption.add_layout(Legend(items=[("Winter 2010", [l11, l12]), ("Spring 2010", [l21, l22]), ("Summer 2010", [l31, l32]), ("Fall 2010", [l41, l42]),],), "below")


miami_seasonality_total_consumption = figure(title="Miami, FL",
                                               x_axis_label="Day of the week",
                                               y_axis_label="Seasonal electric power [MW]",
                                               y_range=rochester_seasonality_total_consumption.y_range,
                                               x_range=rochester_seasonality_total_consumption.x_range,
                                               **FIGURE_TOOLS,
                                              )
miami_seasonality_total_consumption.add_tools(electrical_power_use_season_tool_by_city[NOAA2010Dataset.MIAMI_FL])
l11 = miami_seasonality_total_consumption.line(x="dayofweek", y=f"{NOAA2010Dataset.MIAMI_FL}_winter", source=usa_electricity_season_source, color=color_map["winter"], alpha=0.7, line_width=3)
l12 = miami_seasonality_total_consumption.square(x="dayofweek", y=f"{NOAA2010Dataset.MIAMI_FL}_winter", source=usa_electricity_season_source, color=color_map["winter"], alpha=0.7, size=10)
l21 = miami_seasonality_total_consumption.line(x="dayofweek", y=f"{NOAA2010Dataset.MIAMI_FL}_spring", source=usa_electricity_season_source, color=color_map["spring"], alpha=0.7,  line_width=3)
l22 = miami_seasonality_total_consumption.circle(x="dayofweek", y=f"{NOAA2010Dataset.MIAMI_FL}_spring", source=usa_electricity_season_source, color=color_map["spring"], alpha=0.7,  size=10)
l31 = miami_seasonality_total_consumption.line(x="dayofweek", y=f"{NOAA2010Dataset.MIAMI_FL}_summer", source=usa_electricity_season_source, color=color_map["summer"], alpha=0.7,  line_width=3)
l32 = miami_seasonality_total_consumption.diamond(x="dayofweek", y=f"{NOAA2010Dataset.MIAMI_FL}_summer", source=usa_electricity_season_source, color=color_map["summer"], alpha=0.7,  size=10)
l41 = miami_seasonality_total_consumption.line(x="dayofweek", y=f"{NOAA2010Dataset.MIAMI_FL}_fall", source=usa_electricity_season_source, color=color_map["fall"], alpha=0.7,  line_width=3)
l42 = miami_seasonality_total_consumption.cross(x="dayofweek", y=f"{NOAA2010Dataset.MIAMI_FL}_fall", source=usa_electricity_season_source, color=color_map["fall"], alpha=0.7, size=10)
miami_seasonality_total_consumption.add_layout(Legend(items=[("Winter 2010", [l11, l12]), ("Spring 2010", [l21, l22]), ("Summer 2010", [l31, l32]), ("Fall 2010", [l41, l42]),],), "below")


In [ ]:
show(layout([[rochester_seasonality_total_consumption, olympia_seasonality_total_consumption],[fresno_seasonality_total_consumption, miami_seasonality_total_consumption]])) 

## Electrical Power Use by hour of day and season

### Europe

In [ ]:
hours = range(0,24)
seasons = ["winter", "spring", "summer", "fall"]
cities = [InsPireDataset.LONDON_UK, InsPireDataset.MADRID_SPA, InsPireDataset.ROME_IT, InsPireDataset.STUTTGART_GER]

total_consumption_average_per_hour = dict()
electrical_power_use_season_tool_by_city = dict()
for city in cities:
    for season in seasons:
        hours_for_city_and_season = np.empty(shape=(24,), dtype=np.float32)
        
        for hour in hours:
            hours_for_city_and_season[hour] = europe_data[(europe_data.city_key == city) & (europe_data.season == season) & (europe_data.hour == hour)]["E_el"].mean() #/ 1000
            
        total_consumption_average_per_hour[f"{city}_{season}"] = hours_for_city_and_season
        electrical_power_use_season_tool_by_city[city] = HoverTool(tooltips=[("Hour", "@hour{%d}"),
                                                                             ('Electricity Winter', f'@{city}_winter' + '{%0.2f MW}'),
                                                                             ('Electricity Spring', f'@{city}_spring' + '{%0.2f MW}'),
                                                                             ('Electricity Summer', f'@{city}_summer' + '{%0.2f MW}'),
                                                                             ('Electricity Fall', f'@{city}_fall' + '{%0.2f MW}'),],
                                                                   formatters={'@hour': 'printf',
                                                                               f'@{city}_winter': 'printf',
                                                                               f'@{city}_spring': 'printf',
                                                                               f'@{city}_summer': 'printf',
                                                                               f'@{city}_fall': "printf"},)

x=hours

europe_electricity_season_source = ColumnDataSource({"hour": hours, **total_consumption_average_per_hour})

In [ ]:
stuttgart_seasonality_total_consumption = figure(title="Stuttgart, Germany",
                                                 x_axis_label="Hour of the day",
                                                 y_axis_label="Seasonal electric power [MW]",
                                                 **FIGURE_TOOLS,
                                                )
stuttgart_seasonality_total_consumption.add_tools(electrical_power_use_season_tool_by_city[InsPireDataset.STUTTGART_GER])
l11 = stuttgart_seasonality_total_consumption.line(x="hour", y=f"{InsPireDataset.STUTTGART_GER}_winter", source=europe_electricity_season_source, color=color_map["winter"], alpha=0.7, line_width=3)
l12 = stuttgart_seasonality_total_consumption.square(x="hour", y=f"{InsPireDataset.STUTTGART_GER}_winter", source=europe_electricity_season_source, color=color_map["winter"], alpha=0.7, size=10)
l21 = stuttgart_seasonality_total_consumption.line(x="hour", y=f"{InsPireDataset.STUTTGART_GER}_spring", source=europe_electricity_season_source, color=color_map["spring"], alpha=0.7,  line_width=3)
l22 = stuttgart_seasonality_total_consumption.circle(x="hour", y=f"{InsPireDataset.STUTTGART_GER}_spring", source=europe_electricity_season_source, color=color_map["spring"], alpha=0.7,  size=10)
l31 = stuttgart_seasonality_total_consumption.line(x="hour", y=f"{InsPireDataset.STUTTGART_GER}_summer", source=europe_electricity_season_source, color=color_map["summer"], alpha=0.7,  line_width=3)
l32 = stuttgart_seasonality_total_consumption.diamond(x="hour", y=f"{InsPireDataset.STUTTGART_GER}_summer", source=europe_electricity_season_source, color=color_map["summer"], alpha=0.7,  size=10)
l41 = stuttgart_seasonality_total_consumption.line(x="hour", y=f"{InsPireDataset.STUTTGART_GER}_fall", source=europe_electricity_season_source, color=color_map["fall"], alpha=0.7,  line_width=3)
l42 = stuttgart_seasonality_total_consumption.cross(x="hour", y=f"{InsPireDataset.STUTTGART_GER}_fall", source=europe_electricity_season_source, color=color_map["fall"], alpha=0.7, size=10)
stuttgart_seasonality_total_consumption.add_layout(Legend(items=[("Winter 2017", [l11, l12]), ("Spring 2017", [l21, l22]), ("Summer 2017", [l31, l32]), ("Fall 2017", [l41, l42]),],), "below")


london_seasonality_total_consumption = figure(title="London, UK",
                                               x_axis_label="Hour of the day",
                                               y_axis_label="Seasonal electric power [MW]",
                                               y_range=stuttgart_seasonality_total_consumption.y_range,
                                               x_range=stuttgart_seasonality_total_consumption.x_range,
                                               **FIGURE_TOOLS,
                                              )
london_seasonality_total_consumption.add_tools(electrical_power_use_season_tool_by_city[InsPireDataset.LONDON_UK])
l11 = london_seasonality_total_consumption.line(x="hour", y=f"{InsPireDataset.LONDON_UK}_winter", source=europe_electricity_season_source, color=color_map["winter"], alpha=0.7, line_width=3)
l12 = london_seasonality_total_consumption.square(x="hour", y=f"{InsPireDataset.LONDON_UK}_winter", source=europe_electricity_season_source, color=color_map["winter"], alpha=0.7, size=10)
l21 = london_seasonality_total_consumption.line(x="hour", y=f"{InsPireDataset.LONDON_UK}_spring", source=europe_electricity_season_source, color=color_map["spring"], alpha=0.7,  line_width=3)
l22 = london_seasonality_total_consumption.circle(x="hour", y=f"{InsPireDataset.LONDON_UK}_spring", source=europe_electricity_season_source, color=color_map["spring"], alpha=0.7,  size=10)
l31 = london_seasonality_total_consumption.line(x="hour", y=f"{InsPireDataset.LONDON_UK}_summer", source=europe_electricity_season_source, color=color_map["summer"], alpha=0.7,  line_width=3)
l32 = london_seasonality_total_consumption.diamond(x="hour", y=f"{InsPireDataset.LONDON_UK}_summer", source=europe_electricity_season_source, color=color_map["summer"], alpha=0.7,  size=10)
l41 = london_seasonality_total_consumption.line(x="hour", y=f"{InsPireDataset.LONDON_UK}_fall", source=europe_electricity_season_source, color=color_map["fall"], alpha=0.7,  line_width=3)
l42 = london_seasonality_total_consumption.cross(x="hour", y=f"{InsPireDataset.LONDON_UK}_fall", source=europe_electricity_season_source, color=color_map["fall"], alpha=0.7, size=10)
london_seasonality_total_consumption.add_layout(Legend(items=[("Winter 2017", [l11, l12]), ("Spring 2017", [l21, l22]), ("Summer 2017", [l31, l32]), ("Fall 2017", [l41, l42]),],), "below")


rome_seasonality_total_consumption = figure(title="Rome, Italy",
                                               x_axis_label="Hour of the day",
                                               y_axis_label="Seasonal electric power [MW]",
                                               y_range=stuttgart_seasonality_total_consumption.y_range,
                                               x_range=stuttgart_seasonality_total_consumption.x_range,
                                               **FIGURE_TOOLS,
                                              )
rome_seasonality_total_consumption.add_tools(electrical_power_use_season_tool_by_city[InsPireDataset.ROME_IT])
l11 = rome_seasonality_total_consumption.line(x="hour", y=f"{InsPireDataset.ROME_IT}_winter", source=europe_electricity_season_source, color=color_map["winter"], alpha=0.7, line_width=3)
l12 = rome_seasonality_total_consumption.square(x="hour", y=f"{InsPireDataset.ROME_IT}_winter", source=europe_electricity_season_source, color=color_map["winter"], alpha=0.7, size=10)
l21 = rome_seasonality_total_consumption.line(x="hour", y=f"{InsPireDataset.ROME_IT}_spring", source=europe_electricity_season_source, color=color_map["spring"], alpha=0.7,  line_width=3)
l22 = rome_seasonality_total_consumption.circle(x="hour", y=f"{InsPireDataset.ROME_IT}_spring", source=europe_electricity_season_source, color=color_map["spring"], alpha=0.7,  size=10)
l31 = rome_seasonality_total_consumption.line(x="hour", y=f"{InsPireDataset.ROME_IT}_summer", source=europe_electricity_season_source, color=color_map["summer"], alpha=0.7,  line_width=3)
l32 = rome_seasonality_total_consumption.diamond(x="hour", y=f"{InsPireDataset.ROME_IT}_summer", source=europe_electricity_season_source, color=color_map["summer"], alpha=0.7,  size=10)
l41 = rome_seasonality_total_consumption.line(x="hour", y=f"{InsPireDataset.ROME_IT}_fall", source=europe_electricity_season_source, color=color_map["fall"], alpha=0.7,  line_width=3)
l42 = rome_seasonality_total_consumption.cross(x="hour", y=f"{InsPireDataset.ROME_IT}_fall", source=europe_electricity_season_source, color=color_map["fall"], alpha=0.7, size=10)
rome_seasonality_total_consumption.add_layout(Legend(items=[("Winter 2017", [l11, l12]), ("Spring 2017", [l21, l22]), ("Summer 2017", [l31, l32]), ("Fall 2017", [l41, l42]),],), "below")



madrid_seasonality_total_consumption = figure(title="Madrid, Spain",
                                               x_axis_label="Hour of the day",
                                               y_axis_label="Seasonal electric power [MW]",
                                               y_range=stuttgart_seasonality_total_consumption.y_range,
                                               x_range=stuttgart_seasonality_total_consumption.x_range,
                                               **FIGURE_TOOLS,
                                              )
madrid_seasonality_total_consumption.add_tools(electrical_power_use_season_tool_by_city[InsPireDataset.MADRID_SPA])
l11 = madrid_seasonality_total_consumption.line(x="hour", y=f"{InsPireDataset.MADRID_SPA}_winter", source=europe_electricity_season_source, color=color_map["winter"], alpha=0.7, line_width=3)
l12 = madrid_seasonality_total_consumption.square(x="hour", y=f"{InsPireDataset.MADRID_SPA}_winter", source=europe_electricity_season_source, color=color_map["winter"], alpha=0.7, size=10)
l21 = madrid_seasonality_total_consumption.line(x="hour", y=f"{InsPireDataset.MADRID_SPA}_spring", source=europe_electricity_season_source, color=color_map["spring"], alpha=0.7,  line_width=3)
l22 = madrid_seasonality_total_consumption.circle(x="hour", y=f"{InsPireDataset.MADRID_SPA}_spring", source=europe_electricity_season_source, color=color_map["spring"], alpha=0.7,  size=10)
l31 = madrid_seasonality_total_consumption.line(x="hour", y=f"{InsPireDataset.MADRID_SPA}_summer", source=europe_electricity_season_source, color=color_map["summer"], alpha=0.7,  line_width=3)
l32 = madrid_seasonality_total_consumption.diamond(x="hour", y=f"{InsPireDataset.MADRID_SPA}_summer", source=europe_electricity_season_source, color=color_map["summer"], alpha=0.7,  size=10)
l41 = madrid_seasonality_total_consumption.line(x="hour", y=f"{InsPireDataset.MADRID_SPA}_fall", source=europe_electricity_season_source, color=color_map["fall"], alpha=0.7,  line_width=3)
l42 = madrid_seasonality_total_consumption.cross(x="hour", y=f"{InsPireDataset.MADRID_SPA}_fall", source=europe_electricity_season_source, color=color_map["fall"], alpha=0.7, size=10)
madrid_seasonality_total_consumption.add_layout(Legend(items=[("Winter 2017", [l11, l12]), ("Spring 2017", [l21, l22]), ("Summer 2017", [l31, l32]), ("Fall 2017", [l41, l42]),],), "below")


In [ ]:
show(layout([[london_seasonality_total_consumption, stuttgart_seasonality_total_consumption],[rome_seasonality_total_consumption, madrid_seasonality_total_consumption]])) 

### USA

In [ ]:
hours = range(0,24)
seasons = ["winter", "spring", "summer", "fall"]
cities = [NOAA2010Dataset.MIAMI_FL, NOAA2010Dataset.FRESNO_CA, NOAA2010Dataset.OLYMPIA_WA, NOAA2010Dataset.ROCHESTER_NY]

total_consumption_average_per_hour = dict()
electrical_power_use_season_tool_by_city = dict()
for city in cities:
    for season in seasons:
        hours_for_city_and_season = np.empty(shape=(24,), dtype=np.float32)
        
        for hour in hours:
            hours_for_city_and_season[hour] = us_data[(us_data.city_key == city) & (us_data.season == season) & (us_data.hour == hour)]["E_el"].mean() #/ 1000
            
        total_consumption_average_per_hour[f"{city}_{season}"] = hours_for_city_and_season
        electrical_power_use_season_tool_by_city[city] = HoverTool(tooltips=[("Hour", "@hour{%d}"),
                                                                             ('Electricity Winter', f'@{city}_winter' + '{%0.2f MW}'),
                                                                             ('Electricity Spring', f'@{city}_spring' + '{%0.2f MW}'),
                                                                             ('Electricity Summer', f'@{city}_summer' + '{%0.2f MW}'),
                                                                             ('Electricity Fall', f'@{city}_fall' + '{%0.2f MW}'),],
                                                                   formatters={'@hour': 'printf',
                                                                               f'@{city}_winter': 'printf',
                                                                               f'@{city}_spring': 'printf',
                                                                               f'@{city}_summer': 'printf',
                                                                               f'@{city}_fall': "printf"},)

x=hours

usa_electricity_season_source = ColumnDataSource({"hour": hours, **total_consumption_average_per_hour})

In [ ]:
rochester_seasonality_total_consumption = figure(title="Rochester, NY",
                                                 x_axis_label="Hour of the day",
                                                 y_axis_label="Seasonal electric power [MW]",
                                                 **FIGURE_TOOLS,
                                                )
rochester_seasonality_total_consumption.add_tools(electrical_power_use_season_tool_by_city[NOAA2010Dataset.ROCHESTER_NY])
l11 = rochester_seasonality_total_consumption.line(x="hour", y=f"{NOAA2010Dataset.ROCHESTER_NY}_winter", source=usa_electricity_season_source, color=color_map["winter"], alpha=0.7, line_width=3)
l12 = rochester_seasonality_total_consumption.square(x="hour", y=f"{NOAA2010Dataset.ROCHESTER_NY}_winter", source=usa_electricity_season_source, color=color_map["winter"], alpha=0.7, size=10)
l21 = rochester_seasonality_total_consumption.line(x="hour", y=f"{NOAA2010Dataset.ROCHESTER_NY}_spring", source=usa_electricity_season_source, color=color_map["spring"], alpha=0.7,  line_width=3)
l22 = rochester_seasonality_total_consumption.circle(x="hour", y=f"{NOAA2010Dataset.ROCHESTER_NY}_spring", source=usa_electricity_season_source, color=color_map["spring"], alpha=0.7,  size=10)
l31 = rochester_seasonality_total_consumption.line(x="hour", y=f"{NOAA2010Dataset.ROCHESTER_NY}_summer", source=usa_electricity_season_source, color=color_map["summer"], alpha=0.7,  line_width=3)
l32 = rochester_seasonality_total_consumption.diamond(x="hour", y=f"{NOAA2010Dataset.ROCHESTER_NY}_summer", source=usa_electricity_season_source, color=color_map["summer"], alpha=0.7,  size=10)
l41 = rochester_seasonality_total_consumption.line(x="hour", y=f"{NOAA2010Dataset.ROCHESTER_NY}_fall", source=usa_electricity_season_source, color=color_map["fall"], alpha=0.7,  line_width=3)
l42 = rochester_seasonality_total_consumption.cross(x="hour", y=f"{NOAA2010Dataset.ROCHESTER_NY}_fall", source=usa_electricity_season_source, color=color_map["fall"], alpha=0.7, size=10)
rochester_seasonality_total_consumption.add_layout(Legend(items=[("Winter 2010", [l11, l12]), ("Spring 2010", [l21, l22]), ("Summer 2010", [l31, l32]), ("Fall 2010", [l41, l42]),],), "below")


olympia_seasonality_total_consumption = figure(title="Olympia, WA",
                                               x_axis_label="Hour of the day",
                                               y_axis_label="Seasonal electric power [MW]",
                                               y_range=rochester_seasonality_total_consumption.y_range,
                                               x_range=rochester_seasonality_total_consumption.x_range,
                                               **FIGURE_TOOLS,
                                              )
olympia_seasonality_total_consumption.add_tools(electrical_power_use_season_tool_by_city[NOAA2010Dataset.OLYMPIA_WA])
l11 = olympia_seasonality_total_consumption.line(x="hour", y=f"{NOAA2010Dataset.OLYMPIA_WA}_winter", source=usa_electricity_season_source, color=color_map["winter"], alpha=0.7, line_width=3)
l12 = olympia_seasonality_total_consumption.square(x="hour", y=f"{NOAA2010Dataset.OLYMPIA_WA}_winter", source=usa_electricity_season_source, color=color_map["winter"], alpha=0.7, size=10)
l21 = olympia_seasonality_total_consumption.line(x="hour", y=f"{NOAA2010Dataset.OLYMPIA_WA}_spring", source=usa_electricity_season_source, color=color_map["spring"], alpha=0.7,  line_width=3)
l22 = olympia_seasonality_total_consumption.circle(x="hour", y=f"{NOAA2010Dataset.OLYMPIA_WA}_spring", source=usa_electricity_season_source, color=color_map["spring"], alpha=0.7,  size=10)
l31 = olympia_seasonality_total_consumption.line(x="hour", y=f"{NOAA2010Dataset.OLYMPIA_WA}_summer", source=usa_electricity_season_source, color=color_map["summer"], alpha=0.7,  line_width=3)
l32 = olympia_seasonality_total_consumption.diamond(x="hour", y=f"{NOAA2010Dataset.OLYMPIA_WA}_summer", source=usa_electricity_season_source, color=color_map["summer"], alpha=0.7,  size=10)
l41 = olympia_seasonality_total_consumption.line(x="hour", y=f"{NOAA2010Dataset.OLYMPIA_WA}_fall", source=usa_electricity_season_source, color=color_map["fall"], alpha=0.7,  line_width=3)
l42 = olympia_seasonality_total_consumption.cross(x="hour", y=f"{NOAA2010Dataset.OLYMPIA_WA}_fall", source=usa_electricity_season_source, color=color_map["fall"], alpha=0.7, size=10)
olympia_seasonality_total_consumption.add_layout(Legend(items=[("Winter 2010", [l11, l12]), ("Spring 2010", [l21, l22]), ("Summer 2010", [l31, l32]), ("Fall 2010", [l41, l42]),],), "below")


fresno_seasonality_total_consumption = figure(title="Fresno, CA",
                                              x_axis_label="Hour of the day",
                                              y_axis_label="Seasonal electric power [MW]",
                                              y_range=rochester_seasonality_total_consumption.y_range,
                                              x_range=rochester_seasonality_total_consumption.x_range,
                                              **FIGURE_TOOLS,
                                             )
fresno_seasonality_total_consumption.add_tools(electrical_power_use_season_tool_by_city[NOAA2010Dataset.FRESNO_CA])
l11 = fresno_seasonality_total_consumption.line(x="hour", y=f"{NOAA2010Dataset.FRESNO_CA}_winter", source=usa_electricity_season_source, color=color_map["winter"], alpha=0.7, line_width=3)
l12 = fresno_seasonality_total_consumption.square(x="hour", y=f"{NOAA2010Dataset.FRESNO_CA}_winter", source=usa_electricity_season_source, color=color_map["winter"], alpha=0.7, size=10)
l21 = fresno_seasonality_total_consumption.line(x="hour", y=f"{NOAA2010Dataset.FRESNO_CA}_spring", source=usa_electricity_season_source, color=color_map["spring"], alpha=0.7,  line_width=3)
l22 = fresno_seasonality_total_consumption.circle(x="hour", y=f"{NOAA2010Dataset.FRESNO_CA}_spring", source=usa_electricity_season_source, color=color_map["spring"], alpha=0.7,  size=10)
l31 = fresno_seasonality_total_consumption.line(x="hour", y=f"{NOAA2010Dataset.FRESNO_CA}_summer", source=usa_electricity_season_source, color=color_map["summer"], alpha=0.7,  line_width=3)
l32 = fresno_seasonality_total_consumption.diamond(x="hour", y=f"{NOAA2010Dataset.FRESNO_CA}_summer", source=usa_electricity_season_source, color=color_map["summer"], alpha=0.7,  size=10)
l41 = fresno_seasonality_total_consumption.line(x="hour", y=f"{NOAA2010Dataset.FRESNO_CA}_fall", source=usa_electricity_season_source, color=color_map["fall"], alpha=0.7,  line_width=3)
l42 = fresno_seasonality_total_consumption.cross(x="hour", y=f"{NOAA2010Dataset.FRESNO_CA}_fall", source=usa_electricity_season_source, color=color_map["fall"], alpha=0.7, size=10)
fresno_seasonality_total_consumption.add_layout(Legend(items=[("Winter 2010", [l11, l12]), ("Spring 2010", [l21, l22]), ("Summer 2010", [l31, l32]), ("Fall 2010", [l41, l42]),],), "below")


miami_seasonality_total_consumption = figure(title="Miami, FL",
                                             x_axis_label="Hour of the day",
                                             y_axis_label="Seasonal electric power [MW]",
                                             y_range=rochester_seasonality_total_consumption.y_range,
                                             x_range=rochester_seasonality_total_consumption.x_range,
                                             **FIGURE_TOOLS,
                                            )
miami_seasonality_total_consumption.add_tools(electrical_power_use_season_tool_by_city[NOAA2010Dataset.MIAMI_FL])
l11 = miami_seasonality_total_consumption.line(x="hour", y=f"{NOAA2010Dataset.MIAMI_FL}_winter", source=usa_electricity_season_source, color=color_map["winter"], alpha=0.7, line_width=3)
l12 = miami_seasonality_total_consumption.square(x="hour", y=f"{NOAA2010Dataset.MIAMI_FL}_winter", source=usa_electricity_season_source, color=color_map["winter"], alpha=0.7, size=10)
l21 = miami_seasonality_total_consumption.line(x="hour", y=f"{NOAA2010Dataset.MIAMI_FL}_spring", source=usa_electricity_season_source, color=color_map["spring"], alpha=0.7,  line_width=3)
l22 = miami_seasonality_total_consumption.circle(x="hour", y=f"{NOAA2010Dataset.MIAMI_FL}_spring", source=usa_electricity_season_source, color=color_map["spring"], alpha=0.7,  size=10)
l31 = miami_seasonality_total_consumption.line(x="hour", y=f"{NOAA2010Dataset.MIAMI_FL}_summer", source=usa_electricity_season_source, color=color_map["summer"], alpha=0.7,  line_width=3)
l32 = miami_seasonality_total_consumption.diamond(x="hour", y=f"{NOAA2010Dataset.MIAMI_FL}_summer", source=usa_electricity_season_source, color=color_map["summer"], alpha=0.7,  size=10)
l41 = miami_seasonality_total_consumption.line(x="hour", y=f"{NOAA2010Dataset.MIAMI_FL}_fall", source=usa_electricity_season_source, color=color_map["fall"], alpha=0.7,  line_width=3)
l42 = miami_seasonality_total_consumption.cross(x="hour", y=f"{NOAA2010Dataset.MIAMI_FL}_fall", source=usa_electricity_season_source, color=color_map["fall"], alpha=0.7, size=10)
miami_seasonality_total_consumption.add_layout(Legend(items=[("Winter 2010", [l11, l12]), ("Spring 2010", [l21, l22]), ("Summer 2010", [l31, l32]), ("Fall 2010", [l41, l42]),],), "below")



In [ ]:
show(layout([[rochester_seasonality_total_consumption, olympia_seasonality_total_consumption],[fresno_seasonality_total_consumption, miami_seasonality_total_consumption]])) 

## Thermal Energy from sources

In [ ]:
stack_month_average_heat_tool = HoverTool(tooltips=[('Month', '@month'),
                                               ('Heat Source 1', '@heat_source1{%0.2f GWh}'),
                                               ('Heat Source 2', '@heat_source2{%0.2f GWh}'),
                                               ('Heat Aquifer', '@heat_aquifer{%0.2f GWh}'),
                                               ('Heat Network', '@E_el{%0.2f GWh}')],
                                     formatters={'@heat_source1': 'printf',
                                                 '@heat_source2': 'printf',
                                                 '@heat_aquifer': 'printf',
                                                 '@E_el': 'printf',},)

stack_dayofweek_average_heat_tool = HoverTool(tooltips=[('Day of Week', '@dayofweek'),
                                                   ('Heat Source 1', '@heat_source1{%0.2f MWh}'),
                                                   ('Heat Source 2', '@heat_source2{%0.2f MWh}'),
                                                   ('Heat Aquifer', '@heat_aquifer{%0.2f MWh}'),
                                                   ('Heat Network', '@E_el{%0.2f MWh}')],
                                         formatters={'@heat_source1': 'printf',
                                                     '@heat_source2': 'printf',
                                                     '@heat_aquifer': 'printf',
                                                     '@E_el': 'printf',},)

### Europe

In [ ]:
data = london.copy()

data_month = data.groupby(by=["month"]).sum() / 1000 
source_month = ColumnDataSource(data_month)

data_week = data.groupby(by=["dayofweek"]).mean() #/ 1000
source_week = ColumnDataSource(data_week)

data_week_winter = data[data.season == "winter"].groupby(by=["dayofweek"]).mean() #/ 1000
source_week_winter = ColumnDataSource(data_week_winter)

data_week_spring = data[data.season == "spring"].groupby(by=["dayofweek"]).mean() #/ 1000
source_week_spring = ColumnDataSource(data_week_spring)

data_week_summer = data[data.season == "summer"].groupby(by=["dayofweek"]).mean() #/ 1000
source_week_summer = ColumnDataSource(data_week_summer)

data_week_fall = data[data.season == "fall"].groupby(by=["dayofweek"]).mean() #/ 1000
source_week_fall = ColumnDataSource(data_week_fall)

stackers = ['heat_source1', 'heat_source2', 'heat_aquifer', "E_el"]
colors = [color_map.get(key) for key in ["source1", "source2", "source_water", "electricity",]]
legends = ["Source 1","Source 2","Aquifer wells","Electricity"]

In [ ]:
london_stack_month = figure(title ="London, UK - Thermal energy by different sources", 
                               y_axis_label="Monthly thermal energy [GWh]",
                               x_axis_label="Month",
                               **{**FIGURE_TOOLS, **FULL_WIDTH_FIGURE_TOOLS})
london_stack_month.add_tools(stack_month_average_heat_tool)
renderers = london_stack_month.varea_stack(x='month', stackers=stackers, fill_color=colors, source=source_month)
london_stack_month.add_layout(Legend(items=[(legend, [renderer]) for legend, renderer in zip(legends, renderers)],), "below")


london_stack_dayofweek_winter = figure(title = "London, UK, - Thermal energy by different sources on Winter", 
                                          y_axis_label="Average thermal energy [MWh]",
                                          x_axis_label="Day of week",
                                          **FIGURE_TOOLS)
london_stack_dayofweek_winter.add_tools(stack_dayofweek_average_heat_tool)
renderers = london_stack_dayofweek_winter.varea_stack(x='dayofweek', stackers=stackers, fill_color=colors, source=source_week_winter)
london_stack_dayofweek_winter.add_layout(Legend(items=[(legend, [renderer]) for legend, renderer in zip(legends, renderers)],), "below")


london_stack_dayofweek_spring = figure(title ="London, UK, - Thermal energy by different sources on Spring",
                                       y_range=london_stack_dayofweek_winter.y_range,
                                       x_range=london_stack_dayofweek_winter.x_range,
                                          y_axis_label="Average thermal energy [MWh]",
                                          x_axis_label="Day of week", 
                                          **FIGURE_TOOLS
                                         )
london_stack_dayofweek_spring.add_tools(stack_dayofweek_average_heat_tool)
renderers = london_stack_dayofweek_spring.varea_stack(x='dayofweek', stackers=stackers, fill_color=colors, source=source_week_spring)
london_stack_dayofweek_spring.add_layout(Legend(items=[(legend, [renderer]) for legend, renderer in zip(legends, renderers)],), "below")


london_stack_dayofweek_summer = figure(title ="London, UK, - Thermal energy by different sources on Summer",
                                       y_range=london_stack_dayofweek_winter.y_range,
                                       x_range=london_stack_dayofweek_winter.x_range,
                                          y_axis_label="Average thermal energy [MWh]",
                                          x_axis_label="Day of week", 
                                          **FIGURE_TOOLS)
london_stack_dayofweek_summer.add_tools(stack_dayofweek_average_heat_tool)
renderers = london_stack_dayofweek_summer.varea_stack(x='dayofweek', stackers=stackers, fill_color=colors, source=source_week_summer)
london_stack_dayofweek_summer.add_layout(Legend(items=[(legend, [renderer]) for legend, renderer in zip(legends, renderers)],), "below")


london_stack_dayofweek_fall = figure(title ="London, UK, - Thermal energy by different sources on Fall",
                                     y_range=london_stack_dayofweek_winter.y_range,
                                     x_range=london_stack_dayofweek_winter.x_range,
                                        y_axis_label="Average thermal energy [MWh]",
                                        x_axis_label="Day of week", 
                                          **FIGURE_TOOLS)
london_stack_dayofweek_fall.add_tools(stack_dayofweek_average_heat_tool)
renderers = london_stack_dayofweek_fall.varea_stack(x='dayofweek', stackers=stackers, fill_color=colors, source=source_week_fall)
london_stack_dayofweek_fall.add_layout(Legend(items=[(legend, [renderer]) for legend, renderer in zip(legends, renderers)],), "below")


london_stack_dayofweek = figure(title ="London, UK - Average thermal energy by day of week", 
                                   y_axis_label="Average thermal energy [MWh]",
                                   x_axis_label="Day of week", 
                                   **{**FIGURE_TOOLS, **FULL_WIDTH_FIGURE_TOOLS})
london_stack_dayofweek.add_tools(stack_dayofweek_average_heat_tool)
renderers = london_stack_dayofweek.varea_stack(x='dayofweek', stackers=stackers, fill_color=colors, source=source_week)
london_stack_dayofweek.add_layout(Legend(items=[(legend, [renderer]) for legend, renderer in zip(legends, renderers)],), "below")


In [ ]:
show(layout([[london_stack_month],[london_stack_dayofweek_winter, london_stack_dayofweek_spring],[london_stack_dayofweek_summer, london_stack_dayofweek_fall],[london_stack_dayofweek],]))

### USA

In [ ]:
data = rochester.copy()

data_month = data.groupby(by=["month"]).sum()/ 1000 
source_month = ColumnDataSource(data_month)

data_week = data.groupby(by=["dayofweek"]).mean() #/ 1000
source_week = ColumnDataSource(data_week)

data_week_winter = data[data.season == "winter"].groupby(by=["dayofweek"]).mean() #/ 1000
source_week_winter = ColumnDataSource(data_week_winter)

data_week_spring = data[data.season == "spring"].groupby(by=["dayofweek"]).mean() #/ 1000
source_week_spring = ColumnDataSource(data_week_spring)

data_week_summer = data[data.season == "summer"].groupby(by=["dayofweek"]).mean() #/ 1000
source_week_summer = ColumnDataSource(data_week_summer)

data_week_fall = data[data.season == "fall"].groupby(by=["dayofweek"]).mean() #/ 1000
source_week_fall = ColumnDataSource(data_week_fall)

stackers = ['heat_source1', 'heat_source2', 'heat_aquifer', "E_el"]
colors = [color_map.get(key) for key in ["source1", "source2", "source_water", "electricity",]]
legends = ["Source 1","Source 2","Aquifer wells","Electricity"]

In [ ]:
rochester_stack_month = figure(title ="Rochester, NY - Thermal energy by different sources", 
                               y_axis_label="Monthly thermal energy [GWh]",
                               x_axis_label="Month",
                               **{**FIGURE_TOOLS, **FULL_WIDTH_FIGURE_TOOLS})
rochester_stack_month.add_tools(stack_month_average_heat_tool)
renderers = rochester_stack_month.varea_stack(x='month', stackers=stackers, fill_color=colors, source=source_month)
rochester_stack_month.add_layout(Legend(items=[(legend, [renderer]) for legend, renderer in zip(legends, renderers)],), "below")


rochester_stack_dayofweek_winter = figure(title="Rochester, NY - Thermal energy by different sources on Winter", 
                                          y_axis_label="Average thermal energy [MWh]",
                                          x_axis_label="Day of week",
                                          **FIGURE_TOOLS)
rochester_stack_dayofweek_winter.add_tools(stack_dayofweek_average_heat_tool)
renderers = rochester_stack_dayofweek_winter.varea_stack(x='dayofweek', stackers=stackers, fill_color=colors, source=source_week_winter)
rochester_stack_dayofweek_winter.add_layout(Legend(items=[(legend, [renderer]) for legend, renderer in zip(legends, renderers)],), "below")


rochester_stack_dayofweek_spring = figure(title ="Rochester, NY - Thermal energy by different sources on Spring",
                                          y_range=rochester_stack_dayofweek_winter.y_range,
                                          x_range=rochester_stack_dayofweek_winter.x_range,
                                          y_axis_label="Average thermal energy [MWh]",
                                          x_axis_label="Day of week", 
                                          **FIGURE_TOOLS
                                         )
rochester_stack_dayofweek_spring.add_tools(stack_dayofweek_average_heat_tool)
renderers = rochester_stack_dayofweek_spring.varea_stack(x='dayofweek', stackers=stackers, fill_color=colors, source=source_week_spring)
rochester_stack_dayofweek_spring.add_layout(Legend(items=[(legend, [renderer]) for legend, renderer in zip(legends, renderers)],), "below")


rochester_stack_dayofweek_summer = figure(title ="Rochester, NY - Thermal energy by different sources on Summer",
                                          y_range=rochester_stack_dayofweek_winter.y_range,
                                          x_range=rochester_stack_dayofweek_winter.x_range,
                                          y_axis_label="Average thermal energy [MWh]",
                                          x_axis_label="Day of week", 
                                          **FIGURE_TOOLS)
rochester_stack_dayofweek_summer.add_tools(stack_dayofweek_average_heat_tool)
renderers = rochester_stack_dayofweek_summer.varea_stack(x='dayofweek', stackers=stackers, fill_color=colors, source=source_week_summer)
rochester_stack_dayofweek_summer.add_layout(Legend(items=[(legend, [renderer]) for legend, renderer in zip(legends, renderers)],), "below")


rochester_stack_dayofweek_fall = figure(title ="Rochester, NY - Thermal energy by different sources on Fall",
                                        y_range=rochester_stack_dayofweek_winter.y_range,
                                        x_range=rochester_stack_dayofweek_winter.x_range,
                                        y_axis_label="Average thermal energy [MWh]",
                                        x_axis_label="Day of week", 
                                          **FIGURE_TOOLS)
rochester_stack_dayofweek_fall.add_tools(stack_dayofweek_average_heat_tool)
renderers = rochester_stack_dayofweek_fall.varea_stack(x='dayofweek', stackers=stackers, fill_color=colors, source=source_week_fall)
rochester_stack_dayofweek_fall.add_layout(Legend(items=[(legend, [renderer]) for legend, renderer in zip(legends, renderers)],), "below")


rochester_stack_dayofweek = figure(title ="Rochester, NY - Average thermal energy by day of week", 
                                   y_axis_label="Average thermal energy [MWh]",
                                   x_axis_label="Day of week", 
                                   **{**FIGURE_TOOLS, **FULL_WIDTH_FIGURE_TOOLS})
rochester_stack_dayofweek.add_tools(stack_dayofweek_average_heat_tool)
renderers = rochester_stack_dayofweek.varea_stack(x='dayofweek', stackers=stackers, fill_color=colors, source=source_week)
rochester_stack_dayofweek.add_layout(Legend(items=[(legend, [renderer]) for legend, renderer in zip(legends, renderers)],), "below")



In [ ]:
show(layout([[rochester_stack_month],[rochester_stack_dayofweek_winter, rochester_stack_dayofweek_spring],[rochester_stack_dayofweek_summer, rochester_stack_dayofweek_fall],[rochester_stack_dayofweek],]))

## Annual Analysis

In [ ]:
thermal_load_y=[london['Total_consumption'].sum(), 
                stuttgart['Total_consumption'].sum(), 
                madrid['Total_consumption'].sum(), 
                rome['Total_consumption'].sum(), 
                miami['Total_consumption'].sum(), 
                fresno['Total_consumption'].sum(), 
                olympia['Total_consumption'].sum(), 
                rochester['Total_consumption'].sum()]
thermal_y =pd.Series(thermal_load_y, index=["london", "stuttgart", "madrid", "rome", "miami", "fresno", "olympia", "rochester"])
thermal_y

electric_cons_y=[london['E_el'].sum(), 
                 stuttgart['E_el'].sum(), 
                 madrid['E_el'].sum(),
                 rome['E_el'].sum(), 
                 miami['E_el'].sum(), 
                 fresno['E_el'].sum(), 
                 olympia['E_el'].sum(), 
                 rochester['E_el'].sum()]
electric_y =pd.Series(electric_cons_y, index=["london", "stuttgart", "madrid", "rome", "miami", "fresno", "olympia", "rochester"])
electric_y

heat_loss_y=[london['E_loss_tot'].sum(), 
             stuttgart['E_loss_tot'].sum(), 
             madrid['E_loss_tot'].sum(),
             rome['E_loss_tot'].sum(), 
             miami['E_loss_tot'].sum(), 
             fresno['E_loss_tot'].sum(), 
             olympia['E_loss_tot'].sum(), 
             rochester['E_loss_tot'].sum()]
heat_loss_y =pd.Series(heat_loss_y, index=["london", "stuttgart", "madrid", "rome", "miami", "fresno", "olympia", "rochester"])
heat_loss_y


thermal_energy_network= thermal_y - electric_y
#thermal_energy_network =pd.Series(thermal_energy_network)

thermal_peak=[london['Total_consumption'].max(), 
              stuttgart['Total_consumption'].max(), 
              madrid['Total_consumption'].max(), 
              rome['Total_consumption'].max(), 
              miami['Total_consumption'].max(), 
              fresno['Total_consumption'].max(), 
              olympia['Total_consumption'].max(), 
              rochester['Total_consumption'].max()]
thermal_peak_MW=pd.Series(thermal_peak, index=["london", "stuttgart", "madrid", "rome", "miami", "fresno", "olympia", "rochester"])


COP = thermal_y / electric_y

source1_thermal_y= [london['heat_source1'].sum(), 
                    stuttgart['heat_source1'].sum(), 
                    madrid['heat_source1'].sum(), 
                    rome['heat_source1'].sum(), 
                    miami['heat_source1'].sum(), 
                    fresno['heat_source1'].sum(), 
                    olympia['heat_source1'].sum(), 
                    rochester['heat_source1'].sum()]

source2_thermal_y= [london['heat_source2'].sum(), 
                    stuttgart['heat_source2'].sum(), 
                    madrid['heat_source2'].sum(), 
                    rome['heat_source2'].sum(), 
                    miami['heat_source2'].sum(), 
                    fresno['heat_source2'].sum(), 
                    olympia['heat_source2'].sum(), 
                    rochester['heat_source2'].sum()]

source3_aquifer= [london['heat_aquifer'].sum(), 
                  stuttgart['heat_aquifer'].sum(), 
                  madrid['heat_aquifer'].sum(), 
                  rome['heat_aquifer'].sum(), 
                  miami['heat_aquifer'].sum(), 
                  fresno['heat_aquifer'].sum(), 
                  olympia['heat_aquifer'].sum(), 
                  rochester['heat_aquifer'].sum()]

source1_thermal_y =pd.Series(source1_thermal_y, index=["london", "stuttgart", "madrid", "rome", "miami", "fresno", "olympia", "rochester"])
source2_thermal_y =pd.Series(source2_thermal_y, index=["london", "stuttgart", "madrid", "rome", "miami", "fresno", "olympia", "rochester"])
source3_aquifer_y=pd.Series(source3_aquifer, index=["london", "stuttgart", "madrid", "rome", "miami", "fresno", "olympia", "rochester"])

waste_heat_y=source1_thermal_y+source2_thermal_y


pd.DataFrame([thermal_y,
              electric_y,
              waste_heat_y,
              source3_aquifer_y,
              heat_loss_y,
              source1_thermal_y,
              source2_thermal_y,
             thermal_energy_network,],
             index=["thermal_demand", 
                    "electricity", 
                    "waste_heat", 
                    "ground_source", 
                    "thermal_loss",
                    "source1", 
                    "source2",
                   "thermal_energy_network"]).T["thermal_energy_network"]

In [ ]:
thermal_load_y=[london['Total_consumption'].sum(), stuttgart['Total_consumption'].sum(), madrid['Total_consumption'].sum(), rome['Total_consumption'].sum(), miami['Total_consumption'].sum(), fresno['Total_consumption'].sum(), olympia['Total_consumption'].sum(), rochester['Total_consumption'].sum()]
thermal_y =pd.Series(thermal_load_y)
thermal_y

electric_cons_y=[london['E_el'].sum(), stuttgart['E_el'].sum(), madrid['E_el'].sum(),rome['E_el'].sum(), miami['E_el'].sum(), fresno['E_el'].sum(), olympia['E_el'].sum(), rochester['E_el'].sum()]
electric_y =pd.Series(electric_cons_y)
electric_y

heat_loss_y=[london['E_loss_tot'].sum(), stuttgart['E_loss_tot'].sum(), madrid['E_loss_tot'].sum(),rome['E_loss_tot'].sum(), miami['E_loss_tot'].sum(), fresno['E_loss_tot'].sum(), olympia['E_loss_tot'].sum(), rochester['E_loss_tot'].sum()]
heat_loss_y =pd.Series(heat_loss_y)
heat_loss_y

thermal_energy_network= thermal_y+heat_loss_y+electric_y
thermal_energy_network =pd.Series(thermal_energy_network)

thermal_peak=[london['Total_consumption'].max(), stuttgart['Total_consumption'].max(), madrid['Total_consumption'].max(), rome['Total_consumption'].max(), miami['Total_consumption'].max(), fresno['Total_consumption'].max(), olympia['Total_consumption'].max(), rochester['Total_consumption'].max()]
thermal_peak_MW=pd.Series(thermal_peak)


COP=thermal_y/electric_y
COP

source1_thermal_y= [london['heat_source1'].sum(), stuttgart['heat_source1'].sum(), madrid['heat_source1'].sum(), rome['heat_source1'].sum(), miami['heat_source1'].sum(), fresno['heat_source1'].sum(), olympia['heat_source1'].sum(), rochester['heat_source1'].sum()]
source2_thermal_y= [london['heat_source2'].sum(), stuttgart['heat_source2'].sum(), madrid['heat_source2'].sum(), rome['heat_source2'].sum(), miami['heat_source2'].sum(), fresno['heat_source2'].sum(), olympia['heat_source2'].sum(), rochester['heat_source2'].sum()]
source3_aquifer= [london['heat_aquifer'].sum(), stuttgart['heat_aquifer'].sum(), madrid['heat_aquifer'].sum(), rome['heat_aquifer'].sum(), miami['heat_aquifer'].sum(), fresno['heat_aquifer'].sum(), olympia['heat_aquifer'].sum(), rochester['heat_aquifer'].sum()]


source1_thermal_y =pd.Series(source1_thermal_y)
source2_thermal_y =pd.Series(source2_thermal_y)
source3_aquifer_y=pd.Series(source3_aquifer)

waste_heat_y=source1_thermal_y+source2_thermal_y




- We performed the calculations on the notebook for the annual analysis.



- And used Flourish to visualize them. These visualizations are [here](https://public.flourish.studio/story/549099/)

# Thanks for your attention!

## ... There is more 🥳

- Due to time constraints, we focused on the most important visualizations for a project like this

- Fitted Air, Ground & Aquifer Temperatures
- Thermal Loads
- Heat Pumps
- Heat Losses
- Electrical Use


## Fitted Air Temperature

In [ ]:
date_air_temp_air_temp_fit_hover_tool = HoverTool(tooltips=[('Date', '@timestamp{%Y/%m/%d %H:%M:%S}'),
                                                            ('Air Temperature', '@air_temp{%0.2f ºC}'),
                                                            ('Fit Air Temperature', '@air_temp_fit{%0.2f ºC}'),],
                                                  formatters={'@timestamp': 'datetime',
                                                              '@air_temp': 'printf',
                                                              '@air_temp_fit': 'printf',},)

date_air_temp_fit_hover_tool = HoverTool(tooltips=[('Date', '@timestamp{%Y/%m/%d %H:%M:%S}'),
                                                   ('Fit Air Temperature', '@air_temp_fit{%0.2f ºC}'),],
                                         formatters={'@timestamp': 'datetime',
                                                     '@air_temp_fit': 'printf',},)

max_temp = int(us_data.air_temp.max() + 5)
min_temp = int(us_data.air_temp.min() - 5)

### USA

In [ ]:
miami_air_temp_and_fit = figure(x_axis_type="datetime",
                                title="Miami, FL",
                                x_axis_label="Date",
                                y_axis_label="Temperature [ºC]",
                                y_range=(min_temp, max_temp),
                                **FIGURE_TOOLS)
miami_air_temp_and_fit.add_tools(date_air_temp_air_temp_fit_hover_tool)
l1 = miami_air_temp_and_fit.line(x="timestamp", y="air_temp", color=color_map["miami"], alpha=0.4, source=us_source, view=miami_view)
l2 = miami_air_temp_and_fit.circle(x="timestamp", y="air_temp_fit", color=color_map["miami"], alpha=0.4, source=us_source, view=miami_view, size=1)
miami_air_temp_and_fit.add_layout(Legend(items=[("Air Temp", [l1]), ("Fit Air Temp" , [l2])]), 'below')

fresno_air_temp_and_fit = figure(x_axis_type="datetime",
                                 title="Fresno, CA",
                                 x_axis_label="Date",
                                 y_axis_label="Temperature [ºC]",
                                 y_range=miami_air_temp_and_fit.y_range,
                                 x_range=miami_air_temp_and_fit.x_range,
                                 **FIGURE_TOOLS)
fresno_air_temp_and_fit.add_tools(date_air_temp_air_temp_fit_hover_tool)
l1 = fresno_air_temp_and_fit.line(x="timestamp", y="air_temp", color=color_map["fresno"], alpha=0.2, source=us_source, view=fresno_view)
l2 = fresno_air_temp_and_fit.circle(x="timestamp", y="air_temp_fit", color=color_map["fresno"], alpha=0.4, source=us_source, view=fresno_view, size=1)
fresno_air_temp_and_fit.add_layout(Legend(items=[("Air Temp", [l1]), ("Fit Air Temp" , [l2])]), 'below')


olympia_air_temp_and_fit = figure(x_axis_type="datetime",
                                  title="Olympia, WA",
                                  x_axis_label="Date",
                                  y_axis_label="Temperature [ºC]",
                                  y_range=miami_air_temp_and_fit.y_range,
                                  x_range=miami_air_temp_and_fit.x_range,
                                  **FIGURE_TOOLS)
olympia_air_temp_and_fit.add_tools(date_air_temp_air_temp_fit_hover_tool)
l1 = olympia_air_temp_and_fit.line(x="timestamp", y="air_temp", color=color_map["olympia"], alpha=0.2, source=us_source, view=olympia_view)
l2 = olympia_air_temp_and_fit.circle(x="timestamp", y="air_temp_fit", color=color_map["olympia"], alpha=0.4, source=us_source, view=olympia_view, size=1)
olympia_air_temp_and_fit.add_layout(Legend(items=[("Air Temp", [l1]), ("Fit Air Temp" , [l2])]), 'below')


rochester_air_temp_and_fit = figure(x_axis_type="datetime",
                                    title="Rochester, NY",
                                    x_axis_label="Date",
                                    y_axis_label="Temperature [ºC]",
                                    y_range=miami_air_temp_and_fit.y_range,
                                    x_range=miami_air_temp_and_fit.x_range,
                                    **FIGURE_TOOLS)
rochester_air_temp_and_fit.add_tools(date_air_temp_air_temp_fit_hover_tool)
l1 = rochester_air_temp_and_fit.line(x="timestamp", y="air_temp", color=color_map["rochester"], alpha=0.2, source=us_source, view=rochester_view)
l2 = rochester_air_temp_and_fit.circle(x="timestamp", y="air_temp_fit", color=color_map["rochester"], alpha=0.4, source=us_source, view=rochester_view, size=1)
rochester_air_temp_and_fit.add_layout(Legend(items=[("Air Temp", [l1]), ("Fit Air Temp" , [l2])]), 'below')

In [ ]:
only_air_temp_fit = figure(x_axis_type="datetime",
                           title="Adjusted Air Temperature",
                           x_axis_label="Date",
                           y_axis_label="Temperature [ºC]",
                           y_range=miami_air_temp_and_fit.y_range,
                           x_range=miami_air_temp_and_fit.x_range,
                           **{**FIGURE_TOOLS, **FULL_FIGURE_TOOLS})
only_air_temp_fit.add_tools(date_air_temp_fit_hover_tool)
l1 = only_air_temp_fit.line(x="timestamp", y="air_temp_fit", color=color_map["miami"], alpha=0.7, source=us_source, view=miami_view, line_width=3)
l2 = only_air_temp_fit.line(x="timestamp", y="air_temp_fit", color=color_map["fresno"], alpha=0.7, source=us_source, view=fresno_view, line_width=3)
l3 = only_air_temp_fit.line(x="timestamp", y="air_temp_fit", color=color_map["olympia"], alpha=0.7, source=us_source, view=olympia_view, line_width=3)
l4 = only_air_temp_fit.line(x="timestamp", y="air_temp_fit", color=color_map["rochester"], alpha=0.7, source=us_source, view=rochester_view, line_width=3)
only_air_temp_fit.add_layout(Legend(items=[("Miami", [l1]), ("Fresno" , [l2]), ("Olympia", [l3]), ("Rochester" , [l4])]), 'below')


In [ ]:
show(layout([[rochester_air_temp_and_fit, olympia_air_temp_and_fit],[fresno_air_temp_and_fit, miami_air_temp_and_fit],]))

In [ ]:
show(layout([[only_air_temp_fit],]))

### Europe

In [ ]:
max_temp = int(europe_data.air_temp.max() + 5)
min_temp = int(europe_data.air_temp.min() - 5)

london_air_temp_and_fit = figure(x_axis_type="datetime",
                                 title="London, United Kingdom",
                                 x_axis_label="Date",
                                 y_axis_label="Temperature [ºC]",
                                 y_range=(min_temp, max_temp),
                                 **FIGURE_TOOLS)
london_air_temp_and_fit.add_tools(date_air_temp_air_temp_fit_hover_tool)
l1 = london_air_temp_and_fit.line(x="timestamp", y="air_temp", color=color_map["london"], alpha=0.4, source=europe_source, view=london_view)
l2 = london_air_temp_and_fit.circle(x="timestamp", y="air_temp_fit", color=color_map["london"], alpha=0.4, source=europe_source, view=london_view, size=1)
london_air_temp_and_fit.add_layout(Legend(items=[("Air Temp", [l1]), ("Fit Air Temp" , [l2])]), 'below')


madrid_air_temp_and_fit = figure(x_axis_type="datetime",
                                 title="Madrid, Spain",
                                 x_axis_label="Date",
                                 y_axis_label="Temperature [ºC]",
                                 y_range=london_air_temp_and_fit.y_range,
                                 x_range=london_air_temp_and_fit.x_range,
                                 **FIGURE_TOOLS)
madrid_air_temp_and_fit.add_tools(date_air_temp_air_temp_fit_hover_tool)
l1 = madrid_air_temp_and_fit.line(x="timestamp", y="air_temp", color=color_map["madrid"], alpha=0.4, source=europe_source, view=madrid_view)
l2 = madrid_air_temp_and_fit.circle(x="timestamp", y="air_temp_fit", color=color_map["madrid"], alpha=0.4, source=europe_source, view=madrid_view, size=1)
madrid_air_temp_and_fit.add_layout(Legend(items=[("Air Temp", [l1]), ("Fit Air Temp" , [l2])]), 'below')

rome_air_temp_and_fit = figure(x_axis_type="datetime",
                               title="Rome, Italy",
                               x_axis_label="Date",
                               y_axis_label="Temperature [ºC]",
                               y_range=london_air_temp_and_fit.y_range,
                               x_range=london_air_temp_and_fit.x_range,
                               **FIGURE_TOOLS)
rome_air_temp_and_fit.add_tools(date_air_temp_air_temp_fit_hover_tool)
l1 = rome_air_temp_and_fit.line(x="timestamp", y="air_temp", color=color_map["rome"], alpha=0.4, source=europe_source, view=rome_view)
l2 = rome_air_temp_and_fit.circle(x="timestamp", y="air_temp_fit", color=color_map["rome"], alpha=0.4, source=europe_source, view=rome_view, size=1)
rome_air_temp_and_fit.add_layout(Legend(items=[("Air Temp", [l1]), ("Fit Air Temp" , [l2])]), 'below')


stuttgart_air_temp_and_fit = figure(x_axis_type="datetime",
                                    title="Stuttgart, Germany",
                                    x_axis_label="Date",
                                    y_axis_label="Temperature [ºC]",
                                    y_range=london_air_temp_and_fit.y_range,
                                    x_range=london_air_temp_and_fit.x_range,
                                    **FIGURE_TOOLS)
stuttgart_air_temp_and_fit.add_tools(date_air_temp_air_temp_fit_hover_tool)
l1 = stuttgart_air_temp_and_fit.line(x="timestamp", y="air_temp", color=color_map["stuttgart"], alpha=0.4, source=europe_source, view=stuttgart_view)
l2 = stuttgart_air_temp_and_fit.circle(x="timestamp", y="air_temp_fit", color=color_map["stuttgart"], alpha=0.4, source=europe_source, view=stuttgart_view, size=1)
stuttgart_air_temp_and_fit.add_layout(Legend(items=[("Air Temp", [l1]), ("Fit Air Temp" , [l2])]), 'below')


only_air_temp_fit = figure(x_axis_type="datetime",
                                title="Adjusted air temperature",
                          y_range=london_air_temp_and_fit.y_range,
                                x_range=london_air_temp_and_fit.x_range,
                           **{**FIGURE_TOOLS, **FULL_WIDTH_FIGURE_TOOLS})
only_air_temp_fit.add_tools(date_air_temp_fit_hover_tool)
l1 = only_air_temp_fit.line(x="timestamp", y="air_temp_fit", color=color_map["london"], alpha=0.7, source=europe_source, view=london_view, line_width=3)
l2 = only_air_temp_fit.line(x="timestamp", y="air_temp_fit", color=color_map["stuttgart"], alpha=0.7, source=europe_source, view=stuttgart_view, line_width=3)
l3 = only_air_temp_fit.line(x="timestamp", y="air_temp_fit", color=color_map["rome"], alpha=0.7, source=europe_source, view=rome_view, line_width=3)
l4 = only_air_temp_fit.line(x="timestamp", y="air_temp_fit", color=color_map["madrid"], alpha=0.7, source=europe_source, view=madrid_view, line_width=3)
only_air_temp_fit.add_layout(Legend(items=[("London, UK", [l1]), ("Stuttgart, Germany" , [l2]), ("Rome, Italy", [l3]), ("Madrid, Spain" , [l4])]), 'below')




In [ ]:
show(layout([[london_air_temp_and_fit, stuttgart_air_temp_and_fit ,],[rome_air_temp_and_fit, madrid_air_temp_and_fit],]))

In [ ]:
show(layout([[only_air_temp_fit],]))

## Ground Temperature


In [ ]:
date_air_temp_ground_temp_hover_tool = HoverTool(tooltips=[('Date', '@timestamp{%Y/%m/%d %H:%M:%S}'),
                                                           ('Air Temperature', '@air_temp{%0.2f ºC}'),
                                                           ('Ground Temperature', '@ground_temp{%0.2f ºC}'),],
                                                 formatters={'@timestamp': 'datetime',
                                                             '@air_temp': 'printf',
                                                             '@ground_temp': 'printf',},)

date_ground_temp_hover_tool = HoverTool(tooltips=[('Date', '@timestamp{%Y/%m/%d %H:%M:%S}'),
                                                  ('Ground Temperature', '@ground_temp{%0.2f ºC}'),],
                                        formatters={'@timestamp': 'datetime',
                                                    '@ground_temp': 'printf',},)

date_hour_ground_temp_hover_tool = HoverTool(tooltips=[('Date', '@date{%Y/%m/%d}'),
                                                       ("Hour", "@hour"),
                                                       ('Ground Temperature', '@ground_temp{%0.2f ºC}'),],
                                             formatters={'@date': 'datetime',
                                                         "@hour": "numeral",
                                                         '@ground_temp': 'printf',},)

### USA


In [ ]:
max_temp = int(us_data.air_temp.max() + 5)
min_temp = int(us_data.air_temp.min() - 5)

miami_air_temp_and_ground = figure(x_axis_type="datetime",
                                   title="Miami, FL",
                                   x_axis_label="Date",
                                   y_axis_label="Temperature [ºC]",
                                   y_range=(min_temp, max_temp),
                                   **FIGURE_TOOLS)
miami_air_temp_and_ground.add_tools(date_air_temp_ground_temp_hover_tool)
l1 = miami_air_temp_and_ground.line(x="timestamp", y="air_temp", color=color_map["miami"], alpha=0.4, source=us_source, view=miami_view)
l2 = miami_air_temp_and_ground.circle(x="timestamp", y="ground_temp", color=color_map["miami"], alpha=0.4, source=us_source, view=miami_view, size=1)
miami_air_temp_and_ground.add_layout(Legend(items=[("Air Temp", [l1]), ("Ground Temp" , [l2])]), 'below')


fresno_air_temp_and_ground = figure(x_axis_type="datetime",
                                    title="Fresno, California, USA",
                                    x_axis_label="Date",
                                    y_axis_label="Temperature [ºC]",
                                    y_range=miami_air_temp_and_fit.y_range,
                                    x_range=miami_air_temp_and_fit.x_range,
                                    **FIGURE_TOOLS)
fresno_air_temp_and_ground.add_tools(date_air_temp_ground_temp_hover_tool)
l1 = fresno_air_temp_and_ground.line(x="timestamp", y="air_temp", color=color_map["fresno"], alpha=0.4, source=us_source, view=fresno_view)
l2 = fresno_air_temp_and_ground.circle(x="timestamp", y="ground_temp", color=color_map["fresno"], alpha=0.4, source=us_source, view=fresno_view, size=1)
fresno_air_temp_and_ground.add_layout(Legend(items=[("Air Temp", [l1]), ("Ground Temp" , [l2])]), 'below')


olympia_air_temp_and_ground = figure(x_axis_type="datetime",
                                     title="Olympia, Washington, USA",
                                     x_axis_label="Date",
                                     y_axis_label="Temperature [ºC]",
                                     y_range=miami_air_temp_and_fit.y_range,
                                     x_range=miami_air_temp_and_fit.x_range,
                                     **FIGURE_TOOLS)
olympia_air_temp_and_ground.add_tools(date_air_temp_ground_temp_hover_tool)
l1 = olympia_air_temp_and_ground.line(x="timestamp", y="air_temp", color=color_map["olympia"], alpha=0.4, source=us_source, view=olympia_view)
l2 = olympia_air_temp_and_ground.circle(x="timestamp", y="ground_temp", color=color_map["olympia"], alpha=0.4, source=us_source, view=olympia_view, size=1)
olympia_air_temp_and_ground.add_layout(Legend(items=[("Air Temp", [l1]), ("Ground Temp" , [l2])]), 'below')

rochester_air_temp_and_ground = figure(x_axis_type="datetime",
                                       title="Rochester, New York, USA",
                                       x_axis_label="Date",
                                       y_axis_label="Temperature [ºC]",
                                       y_range=miami_air_temp_and_fit.y_range,
                                       x_range=miami_air_temp_and_fit.x_range,
                                       **FIGURE_TOOLS)
rochester_air_temp_and_ground.add_tools(date_air_temp_ground_temp_hover_tool)
l1 = rochester_air_temp_and_ground.line(x="timestamp", y="air_temp", color=color_map["rochester"], alpha=0.4, source=us_source, view=rochester_view)
l2 = rochester_air_temp_and_ground.circle(x="timestamp", y="ground_temp", color=color_map["rochester"], alpha=0.4, source=us_source, view=rochester_view, size=1)
rochester_air_temp_and_ground.add_layout(Legend(items=[("Air Temp", [l1]), ("Ground Temp" , [l2])]), 'below')


ground_temp = figure(x_axis_type="datetime",
                     title="Ground temperature - 1m depth",
                     x_axis_label="Date",
                     y_axis_label="Temperature [ºC]",
                     y_range=london_air_temp_and_fit.y_range,
                     x_range=london_air_temp_and_fit.x_range,
                     **{**FIGURE_TOOLS, **FULL_WIDTH_FIGURE_TOOLS})
ground_temp.add_tools(date_ground_temp_hover_tool)
l1 = ground_temp.line(x="timestamp", y="ground_temp", color=color_map["miami"], alpha=0.7, source=us_source, view=miami_view, line_width=3)
l2 = ground_temp.line(x="timestamp", y="ground_temp", color=color_map["fresno"], alpha=0.7, source=us_source, view=fresno_view, line_width=3)
l3 = ground_temp.line(x="timestamp", y="ground_temp", color=color_map["olympia"], alpha=0.7, source=us_source, view=olympia_view, line_width=3)
l4 = ground_temp.line(x="timestamp", y="ground_temp", color=color_map["rochester"], alpha=0.7, source=us_source, view=rochester_view, line_width=3)
ground_temp.add_layout(Legend(items=[("Miami, FL", [l1]), ("Fresno, CA" , [l2]), ("Olympia, WA", [l3]), ("Rochester, NY" , [l4])]), 'below')

mapper = LinearColorMapper(palette=Spectral11, low=us_data["ground_temp"].min(), high=us_data["ground_temp"].max())
color_bar = ColorBar(color_mapper=mapper, location=(0, 0))

miami_ground_temp_heatmap = heatmap(color_bar, title="Miami, FL - Ground Temperature - 1m depth", x_axis_label="Day in year", y_axis_label="Hour of day", x_axis_type="datetime", **FIGURE_TOOLS)
miami_ground_temp_heatmap.add_tools(date_hour_ground_temp_hover_tool)
miami_ground_temp_heatmap.rect(x="date", y="hour", width=ONE_DAY_WIDTH, height=1, source=us_source, view=miami_view, line_color=None, fill_color=transform('ground_temp', mapper))

fresno_ground_temp_heatmap = heatmap(color_bar, title="Fresno, CA - Ground Temperature - 1m depth", x_axis_label="Day in year", y_axis_label="Hour of day", x_axis_type="datetime", **FIGURE_TOOLS)
fresno_ground_temp_heatmap.add_tools(date_hour_ground_temp_hover_tool)
fresno_ground_temp_heatmap.rect(x="date", y="hour", width=ONE_DAY_WIDTH, height=1, source=us_source, view=fresno_view, line_color=None, fill_color=transform('ground_temp', mapper))

olympia_ground_temp_heatmap = heatmap(color_bar, title="Olympia, WA - Ground Temperature - 1m depth", x_axis_label="Day in year", y_axis_label="Hour of day", x_axis_type="datetime", **FIGURE_TOOLS)
olympia_ground_temp_heatmap.add_tools(date_hour_ground_temp_hover_tool)
olympia_ground_temp_heatmap.rect(x="date", y="hour", width=ONE_DAY_WIDTH, height=1, source=us_source, view=olympia_view, line_color=None, fill_color=transform('ground_temp', mapper))

rochester_ground_temp_heatmap = heatmap(color_bar, title="Rochester, NY - Ground Temperature - 1m depth", x_axis_label="Day in year", y_axis_label="Hour of day", x_axis_type="datetime", **FIGURE_TOOLS)
rochester_ground_temp_heatmap.add_tools(date_hour_ground_temp_hover_tool)
rochester_ground_temp_heatmap.rect(x="date", y="hour", width=ONE_DAY_WIDTH, height=1, source=us_source, view=rochester_view, line_color=None, fill_color=transform('ground_temp', mapper))


In [ ]:
show(layout([[rochester_air_temp_and_ground, olympia_air_temp_and_ground, ],[fresno_air_temp_and_ground, miami_air_temp_and_ground, ],]))

In [ ]:
show(layout([[ground_temp],]))

In [ ]:
show(layout([[rochester_ground_temp_heatmap, olympia_ground_temp_heatmap, ],[fresno_ground_temp_heatmap, miami_ground_temp_heatmap, ],]))

### Europe

In [ ]:
max_temp = int(europe_data.air_temp.max() + 5)
min_temp = int(europe_data.air_temp.min() - 5)

london_ground_temp = figure(x_axis_type="datetime",
                                 title="London, United Kingdom",x_axis_label="Date",
                                 y_axis_label="Temperature [ºC]",
                                 y_range=(min_temp, max_temp),
                                 **FIGURE_TOOLS)
london_ground_temp.add_tools(date_air_temp_ground_temp_hover_tool)
l1 = london_ground_temp.line(x="timestamp", y="air_temp", color=color_map["london"], alpha=0.4, source=europe_source, view=london_view)
l2 = london_ground_temp.circle(x="timestamp", y="ground_temp", color=color_map["london"], alpha=0.4, source=europe_source, view=london_view, size=1)
london_ground_temp.add_layout(Legend(items=[("Air Temp", [l1]), ("Ground Temp" , [l2])]), 'below')

madrid_ground_temp = figure(x_axis_type="datetime",
                                 title="Madrid, Spain",
                                 x_axis_label="Date",
                                 y_axis_label="Temperature [ºC]",
                                 y_range=london_air_temp_and_fit.y_range,
                                 x_range=london_air_temp_and_fit.x_range,
                                 **FIGURE_TOOLS)
madrid_ground_temp.add_tools(date_air_temp_ground_temp_hover_tool)
l1 = madrid_ground_temp.line(x="timestamp", y="air_temp", color=color_map["madrid"], alpha=0.4, source=europe_source, view=madrid_view)
l2 = madrid_ground_temp.circle(x="timestamp", y="ground_temp", color=color_map["madrid"], alpha=0.4, source=europe_source, view=madrid_view, size=1)
madrid_ground_temp.add_layout(Legend(items=[("Air Temp", [l1]), ("Ground Temp" , [l2])]), 'below')


rome_ground_temp = figure(x_axis_type="datetime",
                               title="Rome, Italy",
                               x_axis_label="Date",
                               y_axis_label="Temperature [ºC]",
                               y_range=london_air_temp_and_fit.y_range,
                               x_range=london_air_temp_and_fit.x_range,
                               **FIGURE_TOOLS)
rome_ground_temp.add_tools(date_air_temp_ground_temp_hover_tool)
l1 = rome_ground_temp.line(x="timestamp", y="air_temp", color=color_map["rome"], alpha=0.4, source=europe_source, view=rome_view)
l2 = rome_ground_temp.circle(x="timestamp", y="ground_temp", color=color_map["rome"], alpha=0.4, source=europe_source, view=rome_view, size=1)
rome_ground_temp.add_layout(Legend(items=[("Air Temp", [l1]), ("Ground Temp" , [l2])]), 'below')

stuttgart_ground_temp = figure(x_axis_type="datetime",
                                    title="Stuttgart, Germany",
                                    x_axis_label="Date",
                                    y_axis_label="Temperature [ºC]",
                                    y_range=london_air_temp_and_fit.y_range,
                                    x_range=london_air_temp_and_fit.x_range,
                                    **FIGURE_TOOLS)
stuttgart_ground_temp.add_tools(date_air_temp_ground_temp_hover_tool)
l1 = stuttgart_ground_temp.line(x="timestamp", y="air_temp", color=color_map["stuttgart"], alpha=0.4, source=europe_source, view=stuttgart_view)
l2 = stuttgart_ground_temp.circle(x="timestamp", y="ground_temp", color=color_map["stuttgart"], alpha=0.4, source=europe_source, view=stuttgart_view, size=1)
stuttgart_ground_temp.add_layout(Legend(items=[("Air Temp", [l1]), ("Ground Temp" , [l2])]), 'below')

europe_ground_temp = figure(x_axis_type="datetime",
                            title="Ground temperature - 1m depth",
                            x_axis_label="Date",
                            y_axis_label="Temperature [ºC]",
                            y_range=london_air_temp_and_fit.y_range,
                            x_range=london_air_temp_and_fit.x_range,
                            **{**FIGURE_TOOLS, **FULL_WIDTH_FIGURE_TOOLS})
europe_ground_temp.add_tools(date_ground_temp_hover_tool)
l1 = europe_ground_temp.line(x="timestamp", y="ground_temp", color=color_map["london"], alpha=0.7, source=europe_source, view=london_view, line_width=3)
l2 = europe_ground_temp.line(x="timestamp", y="ground_temp", color=color_map["stuttgart"], alpha=0.7, source=europe_source, view=stuttgart_view, line_width=3)
l3 = europe_ground_temp.line(x="timestamp", y="ground_temp", color=color_map["rome"], alpha=0.7, source=europe_source, view=rome_view, line_width=3)
l4 = europe_ground_temp.line(x="timestamp", y="ground_temp", color=color_map["madrid"], alpha=0.7, source=europe_source, view=madrid_view, line_width=3)
europe_ground_temp.add_layout(Legend(items=[("London, UK", [l1]), ("Stuttgart, Germany" , [l2]), ("Rome, Italy", [l3]), ("Madrid, Spain" , [l4])]), 'below')

mapper = LinearColorMapper(palette=Spectral11, low=europe_data["ground_temp"].min(), high=europe_data["ground_temp"].max())
color_bar = ColorBar(color_mapper=mapper, location=(0, 0))

london_ground_temp_heatmap = heatmap(color_bar, title="London, UK - Ground Temperature - 1m depth", x_axis_label="Day in year", y_axis_label="Hour of day", x_axis_type="datetime", **FIGURE_TOOLS)
london_ground_temp_heatmap.add_tools(date_hour_ground_temp_hover_tool)
london_ground_temp_heatmap.rect(x="date", y="hour", width=ONE_DAY_WIDTH, height=1, source=europe_source, view=london_view, line_color=None, fill_color=transform('ground_temp', mapper))

sttutgart_ground_temp_heatmap = heatmap(color_bar, title="Stuttgart, Germany - Ground Temperature - 1m depth", x_axis_label="Day in year", y_axis_label="Hour of day", x_axis_type="datetime", **FIGURE_TOOLS)
sttutgart_ground_temp_heatmap.add_tools(date_hour_ground_temp_hover_tool)
sttutgart_ground_temp_heatmap.rect(x="date", y="hour", width=ONE_DAY_WIDTH, height=1, source=europe_source, view=stuttgart_view, line_color=None, fill_color=transform('ground_temp', mapper))

rome_ground_temp_heatmap = heatmap(color_bar, title="Rome, Italy - Ground Temperature - 1m depth", x_axis_label="Day in year", y_axis_label="Hour of day", x_axis_type="datetime", **FIGURE_TOOLS)
rome_ground_temp_heatmap.add_tools(date_hour_ground_temp_hover_tool)
rome_ground_temp_heatmap.rect(x="date", y="hour", width=ONE_DAY_WIDTH, height=1, source=europe_source, view=rome_view, line_color=None, fill_color=transform('ground_temp', mapper))

madrid_ground_temp_heatmap = heatmap(color_bar, title="Madrid, Spain - Ground Temperature - 1m depth", x_axis_label="Day in year", y_axis_label="Hour of day", x_axis_type="datetime", **FIGURE_TOOLS)
madrid_ground_temp_heatmap.add_tools(date_hour_ground_temp_hover_tool)
madrid_ground_temp_heatmap.rect(x="date", y="hour", width=ONE_DAY_WIDTH, height=1, source=europe_source, view=madrid_view, line_color=None, fill_color=transform('ground_temp', mapper))



In [ ]:
show(layout([[london_ground_temp, stuttgart_ground_temp,],[rome_ground_temp, madrid_ground_temp],]))

In [ ]:
show(layout([[europe_ground_temp],]))

In [ ]:
show(layout([[london_ground_temp_heatmap, sttutgart_ground_temp_heatmap],[rome_ground_temp_heatmap, madrid_ground_temp_heatmap],]))

## Aquifer Temperature

In [ ]:
aquifer_temp_hover_tool = HoverTool(tooltips=[('City, Statistic', '@x'),
                                              ('Value', '@counts{%0.2f ºC}'),],
                                    formatters={'@counts': 'printf',
                                                '@x': 'printf',})

### USA

In [ ]:
cities = ["Miami, FL", "Fresno, CA", "Olympia, WA", "Rochester, NY"]
statistics = ['Aquifer Temp','Mean Air Temp']
mdata = {'cities' : cities,
         'Aquifer Temp': [miami.aquifer_temp.head(1).item(), 
                          fresno.aquifer_temp.head(1).item(),
                          olympia.aquifer_temp.head(1).item(), 
                          rochester.aquifer_temp.head(1).item()],
         'Mean Air Temp': [miami.air_temp.mean(), 
                           fresno.air_temp.mean(),
                           olympia.air_temp.mean(), 
                           rochester.air_temp.mean()],}

x=[(city, stat) for city in cities for stat in statistics]
counts = sum(zip(mdata['Aquifer Temp'], mdata['Mean Air Temp']), ())

source = ColumnDataSource(data=dict(x=x, 
                                    counts=counts, 
                                    color=[color_map["miami"], color_map["miami"], 
                                           color_map["fresno"], color_map["fresno"],
                                           color_map["olympia"], color_map["olympia"], 
                                           color_map["rochester"], color_map["rochester"]],
                                   alpha=[0.8, 0.4,
                                          0.8, 0.4,
                                          0.8, 0.4,
                                          0.8, 0.4,]))

aquifer_temp = figure(x_range=FactorRange(*x), 
                      title="Aquifer & mean air temperatures across cities in the USA",
                      y_axis_label="Temperature [ºC]",
                      **{**FIGURE_TOOLS, **FULL_FIGURE_TOOLS})
aquifer_temp.add_tools(aquifer_temp_hover_tool)
aquifer_temp.vbar(x='x', top='counts', fill_color='color', alpha="alpha", width=0.9, source=source)
aquifer_temp.y_range.start = 0
aquifer_temp.x_range.range_padding = 0.1
aquifer_temp.xaxis.major_label_orientation = .9
aquifer_temp.xgrid.grid_line_color = None


In [ ]:
show(aquifer_temp)

### Europe

In [ ]:
cities = ["London, UK", "Stuttgart, Germany", "Madrid, Spain", "Rome, Italy"]
statistics = ['Aquifer Temp','Mean Air Temp']
mdata = {'cities' : cities,
         'Aquifer Temp': [london.aquifer_temp.head(1).item(), 
                          stuttgart.aquifer_temp.head(1).item(),
                          madrid.aquifer_temp.head(1).item(), 
                          rome.aquifer_temp.head(1).item()],
         'Mean Air Temp': [london.air_temp.mean(), 
                           stuttgart.air_temp.mean(),
                           madrid.air_temp.mean(), 
                           rome.air_temp.mean()],}

x = [(city, stat) for city in cities for stat in statistics]
counts = sum(zip(mdata['Aquifer Temp'], mdata['Mean Air Temp']), ())

source = ColumnDataSource(data=dict(x=x, 
                                    counts=counts, 
                                    color=[color_map["london"], color_map["london"], 
                                           color_map["stuttgart"], color_map["stuttgart"],
                                           color_map["madrid"], color_map["madrid"], 
                                           color_map["rome"], color_map["rome"]],
                                   alpha=[0.8, 0.4,
                                          0.8, 0.4,
                                          0.8, 0.4,
                                          0.8, 0.4,]))

europe_aquifer_temp = figure(x_range=FactorRange(*x), 
                             title="Aquifer & mean air temperatures across cities in Europe",
                             y_axis_label="Temperature [ºC]",
                             **{**FIGURE_TOOLS, **FULL_FIGURE_TOOLS})
europe_aquifer_temp.add_tools(aquifer_temp_hover_tool)
europe_aquifer_temp.vbar(x='x', top='counts', fill_color='color', alpha="alpha", width=0.9, source=source)
europe_aquifer_temp.y_range.start = 0
europe_aquifer_temp.x_range.range_padding = 0.1
europe_aquifer_temp.xaxis.major_label_orientation = .9
europe_aquifer_temp.xgrid.grid_line_color = None


In [ ]:
show(europe_aquifer_temp)

## Thermal load by season and hour of the day

### USA

In [ ]:
hours = range(0,24)
seasons = ["winter", "spring", "summer", "fall"]
cities = [NOAA2010Dataset.MIAMI_FL, NOAA2010Dataset.FRESNO_CA, NOAA2010Dataset.OLYMPIA_WA, NOAA2010Dataset.ROCHESTER_NY]

total_consumption_average_per_hour = dict()
thermal_load_season_tool_by_city = dict()
for city in cities:
    for season in seasons:
        hours_for_city_and_season = np.empty(shape=(24,), dtype=np.float32)
        
        for hour in hours:
            hours_for_city_and_season[hour] = us_data[(us_data.city_key == city) & (us_data.season == season) & (us_data.hour == hour)]["Total_consumption"].mean() #/ 1000
            
        total_consumption_average_per_hour[f"{city}_{season}"] = hours_for_city_and_season
        thermal_load_season_tool_by_city[city] = HoverTool(tooltips=[("Hour", "@hour{%d}"),
                                                                     ('TL Winter', f'@{city}_winter' + '{%0.2f MW}'),
                                                                     ('TL Spring', f'@{city}_spring' + '{%0.2f MW}'),
                                                                     ('TL Summer', f'@{city}_summer' + '{%0.2f MW}'),
                                                                     ('TL Fall', f'@{city}_fall' + '{%0.2f MW}'),],
                                                           formatters={'@hour': 'printf',
                                                                       f'@{city}_winter': 'printf',
                                                                       f'@{city}_spring': 'printf',
                                                                       f'@{city}_summer': 'printf',
                                                                       f'@{city}_fall': "printf"},)

x=hours

usa_thermal_load_season_source = ColumnDataSource({"hour": hours, **total_consumption_average_per_hour})

In [ ]:
miami_thermal_load_by_seasons = figure(title="Miami, FL",
                                       x_axis_label="Hour of the day",
                                       y_axis_label="Thermal load [MW]",
                                       **FIGURE_TOOLS,
                                      )
miami_thermal_load_by_seasons.add_tools(thermal_load_season_tool_by_city[NOAA2010Dataset.MIAMI_FL])
l11 = miami_thermal_load_by_seasons.line(x="hour", y=f"{NOAA2010Dataset.MIAMI_FL}_winter", source=usa_thermal_load_season_source, color=color_map["winter"], alpha=0.7, line_width=3)
l12 = miami_thermal_load_by_seasons.square(x="hour", y=f"{NOAA2010Dataset.MIAMI_FL}_winter", source=usa_thermal_load_season_source, color=color_map["winter"], alpha=0.7, size=10)
l21 = miami_thermal_load_by_seasons.line(x="hour", y=f"{NOAA2010Dataset.MIAMI_FL}_spring", source=usa_thermal_load_season_source, color=color_map["spring"], alpha=0.7,  line_width=3)
l22 = miami_thermal_load_by_seasons.circle(x="hour", y=f"{NOAA2010Dataset.MIAMI_FL}_spring", source=usa_thermal_load_season_source, color=color_map["spring"], alpha=0.7,  size=10)
l31 = miami_thermal_load_by_seasons.line(x="hour", y=f"{NOAA2010Dataset.MIAMI_FL}_summer", source=usa_thermal_load_season_source, color=color_map["summer"], alpha=0.7,  line_width=3)
l32 = miami_thermal_load_by_seasons.diamond(x="hour", y=f"{NOAA2010Dataset.MIAMI_FL}_summer", source=usa_thermal_load_season_source, color=color_map["summer"], alpha=0.7,  size=10)
l41 = miami_thermal_load_by_seasons.line(x="hour", y=f"{NOAA2010Dataset.MIAMI_FL}_fall", source=usa_thermal_load_season_source, color=color_map["fall"], alpha=0.7, line_width=3)
l42 = miami_thermal_load_by_seasons.cross(x="hour", y=f"{NOAA2010Dataset.MIAMI_FL}_fall", source=usa_thermal_load_season_source, color=color_map["fall"], alpha=0.7, size=10)
miami_thermal_load_by_seasons.add_layout(Legend(items=[("Winter 2010", [l11, l12]), ("Spring 2010", [l21, l22]), ("Summer 2010", [l31, l32]), ("Fall 2010", [l41, l42]),],), "below")


rochester_thermal_load_by_seasons = figure(title="Rochester, NY",
                                           y_range=miami_thermal_load_by_seasons.y_range,
                                           x_range=miami_thermal_load_by_seasons.x_range,
                                           x_axis_label="Hour of the day",
                                           y_axis_label="Thermal load [MW]",
                                           **FIGURE_TOOLS,
                                          )
rochester_thermal_load_by_seasons.add_tools(thermal_load_season_tool_by_city[NOAA2010Dataset.ROCHESTER_NY])
l11 = rochester_thermal_load_by_seasons.line(x="hour", y=f"{NOAA2010Dataset.ROCHESTER_NY}_winter", source=usa_thermal_load_season_source, color=color_map["winter"], alpha=0.7, line_width=3)
l12 = rochester_thermal_load_by_seasons.square(x="hour", y=f"{NOAA2010Dataset.ROCHESTER_NY}_winter", source=usa_thermal_load_season_source, color=color_map["winter"], alpha=0.7, size=10)
l21 = rochester_thermal_load_by_seasons.line(x="hour", y=f"{NOAA2010Dataset.ROCHESTER_NY}_spring", source=usa_thermal_load_season_source, color=color_map["spring"], alpha=0.7,  line_width=3)
l22 = rochester_thermal_load_by_seasons.circle(x="hour", y=f"{NOAA2010Dataset.ROCHESTER_NY}_spring", source=usa_thermal_load_season_source, color=color_map["spring"], alpha=0.7,  size=10)
l31 = rochester_thermal_load_by_seasons.line(x="hour", y=f"{NOAA2010Dataset.ROCHESTER_NY}_summer", source=usa_thermal_load_season_source, color=color_map["summer"], alpha=0.7,  line_width=3)
l32 = rochester_thermal_load_by_seasons.diamond(x="hour", y=f"{NOAA2010Dataset.ROCHESTER_NY}_summer", source=usa_thermal_load_season_source, color=color_map["summer"], alpha=0.7,  size=10)
l41 = rochester_thermal_load_by_seasons.line(x="hour", y=f"{NOAA2010Dataset.ROCHESTER_NY}_fall", source=usa_thermal_load_season_source, color=color_map["fall"], alpha=0.7,  line_width=3)
l42 = rochester_thermal_load_by_seasons.cross(x="hour", y=f"{NOAA2010Dataset.ROCHESTER_NY}_fall", source=usa_thermal_load_season_source, color=color_map["fall"], alpha=0.7, size=10)
rochester_thermal_load_by_seasons.add_layout(Legend(items=[("Winter 2010", [l11, l12]), ("Spring 2010", [l21, l22]), ("Summer 2010", [l31, l32]), ("Fall 2010", [l41, l42]),],), "below")


olympia_thermal_load_by_seasons = figure(title="Olympia, WA",
                                         y_range=miami_thermal_load_by_seasons.y_range,
                                         x_range=miami_thermal_load_by_seasons.x_range,
                                         x_axis_label="Hour of the day",
                                         y_axis_label="Thermal load [MW]",
                                         **FIGURE_TOOLS,
                                      )
olympia_thermal_load_by_seasons.add_tools(thermal_load_season_tool_by_city[NOAA2010Dataset.OLYMPIA_WA])
l11 = olympia_thermal_load_by_seasons.line(x="hour", y=f"{NOAA2010Dataset.OLYMPIA_WA}_winter", source=usa_thermal_load_season_source, color=color_map["winter"], alpha=0.7, line_width=3)
l12 = olympia_thermal_load_by_seasons.square(x="hour", y=f"{NOAA2010Dataset.OLYMPIA_WA}_winter", source=usa_thermal_load_season_source, color=color_map["winter"], alpha=0.7, size=10)
l21 = olympia_thermal_load_by_seasons.line(x="hour", y=f"{NOAA2010Dataset.OLYMPIA_WA}_spring", source=usa_thermal_load_season_source, color=color_map["spring"], alpha=0.7,  line_width=3)
l22 = olympia_thermal_load_by_seasons.circle(x="hour", y=f"{NOAA2010Dataset.OLYMPIA_WA}_spring", source=usa_thermal_load_season_source, color=color_map["spring"], alpha=0.7,  size=10)
l31 = olympia_thermal_load_by_seasons.line(x="hour", y=f"{NOAA2010Dataset.OLYMPIA_WA}_summer", source=usa_thermal_load_season_source, color=color_map["summer"], alpha=0.7,  line_width=3)
l32 = olympia_thermal_load_by_seasons.diamond(x="hour", y=f"{NOAA2010Dataset.OLYMPIA_WA}_summer", source=usa_thermal_load_season_source, color=color_map["summer"], alpha=0.7,  size=10)
l41 = olympia_thermal_load_by_seasons.line(x="hour", y=f"{NOAA2010Dataset.OLYMPIA_WA}_fall", source=usa_thermal_load_season_source, color=color_map["fall"], alpha=0.7,  line_width=3)
l42 = olympia_thermal_load_by_seasons.cross(x="hour", y=f"{NOAA2010Dataset.OLYMPIA_WA}_fall", source=usa_thermal_load_season_source, color=color_map["fall"], alpha=0.7, size=10)
olympia_thermal_load_by_seasons.add_layout(Legend(items=[("Winter 2010", [l11, l12]), ("Spring 2010", [l21, l22]), ("Summer 2010", [l31, l32]), ("Fall 2010", [l41, l42]),],), "below")

fresno_thermal_load_by_seasons = figure(title="Fresno, CA",
                                        y_range=miami_thermal_load_by_seasons.y_range,
                                        x_range=miami_thermal_load_by_seasons.x_range,
                                        x_axis_label="Hour of the day",
                                        y_axis_label="Thermal load [MW]",
                                        **FIGURE_TOOLS,
                                       )
fresno_thermal_load_by_seasons.add_tools(thermal_load_season_tool_by_city[NOAA2010Dataset.FRESNO_CA])
l11 = fresno_thermal_load_by_seasons.line(x="hour", y=f"{NOAA2010Dataset.FRESNO_CA}_winter", source=usa_thermal_load_season_source, color=color_map["winter"], alpha=0.7, line_width=3)
l12 = fresno_thermal_load_by_seasons.square(x="hour", y=f"{NOAA2010Dataset.FRESNO_CA}_winter", source=usa_thermal_load_season_source, color=color_map["winter"], alpha=0.7, size=10)
l21 = fresno_thermal_load_by_seasons.line(x="hour", y=f"{NOAA2010Dataset.FRESNO_CA}_spring", source=usa_thermal_load_season_source, color=color_map["spring"], alpha=0.7,  line_width=3)
l22 = fresno_thermal_load_by_seasons.circle(x="hour", y=f"{NOAA2010Dataset.FRESNO_CA}_spring", source=usa_thermal_load_season_source, color=color_map["spring"], alpha=0.7,  size=10)
l31 = fresno_thermal_load_by_seasons.line(x="hour", y=f"{NOAA2010Dataset.FRESNO_CA}_summer", source=usa_thermal_load_season_source, color=color_map["summer"], alpha=0.7,  line_width=3)
l32 = fresno_thermal_load_by_seasons.diamond(x="hour", y=f"{NOAA2010Dataset.FRESNO_CA}_summer", source=usa_thermal_load_season_source, color=color_map["summer"], alpha=0.7,  size=10)
l41 = fresno_thermal_load_by_seasons.line(x="hour", y=f"{NOAA2010Dataset.FRESNO_CA}_fall", source=usa_thermal_load_season_source, color=color_map["fall"], alpha=0.7,  line_width=3)
l42 = fresno_thermal_load_by_seasons.cross(x="hour", y=f"{NOAA2010Dataset.FRESNO_CA}_fall", source=usa_thermal_load_season_source, color=color_map["fall"], alpha=0.7, size=10)
fresno_thermal_load_by_seasons.add_layout(Legend(items=[("Winter 2010", [l11, l12]), ("Spring 2010", [l21, l22]), ("Summer 2010", [l31, l32]), ("Fall 2010", [l41, l42]),],), "below")


In [ ]:
show(layout([[rochester_thermal_load_by_seasons, olympia_thermal_load_by_seasons ],[fresno_thermal_load_by_seasons, miami_thermal_load_by_seasons,]])) 

### Europe

In [ ]:
hours = range(0,24)
seasons = ["winter", "spring", "summer", "fall"]
cities = [InsPireDataset.LONDON_UK, InsPireDataset.MADRID_SPA, InsPireDataset.ROME_IT, InsPireDataset.STUTTGART_GER]

total_consumption_average_per_hour = dict()
for city in cities:
    
    for season in seasons:
        hours_for_city_and_season = np.empty(shape=(24,), dtype=np.float32)
        
        for hour in hours:
            hours_for_city_and_season[hour] = europe_data[(europe_data.city_key == city) & (europe_data.season == season) & (europe_data.hour == hour)]["Total_consumption"].mean() #/ 1000.00
            
        total_consumption_average_per_hour[f"{city}_{season}"] = hours_for_city_and_season
        thermal_load_season_tool_by_city[city] = HoverTool(tooltips=[("Hour", "@hour{%d}"),
                                                                     ('TL Winter', f'@{city}_winter' + '{%0.2f MW}'),
                                                                     ('TL Spring', f'@{city}_spring' + '{%0.2f MW}'),
                                                                     ('TL Summer', f'@{city}_summer' + '{%0.2f MW}'),
                                                                     ('TL Fall', f'@{city}_fall' + '{%0.2f MW}'),],
                                                           formatters={'@hour': 'printf',
                                                                       f'@{city}_winter': 'printf',
                                                                       f'@{city}_spring': 'printf',
                                                                       f'@{city}_summer': 'printf',
                                                                       f'@{city}_fall': "printf"},)

europe_thermal_load_season_source = ColumnDataSource({"hour": hours, **total_consumption_average_per_hour})


In [ ]:
london_seasonality_total_consumption = figure(title="London, United Kingdom",
                                              x_axis_label="Hour of the day",
                                              y_axis_label="Thermal load [MW]",
                                              **FIGURE_TOOLS,
                                             )
london_seasonality_total_consumption.add_tools(thermal_load_season_tool_by_city[InsPireDataset.LONDON_UK])
l11 = london_seasonality_total_consumption.line(x="hour", y=f"{InsPireDataset.LONDON_UK}_winter", source=europe_thermal_load_season_source, color=color_map["winter"], alpha=0.7, line_width=3)
l12 = london_seasonality_total_consumption.square(x="hour", y=f"{InsPireDataset.LONDON_UK}_winter", source=europe_thermal_load_season_source, color=color_map["winter"], alpha=0.7, size=10)
l21 = london_seasonality_total_consumption.line(x="hour", y=f"{InsPireDataset.LONDON_UK}_spring", source=europe_thermal_load_season_source, color=color_map["spring"], alpha=0.7,  line_width=3)
l22 = london_seasonality_total_consumption.circle(x="hour", y=f"{InsPireDataset.LONDON_UK}_spring", source=europe_thermal_load_season_source, color=color_map["spring"], alpha=0.7,  size=10)
l31 = london_seasonality_total_consumption.line(x="hour", y=f"{InsPireDataset.LONDON_UK}_summer", source=europe_thermal_load_season_source, color=color_map["summer"], alpha=0.7,  line_width=3)
l32 = london_seasonality_total_consumption.diamond(x="hour", y=f"{InsPireDataset.LONDON_UK}_summer", source=europe_thermal_load_season_source, color=color_map["summer"], alpha=0.7,  size=10)
l41 = london_seasonality_total_consumption.line(x="hour", y=f"{InsPireDataset.LONDON_UK}_fall", source=europe_thermal_load_season_source, color=color_map["fall"], alpha=0.7,  line_width=3)
l42 = london_seasonality_total_consumption.cross(x="hour", y=f"{InsPireDataset.LONDON_UK}_fall", source=europe_thermal_load_season_source, color=color_map["fall"], alpha=0.7, size=10)
london_seasonality_total_consumption.add_layout(Legend(items=[("Winter 2017", [l11, l12]), ("Spring 2017", [l21, l22]), ("Summer 2017", [l31, l32]), ("Fall 2017", [l41, l42]),],), "below")


stuttgart_seasonality_total_consumption = figure(title="Stuttgart, Germany",
                                                 y_range=london_seasonality_total_consumption.y_range,
                                                 x_range=london_seasonality_total_consumption.x_range,
                                                 x_axis_label="Hour of the day",
                                                 y_axis_label="Thermal load [MW]",
                                                 **FIGURE_TOOLS,
                                                )
stuttgart_seasonality_total_consumption.add_tools(thermal_load_season_tool_by_city[InsPireDataset.STUTTGART_GER])
l11 = stuttgart_seasonality_total_consumption.line(x="hour", y=f"{InsPireDataset.STUTTGART_GER}_winter", source=europe_thermal_load_season_source, color=color_map["winter"], alpha=0.7, line_width=3)
l12 = stuttgart_seasonality_total_consumption.square(x="hour", y=f"{InsPireDataset.STUTTGART_GER}_winter", source=europe_thermal_load_season_source, color=color_map["winter"], alpha=0.7, size=10)
l21 = stuttgart_seasonality_total_consumption.line(x="hour", y=f"{InsPireDataset.STUTTGART_GER}_spring", source=europe_thermal_load_season_source, color=color_map["spring"], alpha=0.7,  line_width=3)
l22 = stuttgart_seasonality_total_consumption.circle(x="hour", y=f"{InsPireDataset.STUTTGART_GER}_spring", source=europe_thermal_load_season_source, color=color_map["spring"], alpha=0.7,  size=10)
l31 = stuttgart_seasonality_total_consumption.line(x="hour", y=f"{InsPireDataset.STUTTGART_GER}_summer", source=europe_thermal_load_season_source, color=color_map["summer"], alpha=0.7,  line_width=3)
l32 = stuttgart_seasonality_total_consumption.diamond(x="hour", y=f"{InsPireDataset.STUTTGART_GER}_summer", source=europe_thermal_load_season_source, color=color_map["summer"], alpha=0.7,  size=10)
l41 = stuttgart_seasonality_total_consumption.line(x="hour", y=f"{InsPireDataset.STUTTGART_GER}_fall", source=europe_thermal_load_season_source, color=color_map["fall"], alpha=0.7,  line_width=3)
l42 = stuttgart_seasonality_total_consumption.cross(x="hour", y=f"{InsPireDataset.STUTTGART_GER}_fall", source=europe_thermal_load_season_source, color=color_map["fall"], alpha=0.7, size=10)
stuttgart_seasonality_total_consumption.add_layout(Legend(items=[("Winter 2017", [l11, l12]), ("Spring 2017", [l21, l22]), ("Summer 2017", [l31, l32]), ("Fall 2017", [l41, l42]),],), "below")


rome_seasonality_total_consumption = figure(title="Rome, Italy",
                                            y_range=london_seasonality_total_consumption.y_range,
                                            x_range=london_seasonality_total_consumption.x_range,
                                            x_axis_label="Hour of the day",
                                            y_axis_label="Thermal load [MW]",
                                            **FIGURE_TOOLS,
                                           )
rome_seasonality_total_consumption.add_tools(thermal_load_season_tool_by_city[InsPireDataset.ROME_IT])
l11 = rome_seasonality_total_consumption.line(x="hour", y=f"{InsPireDataset.ROME_IT}_winter", source=europe_thermal_load_season_source, color=color_map["winter"], alpha=0.7, line_width=3)
l12 = rome_seasonality_total_consumption.square(x="hour", y=f"{InsPireDataset.ROME_IT}_winter", source=europe_thermal_load_season_source, color=color_map["winter"], alpha=0.7, size=10)
l21 = rome_seasonality_total_consumption.line(x="hour", y=f"{InsPireDataset.ROME_IT}_spring", source=europe_thermal_load_season_source, color=color_map["spring"], alpha=0.7,  line_width=3)
l22 = rome_seasonality_total_consumption.circle(x="hour", y=f"{InsPireDataset.ROME_IT}_spring", source=europe_thermal_load_season_source, color=color_map["spring"], alpha=0.7,  size=10)
l31 = rome_seasonality_total_consumption.line(x="hour", y=f"{InsPireDataset.ROME_IT}_summer", source=europe_thermal_load_season_source, color=color_map["summer"], alpha=0.7,  line_width=3)
l32 = rome_seasonality_total_consumption.diamond(x="hour", y=f"{InsPireDataset.ROME_IT}_summer", source=europe_thermal_load_season_source, color=color_map["summer"], alpha=0.7,  size=10)
l41 = rome_seasonality_total_consumption.line(x="hour", y=f"{InsPireDataset.ROME_IT}_fall", source=europe_thermal_load_season_source, color=color_map["fall"], alpha=0.7,  line_width=3)
l42 = rome_seasonality_total_consumption.cross(x="hour", y=f"{InsPireDataset.ROME_IT}_fall", source=europe_thermal_load_season_source, color=color_map["fall"], alpha=0.7, size=10)
rome_seasonality_total_consumption.add_layout(Legend(items=[("Winter 2017", [l11, l12]), ("Spring 2017", [l21, l22]), ("Summer 2017", [l31, l32]), ("Fall 2017", [l41, l42]),],), "below")


madrid_seasonality_total_consumption = figure(title=" Madrid, Spain",
                                              y_range=london_seasonality_total_consumption.y_range,
                                              x_range=london_seasonality_total_consumption.x_range,
                                              x_axis_label="Hour of the day",
                                              y_axis_label="Thermal load [MW]",
                                              **FIGURE_TOOLS,
                                             )
madrid_seasonality_total_consumption.add_tools(thermal_load_season_tool_by_city[InsPireDataset.MADRID_SPA])
l11 = madrid_seasonality_total_consumption.line(x="hour", y=f"{InsPireDataset.MADRID_SPA}_winter", source=europe_thermal_load_season_source, color=color_map["winter"], alpha=0.7, line_width=3)
l12 = madrid_seasonality_total_consumption.square(x="hour", y=f"{InsPireDataset.MADRID_SPA}_winter", source=europe_thermal_load_season_source, color=color_map["winter"], alpha=0.7, size=10)
l21 = madrid_seasonality_total_consumption.line(x="hour", y=f"{InsPireDataset.MADRID_SPA}_spring", source=europe_thermal_load_season_source, color=color_map["spring"], alpha=0.7,  line_width=3)
l22 = madrid_seasonality_total_consumption.circle(x="hour", y=f"{InsPireDataset.MADRID_SPA}_spring", source=europe_thermal_load_season_source, color=color_map["spring"], alpha=0.7,  size=10)
l31 = madrid_seasonality_total_consumption.line(x="hour", y=f"{InsPireDataset.MADRID_SPA}_summer", source=europe_thermal_load_season_source, color=color_map["summer"], alpha=0.7,  line_width=3)
l32 = madrid_seasonality_total_consumption.diamond(x="hour", y=f"{InsPireDataset.MADRID_SPA}_summer", source=europe_thermal_load_season_source, color=color_map["summer"], alpha=0.7,  size=10)
l41 = madrid_seasonality_total_consumption.line(x="hour", y=f"{InsPireDataset.MADRID_SPA}_fall", source=europe_thermal_load_season_source, color=color_map["fall"], alpha=0.7,  line_width=3)
l42 = madrid_seasonality_total_consumption.cross(x="hour", y=f"{InsPireDataset.MADRID_SPA}_fall", source=europe_thermal_load_season_source, color=color_map["fall"], alpha=0.7, size=10)
madrid_seasonality_total_consumption.add_layout(Legend(items=[("Winter 2017", [l11, l12]), ("Spring 2017", [l21, l22]), ("Summer 2017", [l31, l32]), ("Fall 2017", [l41, l42]),],), "below")


In [ ]:
show(layout([[london_seasonality_total_consumption, stuttgart_seasonality_total_consumption],[rome_seasonality_total_consumption, madrid_seasonality_total_consumption]])) 

## Thermal Load in Winter and Summer

In [ ]:
thermal_load_winter_summer_tool = HoverTool(tooltips=[('Date', '@timestamp{%Y/%m/%d %H:%M:%S}'),
                                                      ("Hour of year", "@hourofyear"),
                                                      ('Thermal Power', '@Total_consumption{%0.2f MW}'),
                                                      ('Air Temperature', '@air_temp{%0.2f ºC}'),],
                                            formatters={'@timestamp': 'datetime',
                                                        '@Total_consumption': 'printf',
                                                        '@air_temp': 'printf',},)

### USA

In [ ]:
winter_data = us_data[(us_data.timestamp >= "2010-01-10") & (us_data.timestamp <= "2010-01-24")].copy()
summer_data = us_data[(us_data.timestamp >= "2010-07-15") & (us_data.timestamp <= "2010-07-29")].copy()

#winter_data["Total_consumption"] = winter_data["Total_consumption"] / 1000 # To transform consumption into MW
#summer_data["Total_consumption"] = summer_data["Total_consumption"] / 1000 # To transform consumption into MW

winter_source = ColumnDataSource(winter_data)
summer_source = ColumnDataSource(summer_data)

winter_min_hour_of_year = winter_data["hourofyear"].min()
winter_max_hour_of_year = winter_data["hourofyear"].max()

summer_min_hour_of_year = summer_data["hourofyear"].min()
summer_max_hour_of_year = summer_data["hourofyear"].max()

winter_min_y_axis_value = winter_data[["Total_consumption", "air_temp"]].min().min() - 5
winter_max_y_axis_value = winter_data[["Total_consumption", "air_temp"]].max().max() + 5

summer_min_y_axis_value = summer_data[["Total_consumption", "air_temp"]].min().min() - 5
summer_max_y_axis_value = summer_data[["Total_consumption", "air_temp"]].max().max() + 5

winter_ticks = [x for x in range(winter_min_hour_of_year, winter_max_hour_of_year + 1, 24)]
summer_ticks = [x for x in range(summer_min_hour_of_year, summer_max_hour_of_year + 1, 24)]

winter_labels_y = [winter_max_y_axis_value for _ in range(15)]
summer_labels_y = [summer_max_y_axis_value for _ in range(15)]

winter_text = ["Su", "Mo", "Tu", "We", "Th", "Fr", "Sa"] * 2 + [""]
summer_text = ["Th", "Fr", "Sa", "Su", "Mo", "Tu", "We",] * 2 + [""]

label_source = ColumnDataSource(dict(winter_x=winter_ticks, winter_y=winter_labels_y, winter_text=winter_text,
                                     summer_x=summer_ticks, summer_y=summer_labels_y, summer_text=summer_text,))

winter_rochester_2weeks_view = CDSView(source=winter_source, filters=[BooleanFilter(winter_data.city_key == NOAA2010Dataset.ROCHESTER_NY)])
summer_rochester_2weeks_view = CDSView(source=summer_source, filters=[BooleanFilter(summer_data.city_key == NOAA2010Dataset.ROCHESTER_NY)])

### Rochester, New York, USA

In [ ]:
rochester_thermal_load_winter_2_weeks = figure(title="Rochester, New York, USA - Heat Load from 10/01/2010 to 24/01/2010",
                                               y_range=(winter_min_y_axis_value, winter_max_y_axis_value),
                                               x_axis_label="Hour of the year",
                                               y_axis_label="Thermal Power [MW]",
                                               **{**FIGURE_TOOLS, **FULL_WIDTH_FIGURE_TOOLS},
                                              )
l1_winter = rochester_thermal_load_winter_2_weeks.line(x="hourofyear", y="Total_consumption", source=winter_source, view=winter_rochester_2weeks_view, color=color_map["madrid"], alpha=0.7, line_width=2)
l2_winter = rochester_thermal_load_winter_2_weeks.line(x="hourofyear", y="air_temp", source=winter_source, view=winter_rochester_2weeks_view, color=color_map["rome"], alpha=0.7, line_width=2, y_range_name="air_temp_range")
rochester_thermal_load_winter_2_weeks.add_tools(thermal_load_winter_summer_tool)
rochester_thermal_load_winter_2_weeks.add_layout(Legend(items=[("Thermal Load [MW]", [l1_winter]), ("Air Temperature [°C]", [l2_winter]),], ), "below")
rochester_thermal_load_winter_2_weeks.add_layout(LabelSet(x="winter_x", y="winter_y", text="winter_text", source=label_source, level='glyph', x_offset=25, y_offset=-25,  render_mode='canvas'))
rochester_thermal_load_winter_2_weeks.add_layout(LinearAxis(y_range_name="air_temp_range", axis_label="Temperature[°C]"), 'right')
rochester_thermal_load_winter_2_weeks.extra_y_ranges = {"air_temp_range": Range1d(start=winter_min_y_axis_value, end=winter_max_y_axis_value)}
rochester_thermal_load_winter_2_weeks.ygrid.grid_line_color = None
rochester_thermal_load_winter_2_weeks.xaxis[0].ticker = winter_ticks
rochester_thermal_load_winter_2_weeks.xgrid[0].ticker = winter_ticks


rochester_thermal_load_summer_2_weeks = figure(title="Rochester, New York, USA  - Heat Load from 15/07/2010 to 29/07/2010",
                                               y_range=(summer_min_y_axis_value, summer_max_y_axis_value),
                                               x_axis_label="Hour of the year",
                                               y_axis_label="Thermal Power [MW]",
                                               **{**FIGURE_TOOLS, **FULL_WIDTH_FIGURE_TOOLS},
                                              )
l1_summer = rochester_thermal_load_summer_2_weeks.line(x="hourofyear", y="Total_consumption", source=summer_source, view=summer_rochester_2weeks_view, color=color_map["madrid"], alpha=0.7, line_width=2)
l2_summer = rochester_thermal_load_summer_2_weeks.line(x="hourofyear", y="air_temp", source=summer_source, view=summer_rochester_2weeks_view, color=color_map["rome"], alpha=0.7, line_width=2, y_range_name="air_temp_range")
rochester_thermal_load_summer_2_weeks.add_tools(thermal_load_winter_summer_tool)
rochester_thermal_load_summer_2_weeks.add_layout(Legend(items=[("Thermal Load [MW]", [l1_summer]), ("Air Temperature [°C]", [l2_summer]),],), "below")
rochester_thermal_load_summer_2_weeks.add_layout(LabelSet(x="summer_x", y="summer_y", text="summer_text", source=label_source, level='glyph', x_offset=25, y_offset=-25,  render_mode='canvas'))
rochester_thermal_load_summer_2_weeks.add_layout(LinearAxis(y_range_name="air_temp_range", axis_label="Temperature[°C]"), 'right')
rochester_thermal_load_summer_2_weeks.extra_y_ranges = {"air_temp_range": Range1d(start=summer_min_y_axis_value, end=summer_max_y_axis_value)}
rochester_thermal_load_summer_2_weeks.ygrid.grid_line_color = None
rochester_thermal_load_summer_2_weeks.xaxis[0].ticker = summer_ticks
rochester_thermal_load_summer_2_weeks.xgrid[0].ticker = summer_ticks



In [ ]:
show(layout([[rochester_thermal_load_winter_2_weeks],[rochester_thermal_load_summer_2_weeks],])) 

### Fresno, California, USA

In [ ]:
winter_fresno_2weeks_view = CDSView(source=winter_source, filters=[BooleanFilter(winter_data.city_key == NOAA2010Dataset.FRESNO_CA)])
summer_fresno_2weeks_view = CDSView(source=summer_source, filters=[BooleanFilter(summer_data.city_key == NOAA2010Dataset.FRESNO_CA)])

fresno_thermal_load_winter_2_weeks = figure(title="Fresno, California, USA - Heat Load from 10/01/2010 to 24/01/2010",
                                            y_range=(winter_min_y_axis_value, winter_max_y_axis_value),
                                            x_axis_label="Hour of the year",
                                            y_axis_label="Thermal Power [MW]",
                                            **{**FIGURE_TOOLS, **FULL_WIDTH_FIGURE_TOOLS},
                                           )
fresno_thermal_load_winter_2_weeks.add_tools(thermal_load_winter_summer_tool)
l1_winter = fresno_thermal_load_winter_2_weeks.line(x="hourofyear", y="Total_consumption", source=winter_source, view=winter_fresno_2weeks_view, color=color_map["madrid"], alpha=0.7, line_width=2)
l2_winter = fresno_thermal_load_winter_2_weeks.line(x="hourofyear", y="air_temp", source=winter_source, view=winter_fresno_2weeks_view, color=color_map["rome"], alpha=0.7, line_width=2, y_range_name="air_temp_range")
fresno_thermal_load_winter_2_weeks.add_layout(Legend(items=[("Thermal Load [MW]", [l1_winter]), ("Air Temperature [°C]", [l2_winter]),],), "below")
fresno_thermal_load_winter_2_weeks.add_layout(LabelSet(x="winter_x", y="winter_y", text="winter_text", source=label_source, level='glyph', x_offset=25, y_offset=-25,  render_mode='canvas'))
fresno_thermal_load_winter_2_weeks.add_layout(LinearAxis(y_range_name="air_temp_range", axis_label="Temperature[°C]"), 'right')
fresno_thermal_load_winter_2_weeks.extra_y_ranges = {"air_temp_range": Range1d(start=winter_min_y_axis_value, end=winter_max_y_axis_value)}
fresno_thermal_load_winter_2_weeks.ygrid.grid_line_color = None
fresno_thermal_load_winter_2_weeks.xaxis[0].ticker = winter_ticks
fresno_thermal_load_winter_2_weeks.xgrid[0].ticker = winter_ticks



fresno_thermal_load_summer_2_weeks = figure(title="Fresno, California, USA- Heat Load from 15/07/2010 to 29/07/2010",
                                            y_range=(summer_min_y_axis_value, summer_max_y_axis_value),
                                            x_axis_label="Hour of the year",
                                            y_axis_label="Thermal Power [MW]",
                                            **{**FIGURE_TOOLS, **FULL_WIDTH_FIGURE_TOOLS},
                                           )
fresno_thermal_load_summer_2_weeks.add_tools(thermal_load_winter_summer_tool)
l1_summer = fresno_thermal_load_summer_2_weeks.line(x="hourofyear", y="Total_consumption", source=summer_source, view=summer_fresno_2weeks_view, color=color_map["madrid"], alpha=0.7, line_width=2)
l2_summer = fresno_thermal_load_summer_2_weeks.line(x="hourofyear", y="air_temp", source=summer_source, view=summer_fresno_2weeks_view, color=color_map["rome"], alpha=0.7, line_width=2, y_range_name="air_temp_range")
fresno_thermal_load_summer_2_weeks.add_layout(Legend(items=[("Thermal Load [MW]", [l1_summer]), ("Air Temperature [°C]", [l2_summer]),],), "below")
fresno_thermal_load_summer_2_weeks.add_layout(LabelSet(x="summer_x", y="summer_y", text="summer_text", source=label_source, level='glyph', x_offset=25, y_offset=-25,  render_mode='canvas'))
fresno_thermal_load_summer_2_weeks.add_layout(LinearAxis(y_range_name="air_temp_range", axis_label="Temperature[°C]"), 'right')
fresno_thermal_load_summer_2_weeks.extra_y_ranges = {"air_temp_range": Range1d(start=summer_min_y_axis_value, end=summer_max_y_axis_value)}
fresno_thermal_load_summer_2_weeks.ygrid.grid_line_color = None
fresno_thermal_load_summer_2_weeks.xaxis[0].ticker = summer_ticks
fresno_thermal_load_summer_2_weeks.xgrid[0].ticker = summer_ticks


In [ ]:
show(layout([[fresno_thermal_load_winter_2_weeks],[fresno_thermal_load_summer_2_weeks],])) 

### Europe

In [ ]:
winter_data = europe_data[(europe_data.timestamp >= "2017-01-10") & (europe_data.timestamp <= "2017-01-24")].copy()
summer_data = europe_data[(europe_data.timestamp >= "2017-07-15") & (europe_data.timestamp <= "2017-07-29")].copy()

#winter_data["Total_consumption"] = winter_data["Total_consumption"] / 1000 # To transform consumption into MW
#summer_data["Total_consumption"] = summer_data["Total_consumption"] / 1000 # To transform consumption into MW

winter_source = ColumnDataSource(winter_data)
summer_source = ColumnDataSource(summer_data)

winter_min_hour_of_year = winter_data["hourofyear"].min()
winter_max_hour_of_year = winter_data["hourofyear"].max()

summer_min_hour_of_year = summer_data["hourofyear"].min()
summer_max_hour_of_year = summer_data["hourofyear"].max()

winter_min_y_axis_value = winter_data[["Total_consumption", "air_temp"]].min().min() - 5
winter_max_y_axis_value = winter_data[["Total_consumption", "air_temp"]].max().max() + 5

summer_min_y_axis_value = summer_data[["Total_consumption", "air_temp"]].min().min() - 5
summer_max_y_axis_value = summer_data[["Total_consumption", "air_temp"]].max().max() + 5

winter_ticks = [x for x in range(winter_min_hour_of_year, winter_max_hour_of_year + 1, 24)]
summer_ticks = [x for x in range(summer_min_hour_of_year, summer_max_hour_of_year + 1, 24)]

winter_labels_y = [winter_max_y_axis_value for _ in range(15)]
summer_labels_y = [summer_max_y_axis_value for _ in range(15)]

winter_text = ["Tu", "We", "Th", "Fr", "Sa", "Su", "Mo"] * 2 + [""]
summer_text = ["Sa", "Su", "Mo", "Tu", "We", "Th", "Fr"] * 2 + [""]

label_source = ColumnDataSource(dict(winter_x=winter_ticks, winter_y=winter_labels_y, winter_text=winter_text,
                                     summer_x=summer_ticks, summer_y=summer_labels_y, summer_text=summer_text,))



### London, United Kingdom

In [ ]:
winter_london_2weeks_view = CDSView(source=winter_source, filters=[BooleanFilter(winter_data.city_key == InsPireDataset.LONDON_UK)])
summer_london_2weeks_view = CDSView(source=summer_source, filters=[BooleanFilter(summer_data.city_key == InsPireDataset.LONDON_UK)])

london_thermal_load_winter_2_weeks = figure(title="London, United Kingdom - Heat Load from 10/01/2017 to 24/01/2017",
                                            y_range=(winter_min_y_axis_value, winter_max_y_axis_value),
                                            x_axis_label="Hour of the year",
                                            y_axis_label="Thermal Power [MW]",
                                            **{**FIGURE_TOOLS, **FULL_WIDTH_FIGURE_TOOLS},
                                           )
london_thermal_load_winter_2_weeks.add_tools(thermal_load_winter_summer_tool)
l1_winter = london_thermal_load_winter_2_weeks.line(x="hourofyear", y="Total_consumption", source=winter_source, view=winter_london_2weeks_view, color=color_map["madrid"], alpha=0.7, line_width=2)
l2_winter = london_thermal_load_winter_2_weeks.line(x="hourofyear", y="air_temp", source=winter_source, view=winter_london_2weeks_view, color=color_map["rome"], alpha=0.7, line_width=2, y_range_name="air_temp_range")
london_thermal_load_winter_2_weeks.add_layout(Legend(items=[("Thermal Load [MW]", [l1_winter]), ("Air Temperature [°C]", [l2_winter]),],), "below")
london_thermal_load_winter_2_weeks.add_layout(LabelSet(x="winter_x", y="winter_y", text="winter_text", source=label_source, level='glyph', x_offset=25, y_offset=-25,  render_mode='canvas'))
london_thermal_load_winter_2_weeks.add_layout(LinearAxis(y_range_name="air_temp_range", axis_label="Temperature[°C]"), 'right')
london_thermal_load_winter_2_weeks.extra_y_ranges = {"air_temp_range": Range1d(start=winter_min_y_axis_value, end=winter_max_y_axis_value)}
london_thermal_load_winter_2_weeks.ygrid.grid_line_color = None
london_thermal_load_winter_2_weeks.xaxis[0].ticker = winter_ticks
london_thermal_load_winter_2_weeks.xgrid[0].ticker = winter_ticks



london_thermal_load_summer_2_weeks = figure(title="London, United Kingdom - Heat Load from 15/07/2017 to 29/07/2017",
                                            y_range=(summer_min_y_axis_value, summer_max_y_axis_value),
                                            x_axis_label="Hour of the year",
                                            y_axis_label="Thermal Power [MW]",
                                            **{**FIGURE_TOOLS, **FULL_WIDTH_FIGURE_TOOLS},
                                           )
london_thermal_load_summer_2_weeks.add_tools(thermal_load_winter_summer_tool)
l1_summer = london_thermal_load_summer_2_weeks.line(x="hourofyear", y="Total_consumption", source=summer_source, view=summer_london_2weeks_view, color=color_map["madrid"], alpha=0.7, line_width=2)
l2_summer = london_thermal_load_summer_2_weeks.line(x="hourofyear", y="air_temp", source=summer_source, view=summer_london_2weeks_view, color=color_map["rome"], alpha=0.7, line_width=2, y_range_name="air_temp_range")
london_thermal_load_summer_2_weeks.add_layout(Legend(items=[("Thermal Load [MW]", [l1_summer]), ("Air Temperature [°C]", [l2_summer]),],), "below")
london_thermal_load_summer_2_weeks.add_layout(LabelSet(x="summer_x", y="summer_y", text="summer_text", source=label_source, level='glyph', x_offset=25, y_offset=-25,  render_mode='canvas'))
london_thermal_load_summer_2_weeks.add_layout(LinearAxis(y_range_name="air_temp_range", axis_label="Temperature[°C]"), 'right')
london_thermal_load_summer_2_weeks.extra_y_ranges = {"air_temp_range": Range1d(start=summer_min_y_axis_value, end=summer_max_y_axis_value)}
london_thermal_load_summer_2_weeks.ygrid.grid_line_color = None
london_thermal_load_summer_2_weeks.xaxis[0].ticker = summer_ticks
london_thermal_load_summer_2_weeks.xgrid[0].ticker = summer_ticks




In [ ]:
show(layout([[london_thermal_load_winter_2_weeks],[london_thermal_load_summer_2_weeks],])) 

### Rome, Italy

In [ ]:
winter_rome_2weeks_view = CDSView(source=winter_source, filters=[BooleanFilter(winter_data.city_key == InsPireDataset.ROME_IT)])
summer_rome_2weeks_view = CDSView(source=summer_source, filters=[BooleanFilter(summer_data.city_key == InsPireDataset.ROME_IT)])

rome_thermal_load_winter_2_weeks = figure(title="Rome, Italy - Heat Load from 10/01/2017 to 24/01/2017",
                                          y_range=(winter_min_y_axis_value, winter_max_y_axis_value),
                                          x_axis_label="Hour of the year",
                                          y_axis_label="Thermal Power [MW]",
                                          **{**FIGURE_TOOLS, **FULL_WIDTH_FIGURE_TOOLS},
                                         )
rome_thermal_load_winter_2_weeks.add_tools(thermal_load_winter_summer_tool)
l1_winter = rome_thermal_load_winter_2_weeks.line(x="hourofyear", y="Total_consumption", source=winter_source, view=winter_rome_2weeks_view, color=color_map["madrid"], alpha=0.7, line_width=2)
l2_winter = rome_thermal_load_winter_2_weeks.line(x="hourofyear", y="air_temp", source=winter_source, view=winter_rome_2weeks_view, color=color_map["rome"], alpha=0.7, line_width=2, y_range_name="air_temp_range")
rome_thermal_load_winter_2_weeks.add_layout(Legend(items=[("Thermal Load [MW]", [l1_winter]), ("Air Temperature [°C]", [l2_winter]),],), "below")
rome_thermal_load_winter_2_weeks.add_layout(LabelSet(x="winter_x", y="winter_y", text="winter_text", source=label_source, level='glyph', x_offset=25, y_offset=-25,  render_mode='canvas'))
rome_thermal_load_winter_2_weeks.add_layout(LinearAxis(y_range_name="air_temp_range", axis_label="Temperature[°C]"), 'right')
rome_thermal_load_winter_2_weeks.extra_y_ranges = {"air_temp_range": Range1d(start=winter_min_y_axis_value, end=winter_max_y_axis_value)}
rome_thermal_load_winter_2_weeks.ygrid.grid_line_color = None
rome_thermal_load_winter_2_weeks.xaxis[0].ticker = winter_ticks
rome_thermal_load_winter_2_weeks.xgrid[0].ticker = winter_ticks


rome_thermal_load_summer_2_weeks = figure(title="Rome, Italy - Heat Load from 15/07/2017 to 29/07/2017",
                                          y_range=(summer_min_y_axis_value, summer_max_y_axis_value),
                                          x_axis_label="Hour of the year",
                                          y_axis_label="Thermal Power [MW]",
                                          **{**FIGURE_TOOLS, **FULL_WIDTH_FIGURE_TOOLS},
                                         )
rome_thermal_load_summer_2_weeks.add_tools(thermal_load_winter_summer_tool)
l1_summer = rome_thermal_load_summer_2_weeks.line(x="hourofyear", y="Total_consumption", source=summer_source, view=summer_rome_2weeks_view, color=color_map["madrid"], alpha=0.7, line_width=2)
l2_summer = rome_thermal_load_summer_2_weeks.line(x="hourofyear", y="air_temp", source=summer_source, view=summer_rome_2weeks_view, color=color_map["rome"], alpha=0.7, line_width=2, y_range_name="air_temp_range")
rome_thermal_load_summer_2_weeks.add_layout(Legend(items=[("Thermal Load [MW]", [l1_summer]), ("Air Temperature [°C]", [l2_summer]),],), "below")
rome_thermal_load_summer_2_weeks.add_layout(LabelSet(x="summer_x", y="summer_y", text="summer_text", source=label_source, level='glyph', x_offset=25, y_offset=-25,  render_mode='canvas'))
rome_thermal_load_summer_2_weeks.add_layout(LinearAxis(y_range_name="air_temp_range", axis_label="Temperature[°C]"), 'right')
rome_thermal_load_summer_2_weeks.extra_y_ranges = {"air_temp_range": Range1d(start=summer_min_y_axis_value, end=summer_max_y_axis_value)}
rome_thermal_load_summer_2_weeks.ygrid.grid_line_color = None
rome_thermal_load_summer_2_weeks.xaxis[0].ticker = summer_ticks
rome_thermal_load_summer_2_weeks.xgrid[0].ticker = summer_ticks


In [ ]:
show(layout([[rome_thermal_load_winter_2_weeks],[rome_thermal_load_summer_2_weeks],])) 

## Heat pumps electricity consumption by city (USA & Europe)

In [ ]:
london_data = europe_data[(europe_data["city_key"] == InsPireDataset.LONDON_UK)]["E_el"]#/1000
stuttgart_data = europe_data[(europe_data["city_key"] == InsPireDataset.STUTTGART_GER)]["E_el"]#/1000
rome_data = europe_data[(europe_data["city_key"] == InsPireDataset.ROME_IT)]["E_el"]#/1000
madrid_data = europe_data[(europe_data["city_key"] == InsPireDataset.MADRID_SPA)]["E_el"]#/1000

miami_data= us_data[(us_data["city_key"] == NOAA2010Dataset.MIAMI_FL)]["E_el"]#/1000
fresno_data= us_data[(us_data["city_key"] == NOAA2010Dataset.FRESNO_CA)]["E_el"]#/1000
olympia_data= us_data[(us_data["city_key"] == NOAA2010Dataset.OLYMPIA_WA)]["E_el"]#/1000
rochester_data= us_data[(us_data["city_key"] == NOAA2010Dataset.ROCHESTER_NY)]["E_el"]#/1000

x_europe = ["London, UK", "Madrid, SPA", "Rome, IT", "Stuttgart, GER",]
x_usa = ["Miami, FL", "Fresno, CA", "Olympia, WA", "Rochester, NY"]
colors_europe = [color_map["london"], color_map["madrid"], color_map["rome"], color_map["stuttgart"], ]
colors_usa = [color_map["miami"], color_map["fresno"], color_map["olympia"], color_map["rochester"], ]
top_europe = [london_data.sum(), madrid_data.sum(), rome_data.sum(), stuttgart_data.sum(),] 
top_usa = [miami_data.sum(), fresno_data.sum(), olympia_data.sum(), rochester_data.sum(),]

electricity_source = ColumnDataSource({"x": x_europe + x_usa, "colors": colors_europe + colors_usa, "top": top_europe + top_usa})
europe_electricity_view = CDSView(source=electricity_source, filters=[BooleanFilter([True, True, True, True, False, False, False, False])])
usa_electricity_view = CDSView(source=electricity_source, filters=[BooleanFilter([False, False, False, False, True, True, True, True])])

electricity_consumption_hover_tool = HoverTool(tooltips=[('City', '@x'),
                                                         ('Electricity', '@top{%0.2f MW}'),],
                                               formatters={'@x': 'printf',
                                                           '@top': 'printf',})

In [ ]:
usa_electricity = figure(x_range= FactorRange(*x_usa),
                         title="Annual electric consumption by cities in USA",
                         y_axis_label="Electric power [MW]",
                         **{**FIGURE_TOOLS, **FULL_WIDTH_FIGURE_TOOLS})
usa_electricity.add_tools(electricity_consumption_hover_tool)
usa_electricity.vbar(x="x", top="top", color="colors", width=0.9, alpha=0.8, source=electricity_source, view=usa_electricity_view)


europe_electricity = figure(x_range= FactorRange(*x_europe),
                            title="Annual electric consumption by cities in Europe",
                            y_axis_label="Electric power [MW]",
                            **{**FIGURE_TOOLS, **FULL_WIDTH_FIGURE_TOOLS})
europe_electricity.add_tools(electricity_consumption_hover_tool)
europe_electricity.vbar(x="x", top="top", color="colors", width=0.9, alpha=0.8, source=electricity_source, view=europe_electricity_view)




In [ ]:
show(layout([[usa_electricity],[europe_electricity],]))

## Heat losses by country (USA & Europe)

In [ ]:
london_data = europe_data[(europe_data["city_key"] == InsPireDataset.LONDON_UK)]["E_loss_tot"]/1000
stuttgart_data = europe_data[(europe_data["city_key"] == InsPireDataset.STUTTGART_GER)]["E_loss_tot"]/1000
rome_data = europe_data[(europe_data["city_key"] == InsPireDataset.ROME_IT)]["E_loss_tot"]/1000
madrid_data = europe_data[(europe_data["city_key"] == InsPireDataset.MADRID_SPA)]["E_loss_tot"]/1000

miami_data= us_data[(us_data["city_key"] == NOAA2010Dataset.MIAMI_FL)]["E_loss_tot"]/1000
fresno_data= us_data[(us_data["city_key"] == NOAA2010Dataset.FRESNO_CA)]["E_loss_tot"]/1000
olympia_data= us_data[(us_data["city_key"] == NOAA2010Dataset.OLYMPIA_WA)]["E_loss_tot"]/1000
rochester_data= us_data[(us_data["city_key"] == NOAA2010Dataset.ROCHESTER_NY)]["E_loss_tot"]/1000

x_europe = ["London, UK", "Madrid, SPA", "Rome, IT", "Stuttgart, GER",]
x_usa = ["Miami, FL", "Fresno, CA", "Olympia, WA", "Rochester, NY"]
colors_europe = [color_map["london"], color_map["madrid"], color_map["rome"], color_map["stuttgart"], ]
colors_usa = [color_map["miami"], color_map["fresno"], color_map["olympia"], color_map["rochester"], ]
top_europe = [london_data.sum(), madrid_data.sum(), rome_data.sum(), stuttgart_data.sum(),] 
top_usa = [miami_data.sum(), fresno_data.sum(), olympia_data.sum(), rochester_data.sum(),]

heat_loss_source = ColumnDataSource({"x": x_europe + x_usa, "colors": colors_europe + colors_usa, "top": top_europe + top_usa})
europe_heat_loss_view = CDSView(source=heat_loss_source, filters=[BooleanFilter([True, True, True, True, False, False, False, False])])
usa_heat_loss_view = CDSView(source=heat_loss_source, filters=[BooleanFilter([False, False, False, False, True, True, True, True])])

heat_loss_consumption_hover_tool = HoverTool(tooltips=[('City', '@x'),
                                                       ('Heat Loss', '@top{%0.2f GWh}'),],
                                             formatters={'@x': 'printf',
                                                         '@top': 'printf',})

In [ ]:
usa_heat_loss = figure(x_range=FactorRange(*x_usa),
                       title="Total heat loss by cities in USA",
                       y_axis_label="Heat Loss [GWh]",
                       **{**FIGURE_TOOLS, **FULL_WIDTH_FIGURE_TOOLS})
usa_heat_loss.add_tools(heat_loss_consumption_hover_tool)
usa_heat_loss.vbar(x="x", top="top", color="colors", width=0.9, alpha=0.8, source=heat_loss_source, view=usa_heat_loss_view)


europe_heat_loss = figure(x_range=FactorRange(*x_europe),
                          title="Total heat loss by cities in Europe",
                          y_axis_label="Heat Loss [GWh]",
                          **{**FIGURE_TOOLS, **FULL_WIDTH_FIGURE_TOOLS})
europe_heat_loss.add_tools(heat_loss_consumption_hover_tool)
europe_heat_loss.vbar(x="x", top="top", color="colors", width=0.9, alpha=0.8, source=heat_loss_source, view=europe_heat_loss_view)


In [ ]:
show(layout([[usa_heat_loss],[europe_heat_loss],]))

## Weekly Electrical Power use by season

### USA

In [ ]:
dayofweek = range(0,7)
seasons = ["winter", "spring", "summer", "fall"]
cities = [NOAA2010Dataset.MIAMI_FL, NOAA2010Dataset.FRESNO_CA, NOAA2010Dataset.OLYMPIA_WA, NOAA2010Dataset.ROCHESTER_NY]

total_consumption_average_per_day = dict()
electrical_power_use_season_tool_by_city = dict()
for city in cities:
    for season in seasons:
        days_for_city_and_season = np.empty(shape=(7,), dtype=np.float32)
        
        for day in dayofweek:
            days_for_city_and_season[day] = us_data[(us_data.city_key == city) & (us_data.season == season) & (us_data.dayofweek == day)]["E_el"].mean()#/1000
    
        total_consumption_average_per_day[f"{city}_{season}"] = days_for_city_and_season
        electrical_power_use_season_tool_by_city[city] = HoverTool(tooltips=[("Day", "@dayofweek{%d}"),
                                                                             ('Electricity Winter', f'@{city}_winter' + '{%0.2f MW}'),
                                                                             ('Electricity Spring', f'@{city}_spring' + '{%0.2f MW}'),
                                                                             ('Electricity Summer', f'@{city}_summer' + '{%0.2f MW}'),
                                                                             ('Electricity Fall', f'@{city}_fall' + '{%0.2f MW}'),],
                                                                   formatters={'@hour': 'printf',
                                                                               f'@{city}_winter': 'printf',
                                                                               f'@{city}_spring': 'printf',
                                                                               f'@{city}_summer': 'printf',
                                                                               f'@{city}_fall': "printf"},)
        
x=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

usa_electricity_season_source = ColumnDataSource({"dayofweek": x, **total_consumption_average_per_day})

In [ ]:
rochester_seasonality_total_consumption = figure(title="Rochester, NY",
                                                 x_range=FactorRange(*x),
                                                 x_axis_label="Day of the week",
                                                 y_axis_label="Seasonal electric power [MW]",
                                                 **FIGURE_TOOLS,
                                                )
rochester_seasonality_total_consumption.add_tools(electrical_power_use_season_tool_by_city[NOAA2010Dataset.ROCHESTER_NY])
l11 = rochester_seasonality_total_consumption.line(x="dayofweek", y=f"{NOAA2010Dataset.ROCHESTER_NY}_winter", source=usa_electricity_season_source, color=color_map["winter"], alpha=0.7, line_width=3)
l12 = rochester_seasonality_total_consumption.square(x="dayofweek", y=f"{NOAA2010Dataset.ROCHESTER_NY}_winter", source=usa_electricity_season_source, color=color_map["winter"], alpha=0.7, size=10)
l21 = rochester_seasonality_total_consumption.line(x="dayofweek", y=f"{NOAA2010Dataset.ROCHESTER_NY}_spring", source=usa_electricity_season_source, color=color_map["spring"], alpha=0.7,  line_width=3)
l22 = rochester_seasonality_total_consumption.circle(x="dayofweek", y=f"{NOAA2010Dataset.ROCHESTER_NY}_spring", source=usa_electricity_season_source, color=color_map["spring"], alpha=0.7,  size=10)
l31 = rochester_seasonality_total_consumption.line(x="dayofweek", y=f"{NOAA2010Dataset.ROCHESTER_NY}_summer", source=usa_electricity_season_source, color=color_map["summer"], alpha=0.7,  line_width=3)
l32 = rochester_seasonality_total_consumption.diamond(x="dayofweek", y=f"{NOAA2010Dataset.ROCHESTER_NY}_summer", source=usa_electricity_season_source, color=color_map["summer"], alpha=0.7,  size=10)
l41 = rochester_seasonality_total_consumption.line(x="dayofweek", y=f"{NOAA2010Dataset.ROCHESTER_NY}_fall", source=usa_electricity_season_source, color=color_map["fall"], alpha=0.7,  line_width=3)
l42 = rochester_seasonality_total_consumption.cross(x="dayofweek", y=f"{NOAA2010Dataset.ROCHESTER_NY}_fall", source=usa_electricity_season_source, color=color_map["fall"], alpha=0.7, size=10)
rochester_seasonality_total_consumption.add_layout(Legend(items=[("Winter 2010", [l11, l12]), ("Spring 2010", [l21, l22]), ("Summer 2010", [l31, l32]), ("Fall 2010", [l41, l42]),],), "below")


olympia_seasonality_total_consumption = figure(title="Olympia, WA",
                                               x_axis_label="Day of the week",
                                               y_axis_label="Seasonal electric power [MW]",
                                               y_range=rochester_seasonality_total_consumption.y_range,
                                               x_range=rochester_seasonality_total_consumption.x_range,
                                               **FIGURE_TOOLS,
                                              )
olympia_seasonality_total_consumption.add_tools(electrical_power_use_season_tool_by_city[NOAA2010Dataset.OLYMPIA_WA])
l11 = olympia_seasonality_total_consumption.line(x="dayofweek", y=f"{NOAA2010Dataset.OLYMPIA_WA}_winter", source=usa_electricity_season_source, color=color_map["winter"], alpha=0.7, line_width=3)
l12 = olympia_seasonality_total_consumption.square(x="dayofweek", y=f"{NOAA2010Dataset.OLYMPIA_WA}_winter", source=usa_electricity_season_source, color=color_map["winter"], alpha=0.7, size=10)
l21 = olympia_seasonality_total_consumption.line(x="dayofweek", y=f"{NOAA2010Dataset.OLYMPIA_WA}_spring", source=usa_electricity_season_source, color=color_map["spring"], alpha=0.7,  line_width=3)
l22 = olympia_seasonality_total_consumption.circle(x="dayofweek", y=f"{NOAA2010Dataset.OLYMPIA_WA}_spring", source=usa_electricity_season_source, color=color_map["spring"], alpha=0.7,  size=10)
l31 = olympia_seasonality_total_consumption.line(x="dayofweek", y=f"{NOAA2010Dataset.OLYMPIA_WA}_summer", source=usa_electricity_season_source, color=color_map["summer"], alpha=0.7,  line_width=3)
l32 = olympia_seasonality_total_consumption.diamond(x="dayofweek", y=f"{NOAA2010Dataset.OLYMPIA_WA}_summer", source=usa_electricity_season_source, color=color_map["summer"], alpha=0.7,  size=10)
l41 = olympia_seasonality_total_consumption.line(x="dayofweek", y=f"{NOAA2010Dataset.OLYMPIA_WA}_fall", source=usa_electricity_season_source, color=color_map["fall"], alpha=0.7,  line_width=3)
l42 = olympia_seasonality_total_consumption.cross(x="dayofweek", y=f"{NOAA2010Dataset.OLYMPIA_WA}_fall", source=usa_electricity_season_source, color=color_map["fall"], alpha=0.7, size=10)
olympia_seasonality_total_consumption.add_layout(Legend(items=[("Winter 2010", [l11, l12]), ("Spring 2010", [l21, l22]), ("Summer 2010", [l31, l32]), ("Fall 2010", [l41, l42]),],), "below")


fresno_seasonality_total_consumption = figure(title="Fresno, CA",
                                               x_axis_label="Day of the week",
                                               y_axis_label="Seasonal electric power [MW]",
                                               y_range=rochester_seasonality_total_consumption.y_range,
                                               x_range=rochester_seasonality_total_consumption.x_range,
                                               **FIGURE_TOOLS,
                                              )
fresno_seasonality_total_consumption.add_tools(electrical_power_use_season_tool_by_city[NOAA2010Dataset.FRESNO_CA])
l11 = fresno_seasonality_total_consumption.line(x="dayofweek", y=f"{NOAA2010Dataset.FRESNO_CA}_winter", source=usa_electricity_season_source, color=color_map["winter"], alpha=0.7, line_width=3)
l12 = fresno_seasonality_total_consumption.square(x="dayofweek", y=f"{NOAA2010Dataset.FRESNO_CA}_winter", source=usa_electricity_season_source, color=color_map["winter"], alpha=0.7, size=10)
l21 = fresno_seasonality_total_consumption.line(x="dayofweek", y=f"{NOAA2010Dataset.FRESNO_CA}_spring", source=usa_electricity_season_source, color=color_map["spring"], alpha=0.7,  line_width=3)
l22 = fresno_seasonality_total_consumption.circle(x="dayofweek", y=f"{NOAA2010Dataset.FRESNO_CA}_spring", source=usa_electricity_season_source, color=color_map["spring"], alpha=0.7,  size=10)
l31 = fresno_seasonality_total_consumption.line(x="dayofweek", y=f"{NOAA2010Dataset.FRESNO_CA}_summer", source=usa_electricity_season_source, color=color_map["summer"], alpha=0.7,  line_width=3)
l32 = fresno_seasonality_total_consumption.diamond(x="dayofweek", y=f"{NOAA2010Dataset.FRESNO_CA}_summer", source=usa_electricity_season_source, color=color_map["summer"], alpha=0.7,  size=10)
l41 = fresno_seasonality_total_consumption.line(x="dayofweek", y=f"{NOAA2010Dataset.FRESNO_CA}_fall", source=usa_electricity_season_source, color=color_map["fall"], alpha=0.7,  line_width=3)
l42 = fresno_seasonality_total_consumption.cross(x="dayofweek", y=f"{NOAA2010Dataset.FRESNO_CA}_fall", source=usa_electricity_season_source, color=color_map["fall"], alpha=0.7, size=10)
fresno_seasonality_total_consumption.add_layout(Legend(items=[("Winter 2010", [l11, l12]), ("Spring 2010", [l21, l22]), ("Summer 2010", [l31, l32]), ("Fall 2010", [l41, l42]),],), "below")


miami_seasonality_total_consumption = figure(title="Miami, FL",
                                               x_axis_label="Day of the week",
                                               y_axis_label="Seasonal electric power [MW]",
                                               y_range=rochester_seasonality_total_consumption.y_range,
                                               x_range=rochester_seasonality_total_consumption.x_range,
                                               **FIGURE_TOOLS,
                                              )
miami_seasonality_total_consumption.add_tools(electrical_power_use_season_tool_by_city[NOAA2010Dataset.MIAMI_FL])
l11 = miami_seasonality_total_consumption.line(x="dayofweek", y=f"{NOAA2010Dataset.MIAMI_FL}_winter", source=usa_electricity_season_source, color=color_map["winter"], alpha=0.7, line_width=3)
l12 = miami_seasonality_total_consumption.square(x="dayofweek", y=f"{NOAA2010Dataset.MIAMI_FL}_winter", source=usa_electricity_season_source, color=color_map["winter"], alpha=0.7, size=10)
l21 = miami_seasonality_total_consumption.line(x="dayofweek", y=f"{NOAA2010Dataset.MIAMI_FL}_spring", source=usa_electricity_season_source, color=color_map["spring"], alpha=0.7,  line_width=3)
l22 = miami_seasonality_total_consumption.circle(x="dayofweek", y=f"{NOAA2010Dataset.MIAMI_FL}_spring", source=usa_electricity_season_source, color=color_map["spring"], alpha=0.7,  size=10)
l31 = miami_seasonality_total_consumption.line(x="dayofweek", y=f"{NOAA2010Dataset.MIAMI_FL}_summer", source=usa_electricity_season_source, color=color_map["summer"], alpha=0.7,  line_width=3)
l32 = miami_seasonality_total_consumption.diamond(x="dayofweek", y=f"{NOAA2010Dataset.MIAMI_FL}_summer", source=usa_electricity_season_source, color=color_map["summer"], alpha=0.7,  size=10)
l41 = miami_seasonality_total_consumption.line(x="dayofweek", y=f"{NOAA2010Dataset.MIAMI_FL}_fall", source=usa_electricity_season_source, color=color_map["fall"], alpha=0.7,  line_width=3)
l42 = miami_seasonality_total_consumption.cross(x="dayofweek", y=f"{NOAA2010Dataset.MIAMI_FL}_fall", source=usa_electricity_season_source, color=color_map["fall"], alpha=0.7, size=10)
miami_seasonality_total_consumption.add_layout(Legend(items=[("Winter 2010", [l11, l12]), ("Spring 2010", [l21, l22]), ("Summer 2010", [l31, l32]), ("Fall 2010", [l41, l42]),],), "below")



In [ ]:
show(layout([[rochester_seasonality_total_consumption, olympia_seasonality_total_consumption],[fresno_seasonality_total_consumption, miami_seasonality_total_consumption]])) 

### Europe

In [ ]:
dayofweek = range(0,7)
seasons = ["winter", "spring", "summer", "fall"]
cities = [InsPireDataset.LONDON_UK, InsPireDataset.MADRID_SPA, InsPireDataset.ROME_IT, InsPireDataset.STUTTGART_GER]

total_consumption_average_per_day = dict()
electrical_power_use_season_tool_by_city = dict()
for city in cities:
    for season in seasons:
        days_for_city_and_season = np.empty(shape=(7,), dtype=np.float32)
        
        for day in dayofweek:
            days_for_city_and_season[day] = europe_data[(europe_data.city_key == city) & (europe_data.season == season) & (europe_data.dayofweek == day)]["E_el"].mean()#/1000
    
        total_consumption_average_per_day[f"{city}_{season}"] = days_for_city_and_season
        electrical_power_use_season_tool_by_city[city] = HoverTool(tooltips=[("Day", "@dayofweek{%d}"),
                                                                             ('Electricity Winter', f'@{city}_winter' + '{%0.2f MW}'),
                                                                             ('Electricity Spring', f'@{city}_spring' + '{%0.2f MW}'),
                                                                             ('Electricity Summer', f'@{city}_summer' + '{%0.2f MW}'),
                                                                             ('Electricity Fall', f'@{city}_fall' + '{%0.2f MW}'),],
                                                                   formatters={'@hour': 'printf',
                                                                               f'@{city}_winter': 'printf',
                                                                               f'@{city}_spring': 'printf',
                                                                               f'@{city}_summer': 'printf',
                                                                               f'@{city}_fall': "printf"},)
        
x=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

europe_electricity_season_source = ColumnDataSource({"dayofweek": x, **total_consumption_average_per_day})

In [ ]:
london_seasonality_total_consumption = figure(title="London, United Kingdom",
                                              x_range=FactorRange(*x),
                                              x_axis_label="Day of the week",
                                              y_axis_label="Seasonal electric power [MW]",
                                              **FIGURE_TOOLS,
                                             )
london_seasonality_total_consumption.add_tools(electrical_power_use_season_tool_by_city[InsPireDataset.LONDON_UK])
l11 = london_seasonality_total_consumption.line(x="dayofweek", y=f"{InsPireDataset.LONDON_UK}_winter", source=europe_electricity_season_source, color=color_map["winter"], alpha=0.7, line_width=3)
l12 = london_seasonality_total_consumption.square(x="dayofweek", y=f"{InsPireDataset.LONDON_UK}_winter", source=europe_electricity_season_source, color=color_map["winter"], alpha=0.7, size=10)
l21 = london_seasonality_total_consumption.line(x="dayofweek", y=f"{InsPireDataset.LONDON_UK}_spring", source=europe_electricity_season_source, color=color_map["spring"], alpha=0.7,  line_width=3)
l22 = london_seasonality_total_consumption.circle(x="dayofweek", y=f"{InsPireDataset.LONDON_UK}_spring", source=europe_electricity_season_source, color=color_map["spring"], alpha=0.7,  size=10)
l31 = london_seasonality_total_consumption.line(x="dayofweek", y=f"{InsPireDataset.LONDON_UK}_summer", source=europe_electricity_season_source, color=color_map["summer"], alpha=0.7,  line_width=3)
l32 = london_seasonality_total_consumption.diamond(x="dayofweek", y=f"{InsPireDataset.LONDON_UK}_summer", source=europe_electricity_season_source, color=color_map["summer"], alpha=0.7,  size=10)
l41 = london_seasonality_total_consumption.line(x="dayofweek", y=f"{InsPireDataset.LONDON_UK}_fall", source=europe_electricity_season_source, color=color_map["fall"], alpha=0.7,  line_width=3)
l42 = london_seasonality_total_consumption.cross(x="dayofweek", y=f"{InsPireDataset.LONDON_UK}_fall", source=europe_electricity_season_source, color=color_map["fall"], alpha=0.7, size=10)
london_seasonality_total_consumption.add_layout(Legend(items=[("Winter 2017", [l11, l12]), ("Spring 2017", [l21, l22]), ("Summer 2017", [l31, l32]), ("Fall 2017", [l41, l42]),],), "below")


stuttgart_seasonality_total_consumption = figure(title="Stuttgart, Germany",
                                                 y_range=london_seasonality_total_consumption.y_range,
                                                 x_range=london_seasonality_total_consumption.x_range,
                                                 x_axis_label="Day of the week",
                                                 y_axis_label="Seasonal electric power [MW]",
                                                 **FIGURE_TOOLS,
                                                )
stuttgart_seasonality_total_consumption.add_tools(electrical_power_use_season_tool_by_city[InsPireDataset.STUTTGART_GER])
l11 = stuttgart_seasonality_total_consumption.line(x="dayofweek", y=f"{InsPireDataset.STUTTGART_GER}_winter", source=europe_electricity_season_source, color=color_map["winter"], alpha=0.7, line_width=3)
l12 = stuttgart_seasonality_total_consumption.square(x="dayofweek", y=f"{InsPireDataset.STUTTGART_GER}_winter", source=europe_electricity_season_source, color=color_map["winter"], alpha=0.7, size=10)
l21 = stuttgart_seasonality_total_consumption.line(x="dayofweek", y=f"{InsPireDataset.STUTTGART_GER}_spring", source=europe_electricity_season_source, color=color_map["spring"], alpha=0.7,  line_width=3)
l22 = stuttgart_seasonality_total_consumption.circle(x="dayofweek", y=f"{InsPireDataset.STUTTGART_GER}_spring", source=europe_electricity_season_source, color=color_map["spring"], alpha=0.7,  size=10)
l31 = stuttgart_seasonality_total_consumption.line(x="dayofweek", y=f"{InsPireDataset.STUTTGART_GER}_summer", source=europe_electricity_season_source, color=color_map["summer"], alpha=0.7,  line_width=3)
l32 = stuttgart_seasonality_total_consumption.diamond(x="dayofweek", y=f"{InsPireDataset.STUTTGART_GER}_summer", source=europe_electricity_season_source, color=color_map["summer"], alpha=0.7,  size=10)
l41 = stuttgart_seasonality_total_consumption.line(x="dayofweek", y=f"{InsPireDataset.STUTTGART_GER}_fall", source=europe_electricity_season_source, color=color_map["fall"], alpha=0.7,  line_width=3)
l42 = stuttgart_seasonality_total_consumption.cross(x="dayofweek", y=f"{InsPireDataset.STUTTGART_GER}_fall", source=europe_electricity_season_source, color=color_map["fall"], alpha=0.7, size=10)
stuttgart_seasonality_total_consumption.add_layout(Legend(items=[("Winter 2017", [l11, l12]), ("Spring 2017", [l21, l22]), ("Summer 2017", [l31, l32]), ("Fall 2017", [l41, l42]),],), "below")



rome_seasonality_total_consumption = figure(title="Rome, Italy",
                                            y_range=london_seasonality_total_consumption.y_range,
                                            x_range=london_seasonality_total_consumption.x_range,
                                            x_axis_label="Day of the week",
                                            y_axis_label="Seasonal electric power [MW]",
                                            **FIGURE_TOOLS,
                                           )
rome_seasonality_total_consumption.add_tools(electrical_power_use_season_tool_by_city[InsPireDataset.ROME_IT])
l11 = rome_seasonality_total_consumption.line(x="dayofweek", y=f"{InsPireDataset.ROME_IT}_winter", source=europe_electricity_season_source, color=color_map["winter"], alpha=0.7, line_width=3)
l12 = rome_seasonality_total_consumption.square(x="dayofweek", y=f"{InsPireDataset.ROME_IT}_winter", source=europe_electricity_season_source, color=color_map["winter"], alpha=0.7, size=10)
l21 = rome_seasonality_total_consumption.line(x="dayofweek", y=f"{InsPireDataset.ROME_IT}_spring", source=europe_electricity_season_source, color=color_map["spring"], alpha=0.7,  line_width=3)
l22 = rome_seasonality_total_consumption.circle(x="dayofweek", y=f"{InsPireDataset.ROME_IT}_spring", source=europe_electricity_season_source, color=color_map["spring"], alpha=0.7,  size=10)
l31 = rome_seasonality_total_consumption.line(x="dayofweek", y=f"{InsPireDataset.ROME_IT}_summer", source=europe_electricity_season_source, color=color_map["summer"], alpha=0.7,  line_width=3)
l32 = rome_seasonality_total_consumption.diamond(x="dayofweek", y=f"{InsPireDataset.ROME_IT}_summer", source=europe_electricity_season_source, color=color_map["summer"], alpha=0.7,  size=10)
l41 = rome_seasonality_total_consumption.line(x="dayofweek", y=f"{InsPireDataset.ROME_IT}_fall", source=europe_electricity_season_source, color=color_map["fall"], alpha=0.7,  line_width=3)
l42 = rome_seasonality_total_consumption.cross(x="dayofweek", y=f"{InsPireDataset.ROME_IT}_fall", source=europe_electricity_season_source, color=color_map["fall"], alpha=0.7, size=10)
rome_seasonality_total_consumption.add_layout(Legend(items=[("Winter 2017", [l11, l12]), ("Spring 2017", [l21, l22]), ("Summer 2017", [l31, l32]), ("Fall 2017", [l41, l42]),],), "below")


madrid_seasonality_total_consumption = figure(title=" Madrid, Spain",
                                              y_range=london_seasonality_total_consumption.y_range,
                                              x_range=london_seasonality_total_consumption.x_range,
                                              x_axis_label="Day of the week",
                                              y_axis_label="Seasonal electric power [MW]",
                                              **FIGURE_TOOLS,
                                             )
madrid_seasonality_total_consumption.add_tools(electrical_power_use_season_tool_by_city[InsPireDataset.MADRID_SPA])
l11 = madrid_seasonality_total_consumption.line(x="dayofweek", y=f"{InsPireDataset.MADRID_SPA}_winter", source=europe_electricity_season_source, color=color_map["winter"], alpha=0.7, line_width=3)
l12 = madrid_seasonality_total_consumption.square(x="dayofweek", y=f"{InsPireDataset.MADRID_SPA}_winter", source=europe_electricity_season_source, color=color_map["winter"], alpha=0.7, size=10)
l21 = madrid_seasonality_total_consumption.line(x="dayofweek", y=f"{InsPireDataset.MADRID_SPA}_spring", source=europe_electricity_season_source, color=color_map["spring"], alpha=0.7,  line_width=3)
l22 = madrid_seasonality_total_consumption.circle(x="dayofweek", y=f"{InsPireDataset.MADRID_SPA}_spring", source=europe_electricity_season_source, color=color_map["spring"], alpha=0.7,  size=10)
l31 = madrid_seasonality_total_consumption.line(x="dayofweek", y=f"{InsPireDataset.MADRID_SPA}_summer", source=europe_electricity_season_source, color=color_map["summer"], alpha=0.7,  line_width=3)
l32 = madrid_seasonality_total_consumption.diamond(x="dayofweek", y=f"{InsPireDataset.MADRID_SPA}_summer", source=europe_electricity_season_source, color=color_map["summer"], alpha=0.7,  size=10)
l41 = madrid_seasonality_total_consumption.line(x="dayofweek", y=f"{InsPireDataset.MADRID_SPA}_fall", source=europe_electricity_season_source, color=color_map["fall"], alpha=0.7,  line_width=3)
l42 = madrid_seasonality_total_consumption.cross(x="dayofweek", y=f"{InsPireDataset.MADRID_SPA}_fall", source=europe_electricity_season_source, color=color_map["fall"], alpha=0.7, size=10)
madrid_seasonality_total_consumption.add_layout(Legend(items=[("Winter 2017", [l11, l12]), ("Spring 2017", [l21, l22]), ("Summer 2017", [l31, l32]), ("Fall 2017", [l41, l42]),],), "below")



In [ ]:
show(layout([[london_seasonality_total_consumption, stuttgart_seasonality_total_consumption],[rome_seasonality_total_consumption, madrid_seasonality_total_consumption]])) 

## Heat Losses

### USA

In [ ]:
rochester_heat_loss_air_temperature = figure(title="Rochester, NY - Heat Loss",
                                          y_axis_label="Heat Loss",
                                          x_axis_label="Air Temperature [ºC]",
                                            **FIGURE_TOOLS,)
rochester_heat_loss_air_temperature.circle(x="air_temp", y="E_loss_tot", color=color_map["rochester"], alpha=0.2, source=us_source, view=rochester_view)


olympia_heat_loss_air_temperature = figure(title="Olympia, WA - Heat Loss",
                                          y_range=rochester_heat_loss_air_temperature.y_range,
                                          x_range=rochester_heat_loss_air_temperature.x_range,
                                          y_axis_label="Heat Loss",
                                          x_axis_label="Air Temperature [ºC]",
                                          **FIGURE_TOOLS,)
olympia_heat_loss_air_temperature.circle(x="air_temp", y="E_loss_tot", color=color_map["olympia"], alpha=0.2, source=us_source, view=olympia_view)


fresno_heat_loss_air_temperature = figure(title="Fresno, CA - Heat Loss",
                                          y_range=rochester_heat_loss_air_temperature.y_range,
                                          x_range=rochester_heat_loss_air_temperature.x_range,
                                          y_axis_label="Heat Loss",
                                          x_axis_label="Air Temperature [ºC]",
                                          **FIGURE_TOOLS,)
fresno_heat_loss_air_temperature.circle(x="air_temp", y="E_loss_tot", color=color_map["fresno"], alpha=0.2, source=us_source, view=fresno_view)


miami_heat_loss_air_temperature = figure(title="Miami, FL - Heat Loss",
                                         y_range=rochester_heat_loss_air_temperature.y_range,
                                         x_range=rochester_heat_loss_air_temperature.x_range,
                                         y_axis_label="Heat Loss",
                                         x_axis_label="Air Temperature [ºC]",
                                         **FIGURE_TOOLS,)
miami_heat_loss_air_temperature.circle(x="air_temp", y="E_loss_tot", color=color_map["miami"], alpha=0.2, source=us_source, view=miami_view)


all_usa_heat_loss_air_temperature = figure(title="Heat Losses in USA cities",
                                           y_axis_label="Heat Loss",
                                           x_axis_label="Air Temperature [ºC]",
                                          **{**FIGURE_TOOLS, **FULL_WIDTH_FIGURE_TOOLS})
all_usa_heat_loss_air_temperature.circle(x="air_temp", y="E_loss_tot", legend_label="Rochester, NY", color=color_map["rochester"], alpha=0.2, source=us_source, view=rochester_view)
all_usa_heat_loss_air_temperature.circle(x="air_temp", y="E_loss_tot", legend_label="Olympia, WA", color=color_map["olympia"], alpha=0.2, source=us_source, view=olympia_view)
all_usa_heat_loss_air_temperature.circle(x="air_temp", y="E_loss_tot", legend_label="Fresno, CA", color=color_map["fresno"], alpha=0.2, source=us_source, view=fresno_view)
all_usa_heat_loss_air_temperature.circle(x="air_temp", y="E_loss_tot", legend_label="Miami, FL", color=color_map["miami"], alpha=0.2, source=us_source, view=miami_view)



In [ ]:
show(layout([[all_usa_heat_loss_air_temperature],[rochester_heat_loss_air_temperature, olympia_heat_loss_air_temperature ,],[fresno_heat_loss_air_temperature, miami_heat_loss_air_temperature],]))

### Europe

In [ ]:
london_heat_loss_air_temperature = figure(title="London, United Kingdom - Heat Loss",
                                          y_axis_label="Heat Loss",
                                          x_axis_label="Air Temperature [ºC]",
                                            **FIGURE_TOOLS,)
london_heat_loss_air_temperature.circle(x="air_temp", y="E_loss_tot", color=color_map["london"], alpha=0.2, source=europe_source, view=london_view)


madrid_heat_loss_air_temperature = figure(title="Madrid, Spain - Heat Loss",
                                          y_range=london_heat_loss_air_temperature.y_range,
                                          x_range=london_heat_loss_air_temperature.x_range,
                                          y_axis_label="Heat Loss",
                                          x_axis_label="Air Temperature [ºC]",
                                          **FIGURE_TOOLS,)
madrid_heat_loss_air_temperature.circle(x="air_temp", y="E_loss_tot", color=color_map["madrid"], alpha=0.2, source=europe_source, view=madrid_view)


rome_heat_loss_air_temperature = figure(title="Rome, Italy - Heat Loss",
                                        y_range=london_heat_loss_air_temperature.y_range,
                                        x_range=london_heat_loss_air_temperature.x_range,
                                        y_axis_label="Heat Loss",
                                        x_axis_label="Air Temperature [ºC]",
                                          **FIGURE_TOOLS,)
rome_heat_loss_air_temperature.circle(x="air_temp", y="E_loss_tot", color=color_map["rome"], alpha=0.2, source=europe_source, view=rome_view)


stuttgart_heat_loss_air_temperature = figure(title="Stuttgart, Germany - Heat Loss",
                                             y_range=london_heat_loss_air_temperature.y_range,
                                             x_range=london_heat_loss_air_temperature.x_range,
                                             y_axis_label="Heat Loss",
                                             x_axis_label="Air Temperature [ºC]",
                                         **FIGURE_TOOLS,)
stuttgart_heat_loss_air_temperature.circle(x="air_temp", y="E_loss_tot", color=color_map["stuttgart"], alpha=0.2, source=europe_source, view=stuttgart_view)


all_europe_heat_loss_air_temperature = figure(title="Heat Losses in European cities",
                                              y_axis_label="Heat Loss",
                                              x_axis_label="Air Temperature [ºC]",
                                          **{**FIGURE_TOOLS, **FULL_WIDTH_FIGURE_TOOLS})
all_europe_heat_loss_air_temperature.circle(x="air_temp", y="E_loss_tot", legend_label="London, UK", color=color_map["london"], alpha=0.2, source=europe_source, view=london_view)
all_europe_heat_loss_air_temperature.circle(x="air_temp", y="E_loss_tot", legend_label="Stuttgart, Germany", color=color_map["stuttgart"], alpha=0.2, source=europe_source, view=stuttgart_view)
all_europe_heat_loss_air_temperature.circle(x="air_temp", y="E_loss_tot", legend_label="Rome, Italy", color=color_map["rome"], alpha=0.2, source=europe_source, view=rome_view)
all_europe_heat_loss_air_temperature.circle(x="air_temp", y="E_loss_tot", legend_label="Madrid, Spain", color=color_map["madrid"], alpha=0.2, source=europe_source, view=madrid_view)



In [ ]:
show(layout([[all_europe_heat_loss_air_temperature],[london_heat_loss_air_temperature, stuttgart_heat_loss_air_temperature ,],[rome_heat_loss_air_temperature, madrid_heat_loss_air_temperature],]))

## Waste heat potential by city
There are many aspects of potential use of waste heat sources. A theoretical potential reflects the maximum output of the source if working all the hours in a year at its max. capacity. Instead, when considering the temporal allocation, its maximum output is constrained by the actual hours operation in which the source can be exploited.
Moreover, the availability of waste heat does not necessarily match the heat demand, and therefore its practical use is limited by this mismatch.
Visualization can be found here: https://public.flourish.studio/visualisation/3742380/
Rome case: https://public.flourish.studio/visualisation/3742465/

In [ ]:
london['source1_cap'].max()*london['hourofyear'].max()

In [ ]:
source1_theoretical=[london['source1_cap'].max()*london['hourofyear'].max(), stuttgart['source1_cap'].max()*stuttgart['hourofyear'].max(), madrid['source1_cap'].max()*madrid['hourofyear'].max(), rome['source1_cap'].max()*rome['hourofyear'].max(), miami['source1_cap'].max()*miami['hourofyear'].max(), fresno['source1_cap'].max()*fresno['hourofyear'].max(), olympia['source1_cap'].max()*olympia['hourofyear'].max(), rochester['source1_cap'].max()*rochester['hourofyear'].max()]
source2_theoretical=[london['source2_cap'].max()*london['hourofyear'].max(), stuttgart['source2_cap'].max()*stuttgart['hourofyear'].max(), madrid['source2_cap'].max()*madrid['hourofyear'].max(), rome['source2_cap'].max()*rome['hourofyear'].max(), miami['source2_cap'].max()*miami['hourofyear'].max(), fresno['source2_cap'].max()*fresno['hourofyear'].max(), olympia['source2_cap'].max()*olympia['hourofyear'].max(), rochester['source2_cap'].max()*rochester['hourofyear'].max()]

source1_theoretical_y =pd.Series(source1_theoretical)
source2_theoretical_y =pd.Series(source2_theoretical)

source1_technical=[london['source1_cap'].sum(), stuttgart['source1_cap'].sum(), madrid['source1_cap'].sum(), rome['source1_cap'].sum(), miami['source1_cap'].sum(), fresno['source1_cap'].sum(), olympia['source1_cap'].sum(), rochester['source1_cap'].sum()]
source2_technical=[london['source2_cap'].sum(), stuttgart['source2_cap'].sum(), madrid['source2_cap'].sum(), rome['source2_cap'].sum(), miami['source2_cap'].sum(), fresno['source2_cap'].sum(), olympia['source2_cap'].sum(), rochester['source2_cap'].sum()]

source1_technical_y =pd.Series(source1_technical)
source2_technical_y =pd.Series(source2_technical)

source1_practical_y= [london['heat_source1'].sum(), stuttgart['heat_source1'].sum(), madrid['heat_source1'].sum(), rome['heat_source1'].sum(), miami['heat_source1'].sum(), fresno['heat_source1'].sum(), olympia['heat_source1'].sum(), rochester['heat_source1'].sum()]
source2_practical_y= [london['heat_source2'].sum(), stuttgart['heat_source2'].sum(), madrid['heat_source2'].sum(), rome['heat_source2'].sum(), miami['heat_source2'].sum(), fresno['heat_source2'].sum(), olympia['heat_source2'].sum(), rochester['heat_source2'].sum()]


source1_practical_y =pd.Series(source1_practical_y)
source2_practical_y =pd.Series(source2_practical_y)


theoretical_potential= source1_theoretical_y+ source2_theoretical_y
technical_potential= source1_technical_y+ source2_technical_y
practical_potential=source1_practical_y + source2_practical_y

print(theoretical_potential, technical_potential, practical_potential)


# Thanks for your attention!